<a href="https://colab.research.google.com/github/aliahmedd24/Predictive-Maintenance-LSTM/blob/main/pipeline_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb pandas numpy scikit-learn statsmodels scipy torch pmdarima confluent-kafka pyarrow matplotlib seaborn joblib

import os
import sys
from datetime import datetime

# Create project structure
!mkdir -p /content/predictive-maintenance/{src,data,configs,features,artifacts}
!mkdir -p /content/predictive-maintenance/src/{utils,ingestion,cleaning,windowing,features,scaling,monitoring,model_input,model}

# Change to project directory
os.chdir('/content/predictive-maintenance')
sys.path.append('/content/predictive-maintenance')

In [ ]:
%%writefile requirements.txt
wandb>=0.16.0
pandas>=1.5.0
numpy>=1.23.0
scikit-learn>=1.3.0
statsmodels>=0.14.0
scipy>=1.10.0
torch>=2.0.0
pmdarima>=2.0.0
confluent-kafka>=2.0.0
pyarrow>=10.0.0
matplotlib>=3.6.0
seaborn>=0.12.0
joblib>=1.2.0

In [ ]:
%%writefile configs/wandb.yaml
project: predictive-maintenance
entity: aliahmedibrahim24906-constructor-university
tags:
  - time-series
  - sensor-data
  - preprocessing
settings:
  console: "off"
  quiet: true

In [ ]:
%%writefile src/wandb_init.py
import wandb
import yaml
import os
from typing import Dict, Any

def init_wandb(config_path: str = "configs/wandb.yaml", run_name: str = None) -> wandb.run:
    """Initialize W&B with project configuration."""

    # Load config
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)

    # Set entity from environment if not specified
    if 'entity' not in config or config['entity'] == '${WANDB_ENTITY}':
        config['entity'] = os.getenv('WANDB_ENTITY', 'default-entity')

    # Initialize run
    run = wandb.init(
        project=config['project'],
        entity=config['entity'],
        name=run_name,
        tags=config.get('tags', []),
        config=config.get('settings', {})
    )

    return run

# Login with API key
def setup_wandb_colab():
    """Setup W&B for Google Colab environment."""
    from google.colab import userdata
    import wandb

    # Try to get API key from Colab secrets
    try:
        wandb_key = userdata.get('WANDB_API_KEY')
        wandb.login(key=wandb_key)
    except:
        # Fallback to manual login
        wandb.login()

    return True

In [ ]:
%%writefile src/ingestion/loader.py
import pandas as pd
import numpy as np
import wandb
import argparse
import logging
from pathlib import Path
from typing import List, Dict, Union
import pyarrow.parquet as pq

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DataLoader:
    """Handles data ingestion and time alignment with W&B logging."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")

    def load_csv_files(self, file_paths: List[str], timestamp_col: str = 'timestamp') -> pd.DataFrame:
        """Load and concatenate multiple CSV files."""
        dfs = []

        for path in file_paths:
            logger.info(f"Loading {path}")
            df = pd.read_csv(path)

            # Parse timestamp
            df[timestamp_col] = pd.to_datetime(df[timestamp_col])
            df = df.set_index(timestamp_col)

            # Log file stats
            self.run.log({
                f"file_{Path(path).stem}_rows": len(df),
                f"file_{Path(path).stem}_cols": len(df.columns)
            })

            dfs.append(df)

        # Combine all dataframes
        combined_df = pd.concat(dfs, axis=0)
        combined_df = combined_df.sort_index()

        return combined_df

    def resample_to_frequency(self, df: pd.DataFrame, freq: str = '1T') -> pd.DataFrame:
        """Resample data to uniform frequency."""
        logger.info(f"Resampling to {freq} frequency")

        # Log original stats
        self.run.log({
            "original_rows": len(df),
            "original_start": df.index.min().isoformat(),
            "original_end": df.index.max().isoformat()
        })

        # Resample numeric columns with mean, categorical with mode
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns

        resampled_numeric = df[numeric_cols].resample(freq).mean()

        if len(categorical_cols) > 0:
            resampled_categorical = df[categorical_cols].resample(freq).agg(lambda x: x.mode()[0] if len(x) > 0 else np.nan)
            resampled_df = pd.concat([resampled_numeric, resampled_categorical], axis=1)
        else:
            resampled_df = resampled_numeric

        # Log resampled stats
        self.run.log({
            "resampled_rows": len(resampled_df),
            "resampling_freq": freq,
            "data_loss_ratio": 1 - (len(resampled_df) / len(df))
        })

        return resampled_df

    def save_and_log_artifact(self, df: pd.DataFrame, output_path: str = "data/unified.parquet"):
        """Save DataFrame to Parquet and log as W&B artifact."""
        # Create directory if needed
        Path(output_path).parent.mkdir(parents=True, exist_ok=True)

        # Save to parquet
        df.to_parquet(output_path, engine='pyarrow')
        logger.info(f"Saved unified data to {output_path}")

        # Create and log W&B artifact
        artifact = wandb.Artifact("raw-data", type="dataset")
        artifact.add_file(output_path)

        # Add metadata
        artifact.metadata = {
            "rows": len(df),
            "columns": list(df.columns),
            "start_time": df.index.min().isoformat(),
            "end_time": df.index.max().isoformat(),
            "frequency": pd.infer_freq(df.index) or "irregular"
        }

        self.run.log_artifact(artifact)
        logger.info("Logged raw-data artifact to W&B")

def main():
    parser = argparse.ArgumentParser(description="Ingest and align time-series data")
    parser.add_argument("--input-files", nargs="+", required=True, help="Input CSV files")
    parser.add_argument("--timestamp-col", default="timestamp", help="Timestamp column name")
    parser.add_argument("--resample-freq", default="1T", help="Resampling frequency")
    parser.add_argument("--output-path", default="data/unified.parquet", help="Output path")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="data-ingestion")

    # Process data
    loader = DataLoader(run)
    df = loader.load_csv_files(args.input_files, args.timestamp_col)
    df = loader.resample_to_frequency(df, args.resample_freq)
    loader.save_and_log_artifact(df, args.output_path)

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/cleaning/quality_checks.py
import pandas as pd
import numpy as np
import wandb
import json
import logging
from pathlib import Path
from typing import Dict, Tuple, Any

logger = logging.getLogger(__name__)

class QualityChecker:
    """Performs initial data quality checks with W&B logging."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")

    def load_limits(self, limits_path: str = "configs/limits.json") -> Dict[str, Dict[str, float]]:
        """Load sensor limits configuration."""
        try:
            with open(limits_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            logger.warning(f"Limits file not found at {limits_path}, using defaults")
            return {}

    def remove_duplicates(self, df: pd.DataFrame) -> pd.DataFrame:
        """Remove duplicate timestamps."""
        initial_rows = len(df)
        df = df[~df.index.duplicated(keep='first')]

        duplicates_removed = initial_rows - len(df)
        self.run.log({"duplicates_removed": duplicates_removed})
        logger.info(f"Removed {duplicates_removed} duplicate rows")

        return df

    def filter_by_limits(self, df: pd.DataFrame, limits: Dict[str, Dict[str, float]]) -> pd.DataFrame:
        """Filter values outside physical limits."""
        out_of_spec_counts = {}

        for col in df.columns:
            if col in limits:
                min_val = limits[col].get('min', -np.inf)
                max_val = limits[col].get('max', np.inf)

                # Count out-of-spec values
                out_of_spec = (df[col] < min_val) | (df[col] > max_val)
                out_of_spec_counts[col] = out_of_spec.sum()

                # Replace with NaN
                df.loc[out_of_spec, col] = np.nan

        # Log out-of-spec counts
        for col, count in out_of_spec_counts.items():
            self.run.log({f"out_of_spec_{col}": count})

        return df

    def profile_missingness(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Profile missing values and generate report."""
        missingness_report = pd.DataFrame({
            'column': df.columns,
            'missing_count': df.isnull().sum().values,
            'missing_rate': (df.isnull().sum() / len(df)).values,
            'data_type': df.dtypes.astype(str).values
        })

        # Sort by missing rate
        missingness_report = missingness_report.sort_values('missing_rate', ascending=False)

        # Log missing rates to W&B
        missing_rate_dict = dict(zip(
            missingness_report['column'],
            missingness_report['missing_rate']
        ))

        # Log as table and metrics
        table = wandb.Table(dataframe=missingness_report)
        self.run.log({"missingness_table": table})

        for col, rate in missing_rate_dict.items():
            self.run.log({f"missing_rate_{col}": rate})

        # Log summary statistics
        self.run.log({
            "avg_missing_rate": missingness_report['missing_rate'].mean(),
            "max_missing_rate": missingness_report['missing_rate'].max(),
            "columns_with_missing": (missingness_report['missing_rate'] > 0).sum()
        })

        return df, missingness_report

    def save_quality_artifacts(self, df: pd.DataFrame, report: pd.DataFrame,
                              output_dir: str = "data"):
        """Save cleaned data and quality report as W&B artifacts."""
        # Save report
        report_path = f"{output_dir}/missingness_report.csv"
        report.to_csv(report_path, index=False)

        # Create quality metrics artifact
        artifact = wandb.Artifact("quality-metrics", type="dataset")
        artifact.add_file(report_path)

        # Add cleaned data stats to metadata
        artifact.metadata = {
            "total_rows": len(df),
            "total_columns": len(df.columns),
            "avg_missing_rate": float(report['missing_rate'].mean()),
            "columns_with_missing": int((report['missing_rate'] > 0).sum())
        }

        self.run.log_artifact(artifact)
        logger.info("Logged quality-metrics artifact to W&B")

        return df

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Perform quality checks on time-series data")
    parser.add_argument("--input-artifact", default="raw-data:latest", help="Input W&B artifact")
    parser.add_argument("--limits-file", default="configs/limits.json", help="Sensor limits file")
    parser.add_argument("--output-dir", default="data", help="Output directory")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="quality-checks")

    # Load data from artifact
    artifact = run.use_artifact(args.input_artifact)
    artifact_dir = artifact.download()

    # Find parquet file
    parquet_files = list(Path(artifact_dir).glob("*.parquet"))
    if not parquet_files:
        raise ValueError("No parquet files found in artifact")

    df = pd.read_parquet(parquet_files[0])

    # Run quality checks
    checker = QualityChecker(run)

    # Load limits
    # limits = checker.load_limits(args.limits_file)

    # Apply checks
    df = checker.remove_duplicates(df)
    df = checker.filter_by_limits(df, limits)
    df, report = checker.profile_missingness(df)

    # Save results
    checker.save_quality_artifacts(df, report, args.output_dir)

    # Save cleaned data
    df.to_parquet(f"{args.output_dir}/cleaned.parquet")

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
'''%%writefile configs/limits.json
{
    "sensor_0": {"min": 0, "max": 200},
    "sensor_1": {"min": 10, "max": 210},
    "sensor_2": {"min": 20, "max": 220},
    "sensor_3": {"min": 30, "max": 230},
    "sensor_4": {"min": 40, "max": 240}
}'''

In [ ]:
%%writefile src/utils/sensor_utils.py
"""
Centralized utility for identifying sensor columns as ANY numeric column.
This replaces the hardcoded 'sensor_' prefix requirement throughout the codebase.
"""
import pandas as pd
import numpy as np
from typing import List, Union


def get_sensor_columns(df: pd.DataFrame,
                      exclude_columns: List[str] = None,
                      ensure_numeric: bool = True) -> List[str]:
    """
    Identify sensor columns as ANY numeric column in the dataframe.

    This function replaces all hardcoded sensor column identification throughout
    the codebase to make it more flexible and not dependent on naming conventions.

    Args:
        df: Input dataframe
        exclude_columns: List of column names to exclude from sensor identification
        ensure_numeric: If True, only return columns with numeric dtypes

    Returns:
        List of column names identified as sensor columns
    """
    if exclude_columns is None:
        exclude_columns = [
            'timestamp', 'time', 'datetime', 'date',
            'window_id', 'window_start', 'window_end',
            'label', 'target', 'class', 'category',
            'id', 'index', 'row_id', 'sample_id',
            'step', 'phase', 'stage', 'mode',
            'quality', 'confidence', 'flag', 'status'
        ]

    # Get all column names
    all_columns = df.columns.tolist()

    # Filter out excluded columns (case insensitive and partial matching)
    sensor_candidates = []
    for col in all_columns:
        col_lower = col.lower()
        # Exclude if any excluded term is found in the column name
        exclude = False
        for excluded_term in exclude_columns:
            if excluded_term.lower() in col_lower:
                exclude = True
                break

        # Also exclude columns ending with metadata suffixes
        metadata_suffixes = ['_quality', '_confidence', '_flag', '_gap_flag',
                           '_is_spike', '_low_confidence', '_method', '_status']
        for suffix in metadata_suffixes:
            if col.endswith(suffix):
                exclude = True
                break

        if not exclude:
            sensor_candidates.append(col)

    # If ensure_numeric is True, filter to only numeric columns
    if ensure_numeric:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        sensor_columns = [col for col in sensor_candidates if col in numeric_cols]
    else:
        sensor_columns = sensor_candidates

    return sensor_columns


def get_sensor_columns_safe(df: pd.DataFrame,
                           exclude_columns: List[str] = None,
                           fallback_pattern: str = None) -> List[str]:
    """
    Safe version of get_sensor_columns with fallback to pattern matching.

    Args:
        df: Input dataframe
        exclude_columns: List of column names to exclude
        fallback_pattern: Fallback pattern to match if no numeric columns found

    Returns:
        List of column names identified as sensor columns
    """
    sensor_cols = get_sensor_columns(df, exclude_columns)

    # If no sensor columns found and fallback pattern provided
    if not sensor_cols and fallback_pattern:
        sensor_cols = [col for col in df.columns if fallback_pattern in col.lower()]

    return sensor_cols


def validate_sensor_columns(df: pd.DataFrame,
                          sensor_cols: List[str],
                          min_valid_ratio: float = 0.1) -> List[str]:
    """
    Validate that sensor columns have sufficient non-null data for processing.

    Args:
        df: Input dataframe
        sensor_cols: List of sensor column names
        min_valid_ratio: Minimum ratio of non-null values required

    Returns:
        List of validated sensor column names
    """
    validated_cols = []

    for col in sensor_cols:
        if col in df.columns:
            valid_ratio = df[col].notna().sum() / len(df)
            if valid_ratio >= min_valid_ratio:
                validated_cols.append(col)

    return validated_cols


def get_feature_columns(df: pd.DataFrame,
                       exclude_metadata: bool = True) -> List[str]:
    """
    Get feature columns for machine learning (excludes IDs and metadata).

    Args:
        df: Input dataframe
        exclude_metadata: Whether to exclude metadata columns

    Returns:
        List of feature column names suitable for ML
    """
    # Start with all sensor columns
    feature_cols = get_sensor_columns(df)

    if exclude_metadata:
        # Remove additional metadata columns that might be features
        metadata_patterns = ['_id', '_start', '_end', '_timestamp']
        feature_cols = [col for col in feature_cols
                       if not any(pattern in col.lower() for pattern in metadata_patterns)]

    return feature_cols


def log_sensor_column_info(df: pd.DataFrame,
                          sensor_cols: List[str],
                          wandb_run=None) -> dict:
    """
    Log information about identified sensor columns for debugging and monitoring.

    Args:
        df: Input dataframe
        sensor_cols: List of identified sensor columns
        wandb_run: Optional wandb run for logging

    Returns:
        Dictionary with sensor column statistics
    """
    stats = {
        'total_columns': len(df.columns),
        'sensor_columns': len(sensor_cols),
        'sensor_percentage': len(sensor_cols) / len(df.columns) * 100,
        'sensor_column_names': sensor_cols[:20],  # First 20 for logging
        'excluded_columns': [col for col in df.columns if col not in sensor_cols][:10]
    }

    # Calculate data availability for sensor columns
    if sensor_cols:
        sensor_data = df[sensor_cols]
        stats.update({
            'avg_null_percentage': sensor_data.isnull().mean().mean() * 100,
            'min_null_percentage': sensor_data.isnull().mean().min() * 100,
            'max_null_percentage': sensor_data.isnull().mean().max() * 100,
            'total_sensor_values': sensor_data.size,
            'total_null_values': sensor_data.isnull().sum().sum(),
        })

    if wandb_run:
        wandb_run.log({
            'sensor_identification_total_columns': stats['total_columns'],
            'sensor_identification_sensor_count': stats['sensor_columns'],
            'sensor_identification_percentage': stats['sensor_percentage'],
            'sensor_identification_avg_null_pct': stats.get('avg_null_percentage', 0)
        })

    return stats

In [ ]:
%%writefile src/cleaning/time_features.py
import pandas as pd
import numpy as np
import wandb
import matplotlib.pyplot as plt
from typing import Dict, List

class TimeFeatureEngineer:
    """Add time-based features with W&B logging."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")

    def add_cyclical_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add sin/cos encoded time features."""
        # Hour of day
        df['hour'] = df.index.hour
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

        # Day of week
        df['day_of_week'] = df.index.dayofweek
        df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

        # Day of month
        df['day_of_month'] = df.index.day
        df['dom_sin'] = np.sin(2 * np.pi * df['day_of_month'] / 31)
        df['dom_cos'] = np.cos(2 * np.pi * df['day_of_month'] / 31)

        # Month of year
        df['month'] = df.index.month
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

        # Log feature ranges
        for feat in ['hour_sin', 'hour_cos', 'dow_sin', 'dow_cos', 'dom_sin', 'dom_cos', 'month_sin', 'month_cos']:
            self.run.log({
                f"{feat}_min": df[feat].min(),
                f"{feat}_max": df[feat].max(),
                f"{feat}_mean": df[feat].mean()
            })

        return df

    def add_shift_labels(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add shift labels based on hour."""
        def get_shift(hour):
            if 6 <= hour < 14:
                return 'morning'
            elif 14 <= hour < 22:
                return 'evening'
            else:
                return 'night'

        df['shift'] = df.index.hour.map(get_shift)

        # Log shift distribution
        shift_counts = df['shift'].value_counts().to_dict()
        for shift, count in shift_counts.items():
            self.run.log({f"shift_{shift}_count": count})

        # One-hot encode shifts
        shift_dummies = pd.get_dummies(df['shift'], prefix='shift')
        df = pd.concat([df, shift_dummies], axis=1)

        return df

    def log_feature_distributions(self, df: pd.DataFrame):
        """Log histograms and distributions to W&B."""
        # Create histograms for cyclical features
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        axes = axes.ravel()

        cyclical_features = ['hour_sin', 'hour_cos', 'dow_sin', 'dow_cos',
                           'dom_sin', 'dom_cos', 'month_sin', 'month_cos']

        for idx, feat in enumerate(cyclical_features):
            axes[idx].hist(df[feat].dropna(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
            axes[idx].set_title(f'Distribution of {feat}')
            axes[idx].set_xlabel('Value')
            axes[idx].set_ylabel('Frequency')

            # Add vertical lines for mean and median
            mean_val = df[feat].mean()
            median_val = df[feat].median()
            axes[idx].axvline(mean_val, color='red', linestyle='--', alpha=0.7, label=f'Mean: {mean_val:.2f}')
            axes[idx].axvline(median_val, color='green', linestyle='--', alpha=0.7, label=f'Median: {median_val:.2f}')
            axes[idx].legend()

        plt.tight_layout()
        self.run.log({"cyclical_features_distribution": wandb.Image(fig)})
        plt.close()

        # Create shift distribution pie chart
        fig, ax = plt.subplots(figsize=(8, 8))
        shift_counts = df['shift'].value_counts()
        ax.pie(shift_counts.values, labels=shift_counts.index, autopct='%1.1f%%', startangle=90)
        ax.set_title('Distribution of Shifts')
        self.run.log({"shift_distribution": wandb.Image(fig)})
        plt.close()

        # Log feature statistics table
        feature_stats = []
        for feat in cyclical_features:
            stats = {
                'feature': feat,
                'mean': df[feat].mean(),
                'std': df[feat].std(),
                'min': df[feat].min(),
                'max': df[feat].max(),
                'nulls': df[feat].isnull().sum()
            }
            feature_stats.append(stats)

        stats_table = wandb.Table(dataframe=pd.DataFrame(feature_stats))
        self.run.log({"time_feature_statistics": stats_table})

    def add_time_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Main method to add all time features."""
        # Ensure we have a datetime index
        if not isinstance(df.index, pd.DatetimeIndex):
            raise ValueError("DataFrame must have a DatetimeIndex")

        # Add features
        df = self.add_cyclical_features(df)
        df = self.add_shift_labels(df)

        # Log distributions
        self.log_feature_distributions(df)

        # Log total features added
        time_features = ['hour', 'hour_sin', 'hour_cos', 'day_of_week', 'dow_sin', 'dow_cos',
                        'day_of_month', 'dom_sin', 'dom_cos', 'month', 'month_sin', 'month_cos',
                        'shift', 'shift_morning', 'shift_evening', 'shift_night']

        self.run.log({"time_features_added": len(time_features)})

        return df

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Add time features to sensor data")
    parser.add_argument("--input-file", default="data/cleaned.parquet", help="Input parquet file")
    parser.add_argument("--output-file", default="data/time_features.parquet", help="Output file")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="feature-engineering")

    # Load data
    df = pd.read_parquet(args.input_file)

    # Add time features
    engineer = TimeFeatureEngineer(run)
    df = engineer.add_time_features(df)

    # Save enhanced data
    df.to_parquet(args.output_file)

    # Log as artifact
    artifact = wandb.Artifact("time-features", type="dataset")
    artifact.add_file(args.output_file)
    run.log_artifact(artifact)

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/cleaning/impute.py
import pandas as pd
import numpy as np
import wandb
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt

# Import the new sensor column utility
from src.utils.sensor_utils import get_sensor_columns, log_sensor_column_info, validate_sensor_columns


class MissingValueImputer:
    """
    ROBUST Missing Value Imputation with comprehensive gap handling.

    Now uses DYNAMIC sensor column identification (any numeric column) instead of
    hardcoded 'sensor_' prefix requirement.

    Strategy:
    1. Short gaps (≤ short_limit): Linear interpolation
    2. Medium gaps (≤ medium_limit): Seasonal pattern matching
    3. Long gaps: Multiple fallback strategies
    4. Remaining NaNs: Forward fill, backward fill, then median

    This ensures NO NaN values remain for downstream processing.
    """

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.gap_stats = {}
        self.imputation_methods = {}
        self.sensor_statistics = {}

    def calculate_sensor_statistics(self, df: pd.DataFrame) -> Dict:
        """Pre-calculate statistics for each sensor for robust imputation."""
        stats = {}
        # Use new dynamic sensor column identification
        sensor_cols = get_sensor_columns(df)

        # Log sensor identification info
        log_sensor_column_info(df, sensor_cols, self.run)

        for col in sensor_cols:
            non_null_data = df[col].dropna()
            if len(non_null_data) > 0:
                stats[col] = {
                    'mean': float(non_null_data.mean()),
                    'median': float(non_null_data.median()),
                    'std': float(non_null_data.std()),
                    'min': float(non_null_data.min()),
                    'max': float(non_null_data.max()),
                    'q25': float(non_null_data.quantile(0.25)),
                    'q75': float(non_null_data.quantile(0.75)),
                    'valid_count': len(non_null_data),
                    'null_count': df[col].isnull().sum(),
                    'null_percentage': float(df[col].isnull().sum() / len(df) * 100)
                }
            else:
                # Completely null column - use global statistics
                stats[col] = {
                    'mean': 0.0,
                    'median': 0.0,
                    'std': 1.0,
                    'min': -1.0,
                    'max': 1.0,
                    'q25': -0.5,
                    'q75': 0.5,
                    'valid_count': 0,
                    'null_count': len(df),
                    'null_percentage': 100.0
                }

        self.sensor_statistics = stats
        return stats

    def identify_gap_segments(self, series: pd.Series) -> List[Dict]:
        """Identify continuous gaps in time series for targeted imputation."""
        null_mask = series.isnull()

        # Find gap start and end points
        gap_starts = null_mask & ~null_mask.shift(1).fillna(False)
        gap_ends = null_mask & ~null_mask.shift(-1).fillna(False)

        gaps = []
        start_indices = series[gap_starts].index
        end_indices = series[gap_ends].index

        for start_idx, end_idx in zip(start_indices, end_indices):
            gap_length = (series.index.get_loc(end_idx) - series.index.get_loc(start_idx)) + 1
            gaps.append({
                'start_idx': start_idx,
                'end_idx': end_idx,
                'length': gap_length,
                'start_pos': series.index.get_loc(start_idx),
                'end_pos': series.index.get_loc(end_idx)
            })

        return gaps

    def iterative_impute(self, df: pd.DataFrame, sensor_cols: List[str],
                        max_iter: int = 5) -> pd.DataFrame:
        """Apply multivariate iterative imputation for sensor columns."""
        try:
            # Only use columns with reasonable amount of data
            valid_cols = validate_sensor_columns(df, sensor_cols, min_valid_ratio=0.3)

            if len(valid_cols) < 2:
                self.run.log({"multivariate_skip_reason": "insufficient_valid_columns"})
                return df

            # Apply iterative imputation
            imputer = IterativeImputer(
                max_iter=max_iter,
                random_state=42,
                sample_posterior=False
            )

            imputed_data = imputer.fit_transform(df[valid_cols])
            df[valid_cols] = imputed_data

            self.run.log({
                "multivariate_imputation_applied": True,
                "multivariate_columns_processed": len(valid_cols)
            })

        except Exception as e:
            self.run.log({
                "multivariate_imputation_failed": str(e),
                "multivariate_fallback": "univariate_only"
            })

        return df

    def robust_impute_series(self, series: pd.Series, col_name: str,
                           short_limit: int = 5, medium_limit: int = 60) -> pd.Series:
        """Robust imputation strategy for a single sensor column."""
        imputed = series.copy()
        original_nulls = series.isnull().sum()
        methods_used = []

        if original_nulls == 0:
            return imputed

        # Initialize gap tracking
        if col_name not in self.gap_stats:
            self.gap_stats[col_name] = {
                'total_gaps': 0,
                'short_gaps': 0,
                'medium_gaps': 0,
                'long_gaps': 0
            }

        # Get sensor statistics
        sensor_stats = self.sensor_statistics.get(col_name, {})

        # Identify gap segments
        gaps = self.identify_gap_segments(series)

        for gap in gaps:
            gap_length = gap['length']
            start_pos = gap['start_pos']
            end_pos = gap['end_pos']

            # Update gap statistics
            self.gap_stats[col_name]['total_gaps'] += 1

            if gap_length <= short_limit:
                # SHORT GAPS: Linear interpolation
                self.gap_stats[col_name]['short_gaps'] += 1
                imputed.iloc[start_pos:end_pos+1] = imputed.iloc[start_pos:end_pos+1].interpolate(method='linear')
                methods_used.append('linear_interp')

            elif gap_length <= medium_limit:
                # MEDIUM GAPS: Seasonal/time-based interpolation
                self.gap_stats[col_name]['medium_gaps'] += 1

                # Try time-based interpolation if index is datetime
                if isinstance(imputed.index, pd.DatetimeIndex):
                    imputed.iloc[start_pos:end_pos+1] = imputed.iloc[start_pos:end_pos+1].interpolate(method='time')
                    methods_used.append('time_interp')
                else:
                    # Fallback to spline interpolation
                    try:
                        imputed.iloc[start_pos:end_pos+1] = imputed.iloc[start_pos:end_pos+1].interpolate(method='spline', order=2)
                        methods_used.append('spline_interp')
                    except:
                        imputed.iloc[start_pos:end_pos+1] = imputed.iloc[start_pos:end_pos+1].interpolate(method='linear')
                        methods_used.append('linear_fallback')

            else:
                # LONG GAPS: Multiple strategies
                self.gap_stats[col_name]['long_gaps'] += 1

                # Strategy 1: Try polynomial interpolation for shorter long gaps
                if gap_length <= 200:
                    try:
                        imputed.iloc[start_pos:end_pos+1] = imputed.iloc[start_pos:end_pos+1].interpolate(method='polynomial', order=3)
                        methods_used.append('polynomial_interp')
                    except:
                        # Fallback to median
                        median_val = sensor_stats.get('median', 0.0)
                        imputed.iloc[start_pos:end_pos+1] = median_val
                        methods_used.append('median_fill')
                else:
                    # Strategy 2: Bounded random walk for very long gaps
                    median_val = sensor_stats.get('median', 0.0)
                    std_val = sensor_stats.get('std', 1.0)

                    # Generate bounded random walk
                    random_values = np.random.normal(median_val, std_val * 0.5, gap_length)
                    # Bound the values to reasonable range
                    q25, q75 = sensor_stats.get('q25', median_val - std_val), sensor_stats.get('q75', median_val + std_val)
                    random_values = np.clip(random_values, q25, q75)

                    imputed.iloc[start_pos:end_pos+1] = random_values
                    methods_used.append('bounded_random')

        # FINAL CLEANUP: Handle any remaining NaNs
        remaining_nulls = imputed.isnull().sum()
        if remaining_nulls > 0:
            # Forward fill
            imputed = imputed.fillna(method='ffill')
            if imputed.isnull().sum() > 0:
                methods_used.append('forward_fill')

            # Backward fill
            imputed = imputed.fillna(method='bfill')
            if imputed.isnull().sum() > 0:
                methods_used.append('backward_fill')

            # Final fallback to median
            if imputed.isnull().sum() > 0:
                median_val = sensor_stats.get('median', 0.0)
                imputed = imputed.fillna(median_val)
                methods_used.append('zero_fill')

        final_nulls = imputed.isnull().sum()

        # Store methods used
        self.imputation_methods[col_name] = '+'.join(methods_used) if methods_used else 'none'

        # Log per-column statistics
        self.run.log({
            f'imputation_{col_name}_original_nulls': int(original_nulls),
            f'imputation_{col_name}_final_nulls': int(final_nulls),
            f'imputation_{col_name}_filled_points': int(original_nulls - final_nulls),
            f'imputation_{col_name}_null_percentage': float(original_nulls / len(series) * 100),
            f'imputation_{col_name}_methods_used': self.imputation_methods[col_name]
        })

        return imputed

    def impute_missing(self, df: pd.DataFrame,
                      short_limit: int = 5,
                      medium_limit: int = 60,
                      use_multivariate: bool = True,
                      drop_threshold: float = 80.0) -> pd.DataFrame:
        """
        Main imputation method - drops high-missing columns then ensures ALL remaining missing values are handled.

        Args:
            df: Input dataframe
            short_limit: Max gap for linear interpolation
            medium_limit: Max gap for time-based interpolation
            use_multivariate: Whether to try multivariate imputation first
            drop_threshold: Drop columns with more than this percentage of missing values (default 80%)

        Returns:
            DataFrame with NO missing values in remaining sensor columns
        """
        # Log initial state
        initial_shape = df.shape

        # Use new dynamic sensor column identification
        sensor_cols = get_sensor_columns(df)
        initial_nulls = df[sensor_cols].isnull().sum().sum() if sensor_cols else 0

        self.run.log({
            "imputation_initial_shape": str(initial_shape),
            "imputation_initial_nulls": int(initial_nulls),
            "imputation_initial_sensor_count": len(sensor_cols),
            "sensor_identification_method": "dynamic_numeric_columns"
        })

        if not sensor_cols:
            self.run.log({"imputation_warning": "No sensor columns found using dynamic identification"})
            return df

        # Calculate sensor statistics for all sensors
        self.calculate_sensor_statistics(df)

        # STEP 1: DROP columns with > drop_threshold% missing values
        columns_to_drop = []
        columns_to_keep = []
        dropped_stats = {}

        for col in sensor_cols:
            null_percentage = self.sensor_statistics[col]['null_percentage']
            if null_percentage > drop_threshold:
                columns_to_drop.append(col)
                dropped_stats[col] = {
                    'null_percentage': null_percentage,
                    'null_count': self.sensor_statistics[col]['null_count'],
                    'valid_count': self.sensor_statistics[col]['valid_count']
                }
            else:
                columns_to_keep.append(col)

        # Drop high-missing columns
        if columns_to_drop:
            df = df.drop(columns=columns_to_drop)

            # Log dropped columns
            self.run.log({
                "columns_dropped": len(columns_to_drop),
                "columns_dropped_list": ", ".join(columns_to_drop[:20]),  # First 20
                "columns_retained": len(columns_to_keep),
                "drop_threshold_used": drop_threshold
            })

            # Create table of dropped columns
            if dropped_stats:
                dropped_df = pd.DataFrame.from_dict(dropped_stats, orient='index')
                dropped_df.index.name = 'sensor'
                dropped_df.reset_index(inplace=True)
                dropped_table = wandb.Table(dataframe=dropped_df.head(50))
                self.run.log({"dropped_columns_details": dropped_table})

            print(f"Dropped {len(columns_to_drop)} columns with >{drop_threshold}% missing values")
            print(f"Retaining {len(columns_to_keep)} columns for imputation")

        # Update sensor_cols to only include retained columns
        sensor_cols = columns_to_keep

        if not sensor_cols:
            self.run.log({"imputation_warning": "No sensor columns remaining after dropping high-missing columns"})
            return df

        # Recalculate nulls after dropping
        remaining_nulls = df[sensor_cols].isnull().sum().sum()

        self.run.log({
            "nulls_after_dropping": int(remaining_nulls),
            "imputation_working_sensor_count": len(sensor_cols),
            "columns_removed_percentage": (len(columns_to_drop) / (len(columns_to_drop) + len(sensor_cols)) * 100) if columns_to_drop else 0
        })

        # Log sensors with moderate missing rates (for awareness)
        moderate_missing_sensors = [
            col for col in sensor_cols
            if 50 < self.sensor_statistics[col]['null_percentage'] <= drop_threshold
        ]
        if moderate_missing_sensors:
            self.run.log({
                "moderate_missing_sensors": len(moderate_missing_sensors),
                "moderate_missing_sensor_list": ", ".join(moderate_missing_sensors[:10])
            })

        # Step 2: Try multivariate imputation for remaining sensors
        if use_multivariate and len(sensor_cols) > 1:
            df = self.iterative_impute(df, sensor_cols)

        # Step 3: Robust univariate imputation for each remaining sensor
        for col in sensor_cols:
            if df[col].isnull().any():
                df[col] = self.robust_impute_series(
                    df[col],
                    col,
                    short_limit=short_limit,
                    medium_limit=medium_limit
                )

        # Step 4: Verify no NaNs remain in retained sensors
        remaining_nulls = df[sensor_cols].isnull().sum().sum()
        if remaining_nulls > 0:
            # Emergency fallback - should never reach here
            self.run.log({"emergency_fallback_triggered": True})
            for col in sensor_cols:
                if df[col].isnull().any():
                    # Use sensor mean or 0
                    fallback_value = self.sensor_statistics.get(col, {}).get('mean', 0)
                    df[col] = df[col].fillna(fallback_value)

        # Final verification
        final_nulls = df[sensor_cols].isnull().sum().sum()

        # Log results
        self.run.log({
            "imputation_final_nulls": int(final_nulls),
            "imputation_filled_total": int(remaining_nulls),
            "imputation_success": final_nulls == 0,
            "imputation_methods_summary": self.get_methods_summary(),
            "final_sensor_count": len(sensor_cols),
        })

        # Create detailed statistics
        self.log_imputation_stats()

        # Add imputation quality flags (for downstream awareness)
        df = self.add_imputation_quality_flags(df, sensor_cols)

        # Store list of dropped columns as metadata
        if columns_to_drop:
            df.attrs['dropped_columns'] = columns_to_drop
            df.attrs['drop_threshold'] = drop_threshold

        return df

    def add_imputation_quality_flags(self, df: pd.DataFrame, sensor_cols: List[str]) -> pd.DataFrame:
        """
        Add quality flags to indicate imputation confidence.
        These flags can be used downstream for weighted processing.
        """
        for col in sensor_cols:
            if col in self.gap_stats:
                stats = self.gap_stats[col]
                original_null_pct = stats.get('null_percentage', 0)

                # Create quality flag: 1.0 = no imputation, 0.0 = 100% imputed
                quality_score = 1.0 - (original_null_pct / 100.0)

                # Add as metadata column
                df[f"{col}_quality"] = quality_score

                # Add method flag
                method = self.imputation_methods.get(col, 'none')
                if 'bounded_random' in method or 'zero_fill' in method:
                    df[f"{col}_low_confidence"] = True
                else:
                    df[f"{col}_low_confidence"] = False

        return df

    def get_methods_summary(self) -> Dict[str, int]:
        """Summarize imputation methods used."""
        summary = {}
        for methods in self.imputation_methods.values():
            for method in methods.split('+'):
                if method:
                    summary[method] = summary.get(method, 0) + 1
        return summary

    def log_imputation_stats(self):
        """Log comprehensive imputation statistics."""
        # Create gap statistics table
        gap_data = []
        for col, stats in self.gap_stats.items():
            gap_data.append({
                'sensor': col,
                'total_gaps': stats['total_gaps'],
                'short_gaps': stats['short_gaps'],
                'medium_gaps': stats['medium_gaps'],
                'long_gaps': stats['long_gaps']
            })

        if gap_data:
            gap_df = pd.DataFrame(gap_data)
            gap_table = wandb.Table(dataframe=gap_df)
            self.run.log({"gap_statistics": gap_table})

        # Log methods summary
        methods_summary = self.get_methods_summary()
        for method, count in methods_summary.items():
            self.run.log({f"method_count_{method}": count})

    def create_imputation_visualization(self):
        """Create visualization of imputation impact."""
        if not self.gap_stats:
            return

        # Prepare data for visualization
        sensors = list(self.gap_stats.keys())[:20]  # First 20 sensors
        gap_counts = [self.gap_stats[s]['total_gaps'] for s in sensors]
        filled_points = [sum([
            self.gap_stats[s]['short_gaps'],
            self.gap_stats[s]['medium_gaps'],
            self.gap_stats[s]['long_gaps']
        ]) for s in sensors]

        # Create visualization
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Gap counts by sensor
        ax1.bar(range(len(sensors)), gap_counts)
        ax1.set_xlabel('Sensor')
        ax1.set_ylabel('Number of gaps')
        ax1.set_title('Gap Counts by Sensor')
        ax1.set_xticks(range(len(sensors)))
        ax1.set_xticklabels(sensors, rotation=45)

        # Points filled vs flagged
        x = np.arange(len(sensors))
        width = 0.35
        flagged = [sum([
            self.gap_stats[s].get('flagged_points', 0)
        ]) for s in sensors]

        ax2.bar(x - width/2, filled_points, width, label='Points filled')
        ax2.bar(x + width/2, flagged, width, label='Points flagged')
        ax2.set_xlabel('Sensor')
        ax2.set_ylabel('Number of points')
        ax2.set_title('Imputation Impact by Sensor')
        ax2.set_xticks(x)
        ax2.set_xticklabels(sensors, rotation=45)
        ax2.legend()

        plt.tight_layout()
        self.run.log({"imputation_visualization": wandb.Image(fig)})
        plt.close()


def main():
    import argparse

    parser = argparse.ArgumentParser(description="Impute missing values in sensor data")
    parser.add_argument("--input-file", default="data/time_features.parquet", help="Input file")
    parser.add_argument("--output-file", default="data/imputed.parquet", help="Output file")
    parser.add_argument("--short-limit", type=int, default=5, help="Max gap for linear interpolation")
    parser.add_argument("--medium-limit", type=int, default=60, help="Max gap for time-based interpolation")
    parser.add_argument("--drop-threshold", type=float, default=80.0, help="Drop columns with >% missing")
    parser.add_argument("--use-multivariate", action="store_true", help="Use multivariate imputation")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="imputation")

    # Load data
    print(f"Loading data from {args.input_file}...")
    df = pd.read_parquet(args.input_file)

    # Log initial information
    print(f"Initial shape: {df.shape}")

    # Use new dynamic sensor identification
    from src.utils.sensor_utils import get_sensor_columns
    sensor_cols = get_sensor_columns(df)
    print(f"Number of sensor columns (dynamic identification): {len(sensor_cols)}")

    if sensor_cols:
        initial_nulls = df[sensor_cols].isnull().sum().sum()
        print(f"Total missing values in sensors: {initial_nulls:,}")
        print(f"Average missing rate: {initial_nulls/(len(df)*len(sensor_cols))*100:.2f}%")

    # Perform robust imputation with column dropping
    print(f"\nDropping columns with >{args.drop_threshold}% missing values...")
    print("Then performing robust imputation on remaining columns...")

    imputer = MissingValueImputer(run)
    df = imputer.impute_missing(
        df,
        short_limit=args.short_limit,
        medium_limit=args.medium_limit,
        use_multivariate=args.use_multivariate,
        drop_threshold=args.drop_threshold
    )

    # Verify results
    print("\nImputation complete!")
    print(f"Final shape: {df.shape}")

    # Get remaining sensor columns using dynamic identification
    remaining_sensor_cols = get_sensor_columns(df)
    print(f"Remaining sensor columns: {len(remaining_sensor_cols)}")

    if remaining_sensor_cols:
        final_nulls = df[remaining_sensor_cols].isnull().sum().sum()
        print(f"Remaining NaNs in sensors: {final_nulls}")

        if final_nulls == 0:
            print("✅ SUCCESS: All missing values have been imputed in retained columns!")
        else:
            print("⚠️ WARNING: Some NaN values remain (this should not happen)")

    # Report dropped columns if any
    if hasattr(df, 'attrs') and 'dropped_columns' in df.attrs:
        dropped = df.attrs['dropped_columns']
        print(f"\n📊 Dropped {len(dropped)} columns due to >{args.drop_threshold}% missing values")
        if len(dropped) <= 10:
            print(f"   Dropped columns: {', '.join(dropped)}")
        else:
            print(f"   Dropped columns (first 10): {', '.join(dropped[:10])}...")

    # Save results
    print(f"\nSaving to {args.output_file}...")
    df.to_parquet(args.output_file)

    # Log as artifact
    artifact = wandb.Artifact("robust-imputed-data", type="dataset")
    artifact.add_file(args.output_file)
    artifact.metadata = {
        "short_gap_limit": args.short_limit,
        "medium_gap_limit": args.medium_limit,
        "drop_threshold": args.drop_threshold,
        "multivariate_used": args.use_multivariate,
        "final_shape": df.shape,
        "initial_sensors": len(sensor_cols),
        "final_sensors": len(remaining_sensor_cols),
        "columns_dropped": len(sensor_cols) - len(remaining_sensor_cols),
        "quality_flags_added": len([col for col in df.columns if col.endswith('_quality')]),
        "confidence_flags_added": len([col for col in df.columns if col.endswith('_low_confidence')]),
        "sensor_nulls_remaining": 0,  # Should always be 0
        "sensor_identification_method": "dynamic_numeric_columns"
    }
    run.log_artifact(artifact)

    run.finish()
    print("Done!")


if __name__ == "__main__":
    main()

In [ ]:
'''%%writefile src/cleaning/impute.py
import pandas as pd
import numpy as np
import wandb
from typing import Dict, Tuple

class MissingValueImputer:
    """Handle missing values with intelligent imputation and W&B logging."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.gap_stats = {}

    def identify_gaps(self, series: pd.Series) -> Dict[str, list]:
        """Identify continuous gaps in a series."""
        is_null = series.isnull()
        gap_groups = (is_null != is_null.shift()).cumsum()

        gaps = []
        for gap_id in gap_groups[is_null].unique():
            gap_mask = (gap_groups == gap_id) & is_null
            gap_length = gap_mask.sum()
            gap_start = gap_mask.idxmax()
            gaps.append({
                'start': gap_start,
                'length': gap_length,
                'gap_id': gap_id
            })

        return gaps

    def impute_series(self, series: pd.Series, short_limit: int = 5) -> Tuple[pd.Series, pd.Series]:
        """Impute a single series and return imputed series + gap flags."""
        gaps = self.identify_gaps(series)
        gap_flag = pd.Series(False, index=series.index, name=f"{series.name}_gap_flag")

        # Statistics
        total_gaps = len(gaps)
        short_gaps = sum(1 for g in gaps if g['length'] <= short_limit)
        long_gaps = total_gaps - short_gaps

        # Copy series for imputation
        imputed = series.copy()

        # Process each gap
        for gap in gaps:
            gap_slice = slice(gap['start'], gap['start'] + pd.Timedelta(minutes=gap['length']-1))

            if gap['length'] <= short_limit:
                # Interpolate short gaps
                imputed[gap_slice] = imputed[gap_slice].interpolate(method='time')
            else:
                # Flag long gaps
                gap_flag[gap_slice] = True

        # Store stats
        self.gap_stats[series.name] = {
            'total_gaps': total_gaps,
            'short_gaps': short_gaps,
            'long_gaps': long_gaps,
            'filled_points': (series.isnull() & ~imputed.isnull()).sum(),
            'flagged_points': gap_flag.sum()
        }

        return imputed, gap_flag

    def impute_missing(self, df: pd.DataFrame, short_limit: int = 5) -> pd.DataFrame:
        """Impute missing values in entire dataframe."""
        # Separate sensor columns from time features
        sensor_cols = [col for col in df.columns if col.startswith('sensor_')]

        # Store gap flags
        gap_flags = pd.DataFrame(index=df.index)

        # Impute each sensor column
        for col in sensor_cols:
            df[col], gap_flag = self.impute_series(df[col], short_limit)
            gap_flags[f"{col}_gap_flag"] = gap_flag

        # Add gap flags to dataframe
        df = pd.concat([df, gap_flags], axis=1)

        # Log imputation statistics
        self.log_imputation_stats()

        return df

    def log_imputation_stats(self):
        """Log detailed imputation statistics to W&B."""
        # Aggregate statistics
        total_filled = sum(stats['filled_points'] for stats in self.gap_stats.values())
        total_flagged = sum(stats['flagged_points'] for stats in self.gap_stats.values())
        total_gaps = sum(stats['total_gaps'] for stats in self.gap_stats.values())

        # Log overall metrics
        self.run.log({
            "total_gaps_found": total_gaps,
            "total_points_filled": total_filled,
            "total_points_flagged": total_flagged,
            "filled_gaps_ratio": total_filled / (total_filled + total_flagged) if (total_filled + total_flagged) > 0 else 0,
            "long_gaps_ratio": total_flagged / (total_filled + total_flagged) if (total_filled + total_flagged) > 0 else 0
        })

        # Log per-sensor metrics
        for sensor, stats in self.gap_stats.items():
            self.run.log({
                f"{sensor}_gaps": stats['total_gaps'],
                f"{sensor}_filled": stats['filled_points'],
                f"{sensor}_flagged": stats['flagged_points']
            })

        # Create detailed table
        stats_df = pd.DataFrame.from_dict(self.gap_stats, orient='index')
        stats_df.index.name = 'sensor'
        stats_df.reset_index(inplace=True)

        table = wandb.Table(dataframe=stats_df)
        self.run.log({"imputation_details": table})

        # Create visualization
        import matplotlib.pyplot as plt

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

        # Gap distribution
        sensors = list(self.gap_stats.keys())
        short_gaps = [self.gap_stats[s]['short_gaps'] for s in sensors]
        long_gaps = [self.gap_stats[s]['long_gaps'] for s in sensors]

        x = np.arange(len(sensors))
        width = 0.35

        ax1.bar(x - width/2, short_gaps, width, label='Short gaps (filled)')
        ax1.bar(x + width/2, long_gaps, width, label='Long gaps (flagged)')
        ax1.set_xlabel('Sensor')
        ax1.set_ylabel('Number of gaps')
        ax1.set_title('Gap Distribution by Sensor')
        ax1.set_xticks(x)
        ax1.set_xticklabels(sensors, rotation=45)
        ax1.legend()

        # Imputation impact
        filled = [self.gap_stats[s]['filled_points'] for s in sensors]
        flagged = [self.gap_stats[s]['flagged_points'] for s in sensors]

        ax2.bar(x - width/2, filled, width, label='Points filled')
        ax2.bar(x + width/2, flagged, width, label='Points flagged')
        ax2.set_xlabel('Sensor')
        ax2.set_ylabel('Number of points')
        ax2.set_title('Imputation Impact by Sensor')
        ax2.set_xticks(x)
        ax2.set_xticklabels(sensors, rotation=45)
        ax2.legend()

        plt.tight_layout()
        self.run.log({"imputation_visualization": wandb.Image(fig)})
        plt.close()

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Impute missing values in sensor data")
    parser.add_argument("--input-file", default="data/time_features.parquet", help="Input file")
    parser.add_argument("--output-file", default="data/imputed.parquet", help="Output file")
    parser.add_argument("--short-limit", type=int, default=5, help="Max gap size to interpolate")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="imputation")

    # Load data
    df = pd.read_parquet(args.input_file)

    # Impute missing values
    imputer = MissingValueImputer(run)
    df = imputer.impute_missing(df, args.short_limit)

    # Save results
    df.to_parquet(args.output_file)

    # Log as artifact
    artifact = wandb.Artifact("imputed-data", type="dataset")
    artifact.add_file(args.output_file)
    artifact.metadata = {
        "short_gap_limit": args.short_limit,
        "gap_flags_added": len([col for col in df.columns if col.endswith('_gap_flag')])
    }
    run.log_artifact(artifact)

    run.finish()

if __name__ == "__main__":
    main()'''

In [ ]:
%%writefile src/cleaning/outlier_flags.py
import pandas as pd
import numpy as np
import wandb
import json
from scipy import stats
from typing import Dict, Any
from src.utils.sensor_utils import get_sensor_columns, log_sensor_column_info

class OutlierFlagger:
    """Flag outliers without removal, with W&B logging."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.spike_stats = {}

    def load_config(self, config_path: str = "configs/outlier_config.json") -> Dict[str, Any]:
        """Load outlier detection configuration."""
        try:
            with open(config_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            # Default config
            return {
                "zscore_threshold": 3.5,
                "iqr_multiplier": 1.5,
                "min_spike_duration": 1,
                "use_rolling_stats": True,
                "rolling_window": 60
            }

    def flag_physical_limits(self, series: pd.Series, limits: Dict[str, float]) -> pd.Series:
        """Flag values outside physical limits."""
        flags = pd.Series(False, index=series.index)

        if 'min' in limits:
            flags |= series < limits['min']
        if 'max' in limits:
            flags |= series > limits['max']

        return flags

    def flag_statistical_spikes(self, series: pd.Series, config: Dict[str, Any]) -> pd.Series:
        """Flag statistical outliers using multiple methods."""
        flags = pd.Series(False, index=series.index)

        # Remove NaN for calculations
        clean_series = series.dropna()

        if len(clean_series) < 10:
            return flags

        # Method 1: Z-score
        if config.get('use_rolling_stats', False):
            # Rolling statistics for non-stationary data
            window = config.get('rolling_window', 60)
            rolling_mean = series.rolling(window=window, center=True).mean()
            rolling_std = series.rolling(window=window, center=True).std()
            z_scores = np.abs((series - rolling_mean) / rolling_std)
        else:
            # Global statistics
            z_scores = np.abs(stats.zscore(clean_series))
            z_score_flags = pd.Series(z_scores > config['zscore_threshold'],
                                    index=clean_series.index)
            flags.loc[clean_series.index] |= z_score_flags

        # Method 2: IQR
        Q1 = clean_series.quantile(0.25)
        Q3 = clean_series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - config['iqr_multiplier'] * IQR
        upper_bound = Q3 + config['iqr_multiplier'] * IQR

        iqr_flags = (clean_series < lower_bound) | (clean_series > upper_bound)
        flags.loc[clean_series.index] |= iqr_flags

        # Method 3: Isolation Forest (commented out for speed in basic version)
        # Can be added for more sophisticated detection

        return flags

    def flag_spikes(self, df: pd.DataFrame, config: Dict[str, Any]) -> pd.DataFrame:
        """Flag spikes in all sensor columns."""
        # Load physical limits
        limits = {}
        try:
            with open("configs/limits.json", 'r') as f:
                limits = json.load(f)
        except FileNotFoundError:
            pass

        # Process each sensor
        sensor_cols = get_sensor_columns(df)

        for col in sensor_cols:
            physical_flags = pd.Series(False, index=df.index)
            statistical_flags = pd.Series(False, index=df.index)

            # Check physical limits if available
            if col in limits:
                physical_flags = self.flag_physical_limits(df[col], limits[col])

            # Check statistical outliers
            statistical_flags = self.flag_statistical_spikes(df[col], config)

            # Combine flags
            df[f"{col}_is_spike"] = physical_flags | statistical_flags

            # Calculate statistics
            total_points = len(df)
            spike_points = df[f"{col}_is_spike"].sum()
            spike_rate = spike_points / total_points if total_points > 0 else 0

            self.spike_stats[col] = {
                'total_spikes': int(spike_points),
                'spike_rate': float(spike_rate),
                'physical_spikes': int(physical_flags.sum()),
                'statistical_spikes': int(statistical_flags.sum())
            }

            # Log per-sensor metrics
            self.run.log({
                f"spike_rate_{col}": spike_rate,
                f"spike_count_{col}": spike_points
            })

        # Log overall statistics
        self.log_spike_statistics(df)

        return df

    def log_spike_statistics(self, df: pd.DataFrame):
        """Log comprehensive spike statistics to W&B."""
        # Create summary table
        stats_df = pd.DataFrame.from_dict(self.spike_stats, orient='index')
        stats_df.index.name = 'sensor'
        stats_df.reset_index(inplace=True)

        table = wandb.Table(dataframe=stats_df)
        self.run.log({"spike_statistics": table})

        # Log aggregate metrics
        avg_spike_rate = np.mean([s['spike_rate'] for s in self.spike_stats.values()])
        total_spikes = sum(s['total_spikes'] for s in self.spike_stats.values())

        self.run.log({
            "avg_spike_rate_all_sensors": avg_spike_rate,
            "total_spikes_all_sensors": total_spikes
        })

        # Create visualization
        import matplotlib.pyplot as plt

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

        # Spike rates by sensor
        sensors = list(self.spike_stats.keys())
        spike_rates = [self.spike_stats[s]['spike_rate'] for s in sensors]

        ax1.bar(sensors, spike_rates, color='coral')
        ax1.set_xlabel('Sensor')
        ax1.set_ylabel('Spike Rate')
        ax1.set_title('Spike Rates by Sensor')
        ax1.tick_params(axis='x', rotation=45)

        # Add horizontal line for average
        ax1.axhline(y=avg_spike_rate, color='red', linestyle='--',
                   label=f'Average: {avg_spike_rate:.3f}')
        ax1.legend()

        # Spike types distribution
        physical_spikes = [self.spike_stats[s]['physical_spikes'] for s in sensors]
        statistical_spikes = [self.spike_stats[s]['statistical_spikes'] for s in sensors]

        x = np.arange(len(sensors))
        width = 0.35

        ax2.bar(x - width/2, physical_spikes, width, label='Physical limit violations')
        ax2.bar(x + width/2, statistical_spikes, width, label='Statistical outliers')
        ax2.set_xlabel('Sensor')
        ax2.set_ylabel('Number of spikes')
        ax2.set_title('Spike Types by Sensor')
        ax2.set_xticks(x)
        ax2.set_xticklabels(sensors, rotation=45)
        ax2.legend()

        plt.tight_layout()
        self.run.log({"spike_analysis": wandb.Image(fig)})
        plt.close()

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Flag outliers in sensor data")
    parser.add_argument("--input-file", default="data/imputed.parquet", help="Input file")
    parser.add_argument("--output-file", default="data/flagged.parquet", help="Output file")
    parser.add_argument("--config", default="configs/outlier_config.json", help="Config file")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="outlier-detection")

    # Load data
    df = pd.read_parquet(args.input_file)

    # Flag outliers
    flagger = OutlierFlagger(run)
    config = flagger.load_config(args.config)
    df = flagger.flag_spikes(df, config)

    # Save results
    df.to_parquet(args.output_file)

    # Log as artifact
    artifact = wandb.Artifact("flagged-data", type="dataset")
    artifact.add_file(args.output_file)
    artifact.metadata = {
        "outlier_config": config,
        "spike_flags_added": len([col for col in df.columns if col.endswith('_is_spike')])
    }
    run.log_artifact(artifact)

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/windowing/segment.py
import pandas as pd
import numpy as np
import wandb
from typing import Generator, Tuple, Optional
import random
from src.utils.sensor_utils import get_sensor_columns, log_sensor_column_info

class WindowSegmenter:
    """Create overlapping windows with W&B logging."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.window_stats = {}

    def generate_windows(self, df: pd.DataFrame, window_size: int = 60,
                        stride: int = 30) -> Generator[pd.DataFrame, None, None]:
        """Generate overlapping windows from dataframe."""
        # Ensure index is datetime
        if not isinstance(df.index, pd.DatetimeIndex):
            raise ValueError("DataFrame must have DatetimeIndex")

        # Calculate number of windows
        total_points = len(df)
        num_windows = (total_points - window_size) // stride + 1

        self.run.log({
            "total_windows": num_windows,
            "window_size": window_size,
            "stride": stride,
            "overlap_ratio": 1 - (stride / window_size)
        })

        # Generate windows
        windows_generated = 0
        for i in range(0, total_points - window_size + 1, stride):
            window = df.iloc[i:i + window_size].copy()
            window.attrs['window_id'] = windows_generated
            window.attrs['start_idx'] = i
            window.attrs['end_idx'] = i + window_size

            windows_generated += 1
            yield window

        # Store stats
        self.window_stats['total_windows'] = windows_generated
        self.window_stats['window_size'] = window_size
        self.window_stats['stride'] = stride

    def generate_baseline_windows(self, df: pd.DataFrame, window_size: int = 60,
                                 stride: int = 30, baseline_hours: list = None) -> Generator[pd.DataFrame, None, None]:
        """Generate windows only from baseline periods (e.g., night shift)."""
        if baseline_hours is None:
            baseline_hours = [22, 23, 0, 1, 2, 3, 4, 5]  # Night hours

        # Filter baseline periods
        baseline_mask = df.index.hour.isin(baseline_hours)
        baseline_df = df[baseline_mask]

        self.run.log({
            "baseline_data_points": len(baseline_df),
            "baseline_data_ratio": len(baseline_df) / len(df)
        })

        # Generate windows from baseline
        yield from self.generate_windows(baseline_df, window_size, stride)

    def log_sample_windows(self, df: pd.DataFrame, window_size: int = 60,
                          stride: int = 30, num_samples: int = 3):
        """Log sample windows to W&B for inspection."""
        # Collect a few windows
        windows = list(self.generate_windows(df, window_size, stride))

        if len(windows) < num_samples:
            num_samples = len(windows)

        # Random sample
        sample_indices = random.sample(range(len(windows)), num_samples)

        # Create table data
        table_data = []
        for idx in sample_indices:
            window = windows[idx]

            # Calculate window statistics
            sensor_cols = get_sensor_columns(df)

            for sensor in sensor_cols:
                stats = {
                    'window_id': window.attrs['window_id'],
                    'start_time': window.index[0].isoformat(),
                    'end_time': window.index[-1].isoformat(),
                    'sensor': sensor,
                    'mean': float(window[sensor].mean()),
                    'std': float(window[sensor].std()),
                    'min': float(window[sensor].min()),
                    'max': float(window[sensor].max()),
                    'missing_rate': float(window[sensor].isnull().sum() / len(window))
                }
                table_data.append(stats)

        # Log table
        table = wandb.Table(dataframe=pd.DataFrame(table_data))
        self.run.log({"sample_windows": table})

        # Create visualization of sample windows
        import matplotlib.pyplot as plt

        fig, axes = plt.subplots(num_samples, 1, figsize=(12, 4*num_samples))
        if num_samples == 1:
            axes = [axes]

        for i, idx in enumerate(sample_indices):
            window = windows[idx]
            sensor_cols = df.select_dtypes(include=[np.number]).columns.tolist()[:3]  # Plot first 3 sensors

            for sensor in sensor_cols:
                axes[i].plot(window.index, window[sensor], label=sensor, alpha=0.7)

            axes[i].set_title(f"Window {window.attrs['window_id']} - {window.index[0].strftime('%Y-%m-%d %H:%M')}")
            axes[i].set_xlabel('Time')
            axes[i].set_ylabel('Sensor Value')
            axes[i].legend()
            axes[i].grid(True, alpha=0.3)

        plt.tight_layout()
        self.run.log({"sample_windows_plot": wandb.Image(fig)})
        plt.close()

    def save_windows_metadata(self, output_path: str = "data/windows_metadata.csv"):
        """Save window generation metadata."""
        metadata_df = pd.DataFrame([self.window_stats])
        metadata_df.to_csv(output_path, index=False)

        # Log as artifact
        artifact = wandb.Artifact("windows-metadata", type="dataset")
        artifact.add_file(output_path)
        self.run.log_artifact(artifact)

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Generate windows from sensor data")
    parser.add_argument("--input-file", default="data/flagged.parquet", help="Input file")
    parser.add_argument("--window-size", type=int, default=60, help="Window size in minutes")
    parser.add_argument("--stride", type=int, default=30, help="Stride in minutes")
    parser.add_argument("--num-samples", type=int, default=3, help="Number of sample windows to log")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="windowing")

    # Load data
    df = pd.read_parquet(args.input_file)

    # Create windows
    segmenter = WindowSegmenter(run)

    # Log sample windows
    segmenter.log_sample_windows(df, args.window_size, args.stride, args.num_samples)

    # Save metadata
    segmenter.save_windows_metadata()

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/cleaning/window_clean.py
import pandas as pd
import numpy as np
import wandb
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
import joblib
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
from src.utils.sensor_utils import get_sensor_columns, log_sensor_column_info

class WindowCleaner:
    """Clean baseline windows and establish normal operating envelope."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.baseline_stats = {}
        self.models = {}

    def clean_window_mad(self, window: pd.DataFrame, threshold: float = 3.0) -> pd.DataFrame:
        """Clean window using Median Absolute Deviation."""
        cleaned_window = window.copy()

        # FIXED: Use sensor_utils instead of hardcoded df.select_dtypes()
        sensor_cols = get_sensor_columns(window)

        for col in sensor_cols:
            series = window[col].dropna()
            if len(series) < 10:
                continue

            median = series.median()
            mad = np.median(np.abs(series - median))

            # Modified Z-score using MAD
            if mad > 0:
                modified_z_scores = 0.6745 * (series - median) / mad
                outliers = np.abs(modified_z_scores) > threshold
                cleaned_window.loc[outliers.index[outliers], col] = np.nan

        return cleaned_window

    def clean_window_percentile(self, window: pd.DataFrame,
                               lower_percentile: float = 5.0,
                               upper_percentile: float = 95.0) -> pd.DataFrame:
        """Clean window using percentile-based outlier removal."""
        cleaned_window = window.copy()

        # Use dynamic sensor column identification
        sensor_cols = get_sensor_columns(window)

        for col in sensor_cols:
            series = window[col].dropna()
            if len(series) < 10:
                continue

            lower_bound = np.percentile(series, lower_percentile)
            upper_bound = np.percentile(series, upper_percentile)

            # Remove outliers
            outliers = (series < lower_bound) | (series > upper_bound)
            cleaned_window.loc[outliers.index[outliers], col] = np.nan

        return cleaned_window

    def fit_isolation_forest(self, windows: List[pd.DataFrame],
                           contamination: float = 0.1) -> IsolationForest:
        """Fit isolation forest on baseline windows."""
        # Use dynamic sensor column identification
        if not windows:
            return None

        sensor_cols = get_sensor_columns(windows[0])

        # Combine all windows
        combined_data = []
        for window in windows:
            window_data = window[sensor_cols].values
            # Reshape to 1D per window (feature vector)
            combined_data.append(window_data.flatten())

        combined_array = np.array(combined_data)

        # Fit isolation forest
        iso_forest = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_jobs=-1
        )
        iso_forest.fit(combined_array)

        self.models['isolation_forest'] = iso_forest

        # Log statistics
        self.run.log({
            "isolation_forest_contamination": contamination,
            "isolation_forest_features": combined_array.shape[1],
            "isolation_forest_samples": combined_array.shape[0]
        })

        return iso_forest

    def calculate_baseline_statistics(self, windows: List[pd.DataFrame]) -> Dict[str, Dict[str, float]]:
        """Calculate baseline statistics from clean windows."""
        if not windows:
            return {}

        # Use dynamic sensor column identification
        sensor_cols = get_sensor_columns(windows[0])

        stats = {}

        for col in sensor_cols:
            all_values = []
            for window in windows:
                values = window[col].dropna()
                all_values.extend(values.tolist())

            if all_values:
                col_array = np.array(all_values)
                stats[col] = {
                    'mean': float(np.mean(col_array)),
                    'std': float(np.std(col_array)),
                    'median': float(np.median(col_array)),
                    'q25': float(np.percentile(col_array, 25)),
                    'q75': float(np.percentile(col_array, 75)),
                    'min': float(np.min(col_array)),
                    'max': float(np.max(col_array))
                }

        return stats

    def clean_baseline_windows(self, df: pd.DataFrame, window_size: int = 60,
                             stride: int = 30, baseline_hours: list = None,
                             cleaning_method: str = 'mad') -> Tuple[List[pd.DataFrame], Dict]:
        """Clean baseline windows and return statistics."""
        from src.windowing.segment import WindowSegmenter

        # Generate baseline windows
        segmenter = WindowSegmenter(self.run)
        baseline_windows = list(segmenter.generate_baseline_windows(
            df, window_size, stride, baseline_hours
        ))

        self.run.log({"baseline_windows_count": len(baseline_windows)})

        # Clean windows
        cleaned_windows = []
        cleaning_impact = []

        for window in baseline_windows:
            if cleaning_method == 'mad':
                cleaned = self.clean_window_mad(window)
            elif cleaning_method == 'percentile':
                cleaned = self.clean_window_percentile(window)
            else:
                cleaned = window.copy()

            # Calculate cleaning impact using dynamic sensor identification
            sensor_cols = get_sensor_columns(window)

            if sensor_cols:
                original_nulls = window[sensor_cols].isnull().sum().sum()
                cleaned_nulls = cleaned[sensor_cols].isnull().sum().sum()
                points_removed = cleaned_nulls - original_nulls

                cleaning_impact.append({
                    'window_id': window.attrs.get('window_id', 0),
                    'points_removed': points_removed,
                    'removal_rate': points_removed / (len(window) * len(sensor_cols)) if sensor_cols else 0
                })

                cleaned_windows.append(cleaned)

        # Log cleaning impact
        if cleaning_impact:
            avg_removal_rate = np.mean([c['removal_rate'] for c in cleaning_impact])
            self.run.log({
                "avg_baseline_cleaning_rate": avg_removal_rate,
                "total_points_removed": sum(c['points_removed'] for c in cleaning_impact),
                "baseline_cleaning_method": cleaning_method
            })

        # Calculate baseline statistics
        self.baseline_stats = self.calculate_baseline_statistics(cleaned_windows)

        # Fit isolation forest for anomaly detection
        if len(cleaned_windows) > 5:
            self.fit_isolation_forest(cleaned_windows)

        return cleaned_windows, self.baseline_stats

    def save_models_and_stats(self, output_dir: str = "models"):
        """Save trained models and baseline statistics."""
        import os
        import json

        os.makedirs(output_dir, exist_ok=True)

        # Save baseline statistics
        stats_path = f"{output_dir}/baseline_stats.json"
        with open(stats_path, 'w') as f:
            json.dump(self.baseline_stats, f, indent=2)

        # Save isolation forest if available
        if 'isolation_forest' in self.models:
            model_path = f"{output_dir}/isolation_forest.joblib"
            joblib.dump(self.models['isolation_forest'], model_path)

        # Log as artifacts
        artifact = wandb.Artifact("baseline-models", type="model")
        artifact.add_file(stats_path)
        if 'isolation_forest' in self.models:
            artifact.add_file(model_path)
        self.run.log_artifact(artifact)

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Clean baseline windows")
    parser.add_argument("--input-file", default="data/flagged.parquet", help="Input file")
    parser.add_argument("--window-size", type=int, default=60, help="Window size")
    parser.add_argument("--stride", type=int, default=30, help="Stride")
    parser.add_argument("--method", choices=['mad', 'percentile'], default='mad', help="Cleaning method")
    parser.add_argument("--output-dir", default="models", help="Output directory")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="baseline-cleaning")

    # Load data
    df = pd.read_parquet(args.input_file)

    # Clean windows
    cleaner = WindowCleaner(run)
    cleaned_windows, stats = cleaner.clean_baseline_windows(
        df, args.window_size, args.stride, cleaning_method=args.method
    )

    # Save results
    cleaner.save_models_and_stats(args.output_dir)

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
'''%%writefile src/cleaning/window_clean.py
import pandas as pd
import numpy as np
import wandb
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
import joblib
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt

class WindowCleaner:
    """Clean baseline windows and establish normal operating envelope."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.baseline_stats = {}
        self.models = {}

    def clean_window_mad(self, window: pd.DataFrame, threshold: float = 3.0) -> pd.DataFrame:
        """Clean window using Median Absolute Deviation."""
        cleaned_window = window.copy()
        sensor_cols = [col for col in window.columns if col.startswith('sensor_')]

        for col in sensor_cols:
            series = window[col].dropna()
            if len(series) < 10:
                continue

            median = series.median()
            mad = np.median(np.abs(series - median))

            # Modified Z-score using MAD
            if mad > 0:
                modified_z_scores = 0.6745 * (series - median) / mad
                outliers = np.abs(modified_z_scores) > threshold
                cleaned_window.loc[outliers.index[outliers], col] = np.nan

        return cleaned_window

    def clean_window_percentile(self, window: pd.DataFrame,
                               lower_percentile: float = 1,
                               upper_percentile: float = 99) -> pd.DataFrame:
        """Clean window using percentile-based method."""
        cleaned_window = window.copy()
        sensor_cols = [col for col in window.columns if col.startswith('sensor_')]

        for col in sensor_cols:
            series = window[col].dropna()
            if len(series) < 10:
                continue

            lower_bound = series.quantile(lower_percentile / 100)
            upper_bound = series.quantile(upper_percentile / 100)

            mask = (series < lower_bound) | (series > upper_bound)
            cleaned_window.loc[mask.index[mask], col] = np.nan

        return cleaned_window

    def fit_isolation_forest(self, baseline_windows: List[pd.DataFrame],
                           contamination: float = 0.05) -> IsolationForest:
        """Fit Isolation Forest on baseline windows."""
        # Prepare data
        sensor_cols = [col for col in baseline_windows[0].columns if col.startswith('sensor_')]

        # Stack all baseline data
        baseline_data = []
        for window in baseline_windows:
            window_data = window[sensor_cols].dropna()
            if len(window_data) > 0:
                baseline_data.append(window_data)

        if not baseline_data:
            return None

        all_baseline = pd.concat(baseline_data, axis=0)

        # Fit Isolation Forest
        iso_forest = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_estimators=100
        )
        iso_forest.fit(all_baseline)

        # Calculate and log performance metrics
        predictions = iso_forest.predict(all_baseline)
        anomaly_ratio = (predictions == -1).sum() / len(predictions)

        self.run.log({
            "isolation_forest_anomaly_ratio": anomaly_ratio,
            "isolation_forest_n_samples": len(all_baseline)
        })

        return iso_forest

    def calculate_baseline_statistics(self, cleaned_windows: List[pd.DataFrame]) -> Dict:
        """Calculate statistics from cleaned baseline windows."""
        sensor_cols = [
            col for col in cleaned_windows[0].columns
            if col.startswith('sensor_') and not col.endswith('_is_spike') and not col.endswith('_gap_flag')
        ]
        stats = {}

        for col in sensor_cols:
            col_data = []
            for window in cleaned_windows:
                col_data.extend(window[col].dropna().values)

            if col_data:
                col_array = np.array(col_data)
                stats[col] = {
                    'mean': float(np.mean(col_array)),
                    'median': float(np.median(col_array)),
                    'std': float(np.std(col_array)),
                    'iqr': float(np.percentile(col_array, 75) - np.percentile(col_array, 25)),
                    'p5': float(np.percentile(col_array, 5)),
                    'p95': float(np.percentile(col_array, 95)),
                    'min': float(np.min(col_array)),
                    'max': float(np.max(col_array))
                }

        return stats

    def clean_baseline_windows(self, df: pd.DataFrame, window_size: int = 60,
                             stride: int = 30, baseline_hours: list = None,
                             cleaning_method: str = 'mad') -> Tuple[List[pd.DataFrame], Dict]:
        """Clean baseline windows and return statistics."""
        from src.windowing.segment import WindowSegmenter

        # Generate baseline windows
        segmenter = WindowSegmenter(self.run)
        baseline_windows = list(segmenter.generate_baseline_windows(
            df, window_size, stride, baseline_hours
        ))

        self.run.log({"baseline_windows_count": len(baseline_windows)})

        # Clean windows
        cleaned_windows = []
        cleaning_impact = []

        for window in baseline_windows:
            if cleaning_method == 'mad':
                cleaned = self.clean_window_mad(window)
            elif cleaning_method == 'percentile':
                cleaned = self.clean_window_percentile(window)
            else:
                cleaned = window.copy()

            # Calculate cleaning impact
            sensor_cols = [col for col in window.columns if col.startswith('sensor_')]
            original_nulls = window[sensor_cols].isnull().sum().sum()
            cleaned_nulls = cleaned[sensor_cols].isnull().sum().sum()
            points_removed = cleaned_nulls - original_nulls

            cleaning_impact.append({
                'window_id': window.attrs.get('window_id', 0),
                'points_removed': points_removed,
                'removal_rate': points_removed / (len(window) * len(sensor_cols))
            })

            cleaned_windows.append(cleaned)

        # Log cleaning impact
        avg_removal_rate = np.mean([c['removal_rate'] for c in cleaning_impact])
        self.run.log({
            "avg_baseline_cleaning_rate": avg_removal_rate,
            "total_points_removed": sum(c['points_removed'] for c in cleaning_impact)
        })

        # Calculate baseline statistics
        self.baseline_stats = self.calculate_baseline_statistics(cleaned_windows)

        # Fit models
        if len(cleaned_windows) > 10:
            self.models['isolation_forest'] = self.fit_isolation_forest(cleaned_windows)

        # Log baseline statistics
        self.log_baseline_statistics()

        return cleaned_windows, self.baseline_stats

    def log_baseline_statistics(self):
        """Log comprehensive baseline statistics to W&B."""
        # Create statistics table
        stats_data = []
        for sensor, stats in self.baseline_stats.items():
            row = {'sensor': sensor}
            row.update(stats)
            stats_data.append(row)

        stats_df = pd.DataFrame(stats_data)
        table = wandb.Table(dataframe=stats_df)
        self.run.log({"baseline_statistics": table})

        # Log individual metrics
        for sensor, stats in self.baseline_stats.items():
            self.run.log({
                f"baseline_{sensor}_median": stats['median'],
                f"baseline_{sensor}_iqr": stats['iqr']
            })

        # Create visualization
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        axes = axes.ravel()

        sensors = list(self.baseline_stats.keys())[:4]  # Plot first 4 sensors

        for i, sensor in enumerate(sensors):
            stats = self.baseline_stats[sensor]

            # Box plot representation
            ax = axes[i]
            positions = [1]
            box_data = [[stats['min'], stats['p5'], stats['median'], stats['p95'], stats['max']]]

            bp = ax.boxplot(box_data, positions=positions, widths=0.6,
                           patch_artist=True, showfliers=False)
            bp['boxes'][0].set_facecolor('lightblue')

            # Add mean line
            ax.hlines(stats['mean'], 0.7, 1.3, colors='red', linestyles='--', label='Mean')

            # Labels
            ax.set_ylabel('Value')
            ax.set_title(f'Baseline Statistics: {sensor}')
            ax.set_xticklabels([''])
            ax.legend()

            # Add text annotations
            ax.text(1.4, stats['max'], f"Max: {stats['max']:.2f}", fontsize=9)
            ax.text(1.4, stats['p95'], f"P95: {stats['p95']:.2f}", fontsize=9)
            ax.text(1.4, stats['median'], f"Median: {stats['median']:.2f}", fontsize=9)
            ax.text(1.4, stats['p5'], f"P5: {stats['p5']:.2f}", fontsize=9)
            ax.text(1.4, stats['min'], f"Min: {stats['min']:.2f}", fontsize=9)

        plt.tight_layout()
        self.run.log({"baseline_envelope": wandb.Image(fig)})
        plt.close()

    def save_models_and_stats(self, output_dir: str = "models"):
        """Save fitted models and statistics as W&B artifacts."""
        import os
        os.makedirs(output_dir, exist_ok=True)

        # Save Isolation Forest if fitted
        if 'isolation_forest' in self.models and self.models['isolation_forest'] is not None:
            model_path = f"{output_dir}/isolation_forest.joblib"
            joblib.dump(self.models['isolation_forest'], model_path)

            # Log as artifact
            artifact = wandb.Artifact("baseline-isolation-forest", type="model")
            artifact.add_file(model_path)
            self.run.log_artifact(artifact)

        # Save baseline statistics
        stats_path = f"{output_dir}/baseline_stats.json"
        import json
        with open(stats_path, 'w') as f:
            json.dump(self.baseline_stats, f, indent=2)

        # Log as artifact
        artifact = wandb.Artifact("baseline-statistics", type="dataset")
        artifact.add_file(stats_path)
        self.run.log_artifact(artifact)

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Clean baseline windows")
    parser.add_argument("--input-file", default="data/flagged.parquet", help="Input file")
    parser.add_argument("--window-size", type=int, default=60, help="Window size")
    parser.add_argument("--stride", type=int, default=30, help="Stride")
    parser.add_argument("--cleaning-method", default="mad", choices=["mad", "percentile"],
                       help="Cleaning method")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="baseline-cleaning")

    # Load data
    df = pd.read_parquet(args.input_file)

    # Clean baseline windows
    cleaner = WindowCleaner(run)
    cleaned_windows, stats = cleaner.clean_baseline_windows(
        df, args.window_size, args.stride, cleaning_method=args.cleaning_method
    )

    # Save models and statistics
    cleaner.save_models_and_stats()

    run.finish()

if __name__ == "__main__":
    main()'''

In [ ]:
'''%%writefile src/features/decompose.py
import pandas as pd
import numpy as np
import wandb
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional

class TimeSeriesDecomposer:
    """Perform seasonal and trend decomposition with W&B logging."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.decomposition_stats = {}

    def test_stationarity(self, series: pd.Series) -> Dict:
        """Perform Augmented Dickey-Fuller test for stationarity."""
        result = adfuller(series.dropna())

        return {
            'adf_statistic': float(result[0]),
            'p_value': float(result[1]),
            'critical_values': result[4],
            'is_stationary': result[1] < 0.05
        }

    def apply_stl(self, window: pd.DataFrame, period: int = 24,
                  seasonal: int = 7, trend: Optional[int] = None,
                  robust: bool = True) -> Dict[str, pd.DataFrame]:
        """Apply STL decomposition to window."""
        sensor_cols = [col for col in window.columns if col.startswith('sensor_')]
        decomposition_results = {}

        for col in sensor_cols:
            series = window[col].dropna()

            if len(series) < 2 * period:
                self.run.log({f"skipped_{col}_insufficient_data": True})
                continue

            # STL decomposition
            stl = STL(series, period=period, seasonal=seasonal,
                     trend=trend, robust=robust)
            result = stl.fit()

            # Store components
            decomposition_results[col] = pd.DataFrame({
                'original': series,
                'trend': result.trend,
                'seasonal': result.seasonal,
                'residual': result.resid
            })

            # Calculate component strengths
            var_total = np.var(series)
            var_resid = np.var(result.resid)
            var_seasonal = np.var(result.seasonal)

            seasonal_strength = max(0, 1 - var_resid / (var_resid + var_seasonal))
            trend_strength = max(0, 1 - var_resid / var_total)

            # Test stationarity of residuals
            residual_stationarity = self.test_stationarity(result.resid)

            # Store statistics
            self.decomposition_stats[col] = {
                'seasonal_strength': float(seasonal_strength),
                'trend_strength': float(trend_strength),
                'residual_variance': float(var_resid),
                'residual_stationary': residual_stationarity['is_stationary'],
                'residual_adf_pvalue': residual_stationarity['p_value']
            }

            # Log metrics
            self.run.log({
                f"{col}_seasonal_strength": seasonal_strength,
                f"{col}_trend_strength": trend_strength,
                f"{col}_residual_stationary": residual_stationarity['is_stationary']
            })

        return decomposition_results

    def create_decomposition_plot(self, decomposition: pd.DataFrame,
                                 sensor_name: str) -> plt.Figure:
        """Create decomposition visualization."""
        fig, axes = plt.subplots(4, 1, figsize=(12, 10))

        # Original
        axes[0].plot(decomposition.index, decomposition['original'], 'b-', alpha=0.7)
        axes[0].set_ylabel('Original')
        axes[0].set_title(f'STL Decomposition: {sensor_name}')
        axes[0].grid(True, alpha=0.3)

        # Trend
        axes[1].plot(decomposition.index, decomposition['trend'], 'r-', alpha=0.7)
        axes[1].set_ylabel('Trend')
        axes[1].grid(True, alpha=0.3)

        # Seasonal
        axes[2].plot(decomposition.index, decomposition['seasonal'], 'g-', alpha=0.7)
        axes[2].set_ylabel('Seasonal')
        axes[2].grid(True, alpha=0.3)

        # Residual
        axes[3].plot(decomposition.index, decomposition['residual'], 'm-', alpha=0.7)
        axes[3].set_ylabel('Residual')
        axes[3].set_xlabel('Time')
        axes[3].grid(True, alpha=0.3)

        # Add strength annotations
        if sensor_name in self.decomposition_stats:
            stats = self.decomposition_stats[sensor_name]
            fig.text(0.02, 0.98,
                    f"Seasonal Strength: {stats['seasonal_strength']:.3f}\n"
                    f"Trend Strength: {stats['trend_strength']:.3f}\n"
                    f"Residual Stationary: {stats['residual_stationary']}",
                    transform=fig.transFigure, fontsize=10,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        plt.tight_layout()
        return fig

    def decompose_windows(self, df: pd.DataFrame, window_size: int = 720,
                         stride: int = 360, num_examples: int = 3):
        """Decompose multiple windows and log examples."""
        from src.windowing.segment import WindowSegmenter

        segmenter = WindowSegmenter(self.run)
        windows = list(segmenter.generate_windows(df, window_size, stride))

        # Select example windows
        example_indices = np.linspace(0, len(windows)-1, num_examples, dtype=int)

        all_decompositions = []

        for i, window_idx in enumerate(example_indices):
            window = windows[window_idx]

            # Apply STL decomposition
            decomposition_results = self.apply_stl(window)
            all_decompositions.append(decomposition_results)

            # Create and log plots for first sensor
            if decomposition_results:
                first_sensor = list(decomposition_results.keys())[0]
                fig = self.create_decomposition_plot(
                    decomposition_results[first_sensor],
                    first_sensor
                )
                self.run.log({f"stl_plot_window_{i}": wandb.Image(fig)})
                plt.close()

        # Log decomposition statistics table
        stats_df = pd.DataFrame.from_dict(self.decomposition_stats, orient='index')
        stats_df.index.name = 'sensor'
        stats_df.reset_index(inplace=True)

        table = wandb.Table(dataframe=stats_df)
        self.run.log({"decomposition_statistics": table})

        # Create summary visualization
        self.create_summary_visualization()

        return all_decompositions

    def create_summary_visualization(self):
        """Create summary visualization of decomposition results."""
        if not self.decomposition_stats:
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

        # Seasonal vs Trend strength scatter
        sensors = list(self.decomposition_stats.keys())
        seasonal_strengths = [self.decomposition_stats[s]['seasonal_strength'] for s in sensors]
        trend_strengths = [self.decomposition_stats[s]['trend_strength'] for s in sensors]

        scatter = ax1.scatter(seasonal_strengths, trend_strengths,
                            c=range(len(sensors)), cmap='viridis', s=100, alpha=0.7)

        for i, sensor in enumerate(sensors):
            ax1.annotate(sensor, (seasonal_strengths[i], trend_strengths[i]),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)

        ax1.set_xlabel('Seasonal Strength')
        ax1.set_ylabel('Trend Strength')
        ax1.set_title('Component Strength Analysis')
        ax1.grid(True, alpha=0.3)
        ax1.set_xlim(-0.1, 1.1)
        ax1.set_ylim(-0.1, 1.1)

        # Stationarity results
        stationary_counts = sum(1 for s in self.decomposition_stats.values()
                              if s['residual_stationary'])
        non_stationary_counts = len(self.decomposition_stats) - stationary_counts

        ax2.pie([stationary_counts, non_stationary_counts],
               labels=['Stationary', 'Non-stationary'],
               autopct='%1.1f%%', startangle=90,
               colors=['lightgreen', 'lightcoral'])
        ax2.set_title('Residual Stationarity Results')

        plt.tight_layout()
        self.run.log({"decomposition_summary": wandb.Image(fig)})
        plt.close()

    def save_decomposition_artifacts(self, decompositions: list,
                                   output_dir: str = "features"):
        """Save decomposition results as artifacts."""
        import os
        import json

        os.makedirs(output_dir, exist_ok=True)

        # Save statistics
        stats_path = f"{output_dir}/decomposition_stats.json"
        with open(stats_path, 'w') as f:
            json.dump(self.decomposition_stats, f, indent=2)

        # Log as artifact
        artifact = wandb.Artifact("decomposition-stats", type="dataset")
        artifact.add_file(stats_path)
        artifact.metadata = {
            "num_sensors": len(self.decomposition_stats),
            "avg_seasonal_strength": float(np.mean([s['seasonal_strength']
                                                   for s in self.decomposition_stats.values()])),
            "stationary_ratio": sum(1 for s in self.decomposition_stats.values()
                                  if s['residual_stationary']) / len(self.decomposition_stats)
        }
        self.run.log_artifact(artifact)

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Decompose time series data")
    parser.add_argument("--input-file", default="data/flagged.parquet", help="Input file")
    parser.add_argument("--window-size", type=int, default=720, help="Window size for decomposition")
    parser.add_argument("--stride", type=int, default=360, help="Stride")
    parser.add_argument("--num-examples", type=int, default=3, help="Number of example plots")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="decomposition")

    # Load data
    df = pd.read_parquet(args.input_file)

    # Perform decomposition
    decomposer = TimeSeriesDecomposer(run)
    decompositions = decomposer.decompose_windows(
        df, args.window_size, args.stride, args.num_examples
    )

    # Save artifacts
    decomposer.save_decomposition_artifacts(decompositions)

    run.finish()

if __name__ == "__main__":
    main()'''

In [ ]:
%%writefile src/features/decompose.py
import pandas as pd
import numpy as np
import wandb
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
from src.utils.sensor_utils import get_sensor_columns, log_sensor_column_info
class TimeSeriesDecomposer:
    """Perform seasonal and trend decomposition with W&B logging."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.decomposition_stats = {}

    def test_stationarity(self, series: pd.Series) -> Dict:
        """Perform Augmented Dickey-Fuller test for stationarity."""
        # drop missing values
        series_clean = series.dropna()

        # if the series is constant or has zero variance, skip ADF
        if series_clean.nunique() <= 1 or np.var(series_clean) == 0:
            return {
                'adf_statistic': np.nan,
                'p_value': 1.0,
                'critical_values': {},
                'is_stationary': False
            }

        # run the ADF test
        result = adfuller(series_clean)

        return {
            'adf_statistic': float(result[0]),
            'p_value':        float(result[1]),
            'critical_values': result[4],
            'is_stationary':  bool(result[1] < 0.05)
        }

    def apply_stl(self, window: pd.DataFrame, period: int = 24,
                  seasonal: int = 7, trend: Optional[int] = None,
                  robust: bool = True) -> Dict[str, pd.DataFrame]:
        """Apply STL decomposition to window."""
        sensor_cols = get_sensor_columns(window)
        decomposition_results = {}

        for col in sensor_cols:
            series = window[col].dropna()

            if len(series) < 2 * period:
                self.run.log({f"skipped_{col}_insufficient_data": True})
                continue

            # STL decomposition
            stl = STL(series, period=period, seasonal=seasonal,
                     trend=trend, robust=robust)
            result = stl.fit()

            # Store components
            decomposition_results[col] = pd.DataFrame({
                'original': series,
                'trend': result.trend,
                'seasonal': result.seasonal,
                'residual': result.resid
            })

            # Calculate component strengths
            var_total = np.var(series)
            var_resid = np.var(result.resid)
            var_seasonal = np.var(result.seasonal)

            eps = 1e-8  # small constant to prevent division by zero

            # Log warnings if variances are zero
            if var_total == 0:
                self.run.log({f"{col}_zero_variance_total": True})
            if (var_resid + var_seasonal) == 0:
                self.run.log({f"{col}_zero_variance_resid_plus_seasonal": True})

            seasonal_strength = max(0, 1 - var_resid / (var_resid + var_seasonal + eps))
            trend_strength = max(0, 1 - var_resid / (var_total + eps))

            # Test stationarity of residuals
            residual_stationarity = self.test_stationarity(result.resid)

            # Store statistics
            self.decomposition_stats[col] = {
                'seasonal_strength': float(seasonal_strength),
                'trend_strength': float(trend_strength),
                'residual_variance': float(var_resid),
                'residual_stationary': residual_stationarity['is_stationary'],
                'residual_adf_pvalue': residual_stationarity['p_value']
            }

            # Log metrics
            self.run.log({
                f"{col}_seasonal_strength": seasonal_strength,
                f"{col}_trend_strength": trend_strength,
                f"{col}_residual_stationary": residual_stationarity['is_stationary']
            })

        return decomposition_results

    def create_decomposition_plot(self, decomposition: pd.DataFrame,
                                 sensor_name: str) -> plt.Figure:
        """Create decomposition visualization."""
        fig, axes = plt.subplots(4, 1, figsize=(12, 10))

        # Original
        axes[0].plot(decomposition.index, decomposition['original'], 'b-', alpha=0.7)
        axes[0].set_ylabel('Original')
        axes[0].set_title(f'STL Decomposition: {sensor_name}')
        axes[0].grid(True, alpha=0.3)

        # Trend
        axes[1].plot(decomposition.index, decomposition['trend'], 'r-', alpha=0.7)
        axes[1].set_ylabel('Trend')
        axes[1].grid(True, alpha=0.3)

        # Seasonal
        axes[2].plot(decomposition.index, decomposition['seasonal'], 'g-', alpha=0.7)
        axes[2].set_ylabel('Seasonal')
        axes[2].grid(True, alpha=0.3)

        # Residual
        axes[3].plot(decomposition.index, decomposition['residual'], 'm-', alpha=0.7)
        axes[3].set_ylabel('Residual')
        axes[3].set_xlabel('Time')
        axes[3].grid(True, alpha=0.3)

        # Add strength annotations
        if sensor_name in self.decomposition_stats:
            stats = self.decomposition_stats[sensor_name]
            fig.text(0.02, 0.98,
                    f"Seasonal Strength: {stats['seasonal_strength']:.3f}\n"
                    f"Trend Strength: {stats['trend_strength']:.3f}\n"
                    f"Residual Stationary: {stats['residual_stationary']}",
                    transform=fig.transFigure, fontsize=10,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        plt.tight_layout()
        return fig

    def decompose_windows(self, df: pd.DataFrame, window_size: int = 720,
                         stride: int = 360, num_examples: int = 3):
        """Decompose multiple windows and log examples."""
        from src.windowing.segment import WindowSegmenter

        segmenter = WindowSegmenter(self.run)
        windows = list(segmenter.generate_windows(df, window_size, stride))

        # Select example windows
        example_indices = np.linspace(0, len(windows)-1, num_examples, dtype=int)

        all_decompositions = []

        for i, window_idx in enumerate(example_indices):
            window = windows[window_idx]

            # Apply STL decomposition
            decomposition_results = self.apply_stl(window)
            all_decompositions.append(decomposition_results)

            # Create and log plots for first sensor
            if decomposition_results:
                first_sensor = list(decomposition_results.keys())[0]
                fig = self.create_decomposition_plot(
                    decomposition_results[first_sensor],
                    first_sensor
                )
                self.run.log({f"stl_plot_window_{i}": wandb.Image(fig)})
                plt.close()

        # Log decomposition statistics table
        stats_df = pd.DataFrame.from_dict(self.decomposition_stats, orient='index')
        stats_df.index.name = 'sensor'
        stats_df.reset_index(inplace=True)

        table = wandb.Table(dataframe=stats_df)
        self.run.log({"decomposition_statistics": table})

        # Create summary visualization
        self.create_summary_visualization()

        return all_decompositions

    def create_summary_visualization(self):
        """Create summary visualization of decomposition results."""
        if not self.decomposition_stats:
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

        # Seasonal vs Trend strength scatter
        sensors = list(self.decomposition_stats.keys())
        seasonal_strengths = [self.decomposition_stats[s]['seasonal_strength'] for s in sensors]
        trend_strengths = [self.decomposition_stats[s]['trend_strength'] for s in sensors]

        scatter = ax1.scatter(seasonal_strengths, trend_strengths,
                            c=range(len(sensors)), cmap='viridis', s=100, alpha=0.7)

        for i, sensor in enumerate(sensors):
            ax1.annotate(sensor, (seasonal_strengths[i], trend_strengths[i]),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)

        ax1.set_xlabel('Seasonal Strength')
        ax1.set_ylabel('Trend Strength')
        ax1.set_title('Component Strength Analysis')
        ax1.grid(True, alpha=0.3)
        ax1.set_xlim(-0.1, 1.1)
        ax1.set_ylim(-0.1, 1.1)

        # Stationarity results
        stationary_counts = sum(1 for s in self.decomposition_stats.values()
                              if s['residual_stationary'])
        non_stationary_counts = len(self.decomposition_stats) - stationary_counts

        ax2.pie([stationary_counts, non_stationary_counts],
               labels=['Stationary', 'Non-stationary'],
               autopct='%1.1f%%', startangle=90,
               colors=['lightgreen', 'lightcoral'])
        ax2.set_title('Residual Stationarity Results')

        plt.tight_layout()
        self.run.log({"decomposition_summary": wandb.Image(fig)})
        plt.close()

    def save_decomposition_artifacts(self, decompositions: list,
                                  output_dir: str = "features"):
        """Save decomposition results as artifacts."""
        import os
        import json

        os.makedirs(output_dir, exist_ok=True)

        # Save statistics
        stats_path = f"{output_dir}/decomposition_stats.json"
        with open(stats_path, 'w') as f:
            json.dump(self.decomposition_stats, f, indent=2)

        # Log as artifact
        artifact = wandb.Artifact("decomposition-stats", type="dataset")
        artifact.add_file(stats_path)
        artifact.metadata = {
            "num_sensors": len(self.decomposition_stats),
            "avg_seasonal_strength": float(np.mean([s['seasonal_strength']
                                                  for s in self.decomposition_stats.values()])),
            "stationary_ratio": sum(1 for s in self.decomposition_stats.values()
                                  if s['residual_stationary']) / len(self.decomposition_stats)
        }
        self.run.log_artifact(artifact)

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Decompose time series data")
    parser.add_argument("--input-file", default="data/flagged.parquet", help="Input file")
    parser.add_argument("--window-size", type=int, default=720, help="Window size for decomposition")
    parser.add_argument("--stride", type=int, default=360, help="Stride")
    parser.add_argument("--num-examples", type=int, default=3, help="Number of example plots")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="decomposition")

    # Load data
    df = pd.read_parquet(args.input_file)

    # Perform decomposition
    decomposer = TimeSeriesDecomposer(run)
    decompositions = decomposer.decompose_windows(
        df, args.window_size, args.stride, args.num_examples
    )

    # Save artifacts
    decomposer.save_decomposition_artifacts(decompositions)

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/features/extract.py
import pandas as pd
import numpy as np
import wandb
from scipy import stats
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Import the new sensor column utility
from src.utils.sensor_utils import get_sensor_columns, get_feature_columns, log_sensor_column_info


class FeatureExtractor:
    """
    Extract statistical and temporal features from sensor data windows.
    Now uses DYNAMIC sensor column identification instead of hardcoded 'sensor_' prefix.
    """

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.feature_stats = {}

    def extract_statistical_features(self, window: pd.DataFrame) -> Dict[str, float]:
        """Extract statistical features from a single window."""
        features = {}

        # Use dynamic sensor column identification
        sensor_cols = get_sensor_columns(window)

        for col in sensor_cols:
            data = window[col].dropna()

            if len(data) == 0:
                # Handle empty series
                prefix = col
                features.update({
                    f'{prefix}_mean': 0.0,
                    f'{prefix}_std': 0.0,
                    f'{prefix}_min': 0.0,
                    f'{prefix}_max': 0.0,
                    f'{prefix}_median': 0.0,
                    f'{prefix}_q25': 0.0,
                    f'{prefix}_q75': 0.0,
                    f'{prefix}_iqr': 0.0,
                    f'{prefix}_skew': 0.0,
                    f'{prefix}_kurt': 0.0,
                    f'{prefix}_valid_ratio': 0.0
                })
                continue

            prefix = col

            # Basic statistics
            features[f'{prefix}_mean'] = float(data.mean())
            features[f'{prefix}_std'] = float(data.std())
            features[f'{prefix}_min'] = float(data.min())
            features[f'{prefix}_max'] = float(data.max())
            features[f'{prefix}_median'] = float(data.median())

            # Quantiles
            features[f'{prefix}_q25'] = float(data.quantile(0.25))
            features[f'{prefix}_q75'] = float(data.quantile(0.75))
            features[f'{prefix}_iqr'] = features[f'{prefix}_q75'] - features[f'{prefix}_q25']

            # Distribution shape
            features[f'{prefix}_skew'] = float(data.skew())
            features[f'{prefix}_kurt'] = float(data.kurtosis())

            # Data quality
            features[f'{prefix}_valid_ratio'] = float(len(data) / len(window))

            # Range and variation
            features[f'{prefix}_range'] = features[f'{prefix}_max'] - features[f'{prefix}_min']
            features[f'{prefix}_cv'] = features[f'{prefix}_std'] / features[f'{prefix}_mean'] if features[f'{prefix}_mean'] != 0 else 0.0

            # Peak detection (simple)
            if len(data) > 5:
                features[f'{prefix}_peak_count'] = float(len(data[(data > data.quantile(0.95))]))
                features[f'{prefix}_valley_count'] = float(len(data[(data < data.quantile(0.05))]))
            else:
                features[f'{prefix}_peak_count'] = 0.0
                features[f'{prefix}_valley_count'] = 0.0

        return features

    def extract_temporal_features(self, window: pd.DataFrame) -> Dict[str, float]:
        """Extract temporal and trend features."""
        features = {}

        # Use dynamic sensor column identification
        sensor_cols = get_sensor_columns(window)

        for col in sensor_cols:
            data = window[col].dropna()

            if len(data) < 2:
                prefix = col
                features.update({
                    f'{prefix}_trend_slope': 0.0,
                    f'{prefix}_trend_r2': 0.0,
                    f'{prefix}_autocorr_lag1': 0.0,
                    f'{prefix}_zero_crossings': 0.0,
                    f'{prefix}_mean_abs_change': 0.0
                })
                continue

            prefix = col

            # Trend analysis
            if len(data) >= 2:
                x = np.arange(len(data))
                try:
                    slope, intercept, r_value, p_value, std_err = stats.linregress(x, data.values)
                    features[f'{prefix}_trend_slope'] = float(slope)
                    features[f'{prefix}_trend_r2'] = float(r_value ** 2)
                except:
                    features[f'{prefix}_trend_slope'] = 0.0
                    features[f'{prefix}_trend_r2'] = 0.0

            # Autocorrelation (lag-1)
            if len(data) > 2:
                try:
                    autocorr = data.autocorr(lag=1)
                    features[f'{prefix}_autocorr_lag1'] = float(autocorr) if not np.isnan(autocorr) else 0.0
                except:
                    features[f'{prefix}_autocorr_lag1'] = 0.0
            else:
                features[f'{prefix}_autocorr_lag1'] = 0.0

            # Zero crossings (relative to mean)
            mean_centered = data - data.mean()
            zero_crossings = ((mean_centered[:-1] * mean_centered[1:]) < 0).sum()
            features[f'{prefix}_zero_crossings'] = float(zero_crossings)

            # Mean absolute change
            if len(data) > 1:
                abs_changes = np.abs(data.diff()).dropna()
                features[f'{prefix}_mean_abs_change'] = float(abs_changes.mean()) if len(abs_changes) > 0 else 0.0
            else:
                features[f'{prefix}_mean_abs_change'] = 0.0

        return features

    def extract_frequency_features(self, window: pd.DataFrame) -> Dict[str, float]:
        """Extract frequency domain features using FFT."""
        features = {}

        # Use dynamic sensor column identification
        sensor_cols = get_sensor_columns(window)

        for col in sensor_cols:
            data = window[col].dropna()

            if len(data) < 8:  # Need minimum points for FFT
                prefix = col
                features.update({
                    f'{prefix}_dominant_freq': 0.0,
                    f'{prefix}_spectral_energy': 0.0,
                    f'{prefix}_spectral_centroid': 0.0,
                    f'{prefix}_spectral_rolloff': 0.0
                })
                continue

            prefix = col

            try:
                # Compute FFT
                fft_vals = np.fft.fft(data.values)
                fft_freq = np.fft.fftfreq(len(data))

                # Power spectrum
                power_spectrum = np.abs(fft_vals) ** 2

                # Dominant frequency
                dominant_freq_idx = np.argmax(power_spectrum[1:len(power_spectrum)//2]) + 1
                features[f'{prefix}_dominant_freq'] = float(np.abs(fft_freq[dominant_freq_idx]))

                # Spectral energy
                features[f'{prefix}_spectral_energy'] = float(np.sum(power_spectrum))

                # Spectral centroid
                freq_positive = fft_freq[:len(fft_freq)//2]
                power_positive = power_spectrum[:len(power_spectrum)//2]
                if np.sum(power_positive) > 0:
                    spectral_centroid = np.sum(freq_positive * power_positive) / np.sum(power_positive)
                    features[f'{prefix}_spectral_centroid'] = float(np.abs(spectral_centroid))
                else:
                    features[f'{prefix}_spectral_centroid'] = 0.0

                # Spectral rolloff (frequency below which 85% of energy is contained)
                cumsum_power = np.cumsum(power_positive)
                total_power = cumsum_power[-1]
                rolloff_idx = np.where(cumsum_power >= 0.85 * total_power)[0]
                if len(rolloff_idx) > 0:
                    features[f'{prefix}_spectral_rolloff'] = float(np.abs(freq_positive[rolloff_idx[0]]))
                else:
                    features[f'{prefix}_spectral_rolloff'] = 0.0

            except Exception as e:
                # Fallback values if FFT fails
                prefix = col
                features.update({
                    f'{prefix}_dominant_freq': 0.0,
                    f'{prefix}_spectral_energy': 0.0,
                    f'{prefix}_spectral_centroid': 0.0,
                    f'{prefix}_spectral_rolloff': 0.0
                })

        return features

    def extract_features(self, window: pd.DataFrame) -> pd.Series:
        """Extract all features from a single window."""
        features = {}

        # Use dynamic sensor column identification
        sensor_cols = get_sensor_columns(window)

        if not sensor_cols:
            # If no sensor columns found, return empty features
            return pd.Series(features)

        # Extract different types of features
        statistical_features = self.extract_statistical_features(window)
        temporal_features = self.extract_temporal_features(window)
        frequency_features = self.extract_frequency_features(window)

        # Combine all features
        features.update(statistical_features)
        features.update(temporal_features)
        features.update(frequency_features)

        # Add engineering features if we have at least one valid feature
        for feat_name, feat_val in features.items():
            if np.isnan(feat_val) or np.isinf(feat_val):
                features[feat_name] = 0.0
            else:
                features[feat_name] = feat_val

        # Window-level metadata features
        features['window_start'] = window.index[0].timestamp()
        features['window_hour'] = window.index[0].hour
        features['window_dayofweek'] = window.index[0].dayofweek

        # Cross-sensor features (if multiple sensors)
        if len(sensor_cols) > 1:
            # Correlation matrix
            sensor_data = window[sensor_cols].dropna()
            if len(sensor_data) > 10:
                try:
                    corr_matrix = sensor_data.corr()

                    # Extract upper triangle of correlation matrix
                    for i, col1 in enumerate(sensor_cols):
                        for j, col2 in enumerate(sensor_cols[i+1:], i+1):
                            corr_val = corr_matrix.loc[col1, col2]
                            if not np.isnan(corr_val):
                                features[f'corr_{col1}_{col2}'] = float(corr_val)
                            else:
                                features[f'corr_{col1}_{col2}'] = 0.0
                except:
                    # Skip correlation features if calculation fails
                    pass

        return pd.Series(features)

    def extract_window_features(self, df: pd.DataFrame, window_size: int = 60,
                              stride: int = 30) -> pd.DataFrame:
        """Extract features from all windows in the dataframe."""
        from src.windowing.segment import WindowSegmenter

        segmenter = WindowSegmenter(self.run)
        windows = list(segmenter.generate_windows(df, window_size, stride))

        self.run.log({
            "total_windows_for_extraction": len(windows),
            "feature_extraction_method": "dynamic_sensor_identification"
        })

        # Extract features for each window
        feature_list = []
        for i, window in enumerate(windows):
            window_features = self.extract_features(window)
            window_features['window_id'] = i
            feature_list.append(window_features)

            # Log progress
            if (i + 1) % 100 == 0:
                self.run.log({"features_extracted": i + 1})

        # Create feature matrix
        feature_matrix = pd.DataFrame(feature_list)

        # Calculate and log feature statistics
        self.calculate_feature_statistics(feature_matrix)

        return feature_matrix

    def calculate_feature_statistics(self, feature_matrix: pd.DataFrame):
        """Calculate and log statistics about extracted features."""
        # Basic statistics
        num_features = len(feature_matrix.columns) - 2  # Exclude window_id and window_start
        num_windows = len(feature_matrix)

        # Use dynamic sensor identification for logging
        sensor_cols = get_sensor_columns(feature_matrix)

        self.run.log({
            "feature_extraction_total_features": num_features,
            "feature_extraction_total_windows": num_windows,
            "feature_extraction_sensor_columns": len(sensor_cols),
            "feature_matrix_shape": str(feature_matrix.shape)
        })

        # Feature completeness
        completeness = feature_matrix.notna().mean()
        self.run.log({
            "feature_completeness_mean": float(completeness.mean()),
            "feature_completeness_min": float(completeness.min()),
            "features_with_missing": int((completeness < 1.0).sum())
        })

        # Identify potential feature types
        statistical_features = len([col for col in feature_matrix.columns if any(stat in col for stat in ['_mean', '_std', '_min', '_max'])])
        temporal_features = len([col for col in feature_matrix.columns if any(temp in col for temp in ['_trend', '_autocorr', '_change'])])
        frequency_features = len([col for col in feature_matrix.columns if any(freq in col for freq in ['_freq', '_spectral'])])
        correlation_features = len([col for col in feature_matrix.columns if 'corr_' in col])

        self.run.log({
            "statistical_features_count": statistical_features,
            "temporal_features_count": temporal_features,
            "frequency_features_count": frequency_features,
            "correlation_features_count": correlation_features
        })

        # Store statistics
        self.feature_stats = {
            'total_features': num_features,
            'total_windows': num_windows,
            'completeness_stats': completeness.describe().to_dict(),
            'feature_types': {
                'statistical': statistical_features,
                'temporal': temporal_features,
                'frequency': frequency_features,
                'correlation': correlation_features
            }
        }

    def save_feature_matrix(self, feature_matrix: pd.DataFrame, output_path: str):
        """
        Save the feature matrix to a CSV file and log it as a W&B artifact.
        Keeps logic minimal to match existing pipeline calls.
        """
        import os
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Save locally
        feature_matrix.to_csv(output_path, index=False)

        # Log as W&B artifact if a run is active
        try:
            import wandb
            artifact = wandb.Artifact(name=os.path.basename(output_path), type="dataset")
            artifact.add_file(output_path)
            self.run.log_artifact(artifact)
        except Exception as e:
            print(f"[Warning] Could not log feature matrix to W&B: {e}")

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Extract features from windowed sensor data")
    parser.add_argument("--input-file", default="data/imputed.parquet", help="Input file")
    parser.add_argument("--output-file", default="data/features.parquet", help="Output file")
    parser.add_argument("--window-size", type=int, default=60, help="Window size in minutes")
    parser.add_argument("--stride", type=int, default=30, help="Stride between windows in minutes")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="feature-extraction")

    # Load data
    print(f"Loading data from {args.input_file}...")
    df = pd.read_parquet(args.input_file)
    print(f"Loaded data shape: {df.shape}")

    # Log sensor column information using dynamic identification
    from src.utils.sensor_utils import get_sensor_columns, log_sensor_column_info
    sensor_cols = get_sensor_columns(df)
    sensor_info = log_sensor_column_info(df, sensor_cols, run)

    print(f"Identified {len(sensor_cols)} sensor columns using dynamic identification")
    print(f"Sensor columns: {sensor_cols[:10]}{'...' if len(sensor_cols) > 10 else ''}")

    # Extract features
    print(f"Extracting features with window_size={args.window_size}, stride={args.stride}...")
    extractor = FeatureExtractor(run)
    feature_matrix = extractor.extract_window_features(
        df,
        window_size=args.window_size,
        stride=args.stride
    )

    print(f"Extracted feature matrix shape: {feature_matrix.shape}")
    print(f"Total features per window: {len(feature_matrix.columns) - 2}")  # Exclude metadata columns

    # Save results
    print(f"Saving features to {args.output_file}...")
    feature_matrix.to_parquet(args.output_file)

    # Log as artifact
    artifact = wandb.Artifact("extracted-features", type="features")
    artifact.add_file(args.output_file)
    artifact.metadata = {
        "window_size": args.window_size,
        "stride": args.stride,
        "total_features": len(feature_matrix.columns) - 2,
        "total_windows": len(feature_matrix),
        "sensor_columns_used": len(sensor_cols),
        "sensor_identification_method": "dynamic_numeric_columns",
        "feature_types": extractor.feature_stats.get('feature_types', {}),
        "completeness_mean": extractor.feature_stats.get('completeness_stats', {}).get('mean', 0)
    }
    run.log_artifact(artifact)

    run.finish()
    print("Feature extraction completed!")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/scaling/scale.py
import pandas as pd
import numpy as np
import wandb
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
import joblib
import matplotlib.pyplot as plt
from typing import Tuple, Optional

# Import the new sensor column utility
from src.utils.sensor_utils import get_feature_columns, log_sensor_column_info


class FeatureScaler:
    """
    Scale features with robust methods and W&B tracking.
    Now uses DYNAMIC feature column identification instead of hardcoded patterns.
    """

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.scaler = None
        self.scaling_stats = {}

    def fit_scaler(self, baseline_features: pd.DataFrame,
                   method: str = 'robust') -> RobustScaler:
        """Fit scaler on baseline features using dynamic column identification."""
        # Use dynamic feature column identification
        feature_cols = get_feature_columns(baseline_features, exclude_metadata=True)

        # Log feature identification info
        log_sensor_column_info(baseline_features, feature_cols, self.run)

        self.run.log({
            "scaling_method": method,
            "scaling_feature_count": len(feature_cols),
            "scaling_identification_method": "dynamic_feature_columns"
        })

        # Select scaler
        if method == 'robust':
            self.scaler = RobustScaler()
        elif method == 'standard':
            self.scaler = StandardScaler()
        elif method == 'minmax':
            self.scaler = MinMaxScaler()
        else:
            raise ValueError(f"Unknown scaling method: {method}")

        # Fit on baseline data
        baseline_data = baseline_features[feature_cols]
        self.scaler.fit(baseline_data)

        # Calculate and log fit statistics
        if method == 'robust':
            centers = self.scaler.center_
            scales = self.scaler.scale_

            self.scaling_stats = {
                'method': method,
                'num_features': len(feature_cols),
                'median_center': float(np.median(centers)),
                'median_scale': float(np.median(scales)),
                'features_with_zero_scale': int(np.sum(scales == 0))
            }
        elif method == 'standard':
            means = self.scaler.mean_
            stds = self.scaler.scale_

            self.scaling_stats = {
                'method': method,
                'num_features': len(feature_cols),
                'mean_center': float(np.mean(means)),
                'mean_scale': float(np.mean(stds)),
                'features_with_zero_scale': int(np.sum(stds == 0))
            }
        elif method == 'minmax':
            mins = self.scaler.data_min_
            maxs = self.scaler.data_max_
            ranges = maxs - mins

            self.scaling_stats = {
                'method': method,
                'num_features': len(feature_cols),
                'mean_min': float(np.mean(mins)),
                'mean_max': float(np.mean(maxs)),
                'mean_range': float(np.mean(ranges)),
                'features_with_zero_range': int(np.sum(ranges == 0))
            }

        # Log scaler statistics
        for key, value in self.scaling_stats.items():
            self.run.log({f"scaler_{key}": value})

        return self.scaler

    def transform_features(self, features: pd.DataFrame) -> pd.DataFrame:
        """Transform features using fitted scaler with dynamic column identification."""
        if self.scaler is None:
            raise ValueError("Scaler must be fitted before transforming")

        # Use dynamic feature column identification
        feature_cols = get_feature_columns(features, exclude_metadata=True)

        # Create copy for transformation
        scaled_features = features.copy()

        try:
            # Transform only the feature columns
            feature_data = features[feature_cols]
            scaled_data = self.scaler.transform(feature_data)
            scaled_features[feature_cols] = scaled_data

            # Log transformation statistics
            self.run.log({
                "transform_feature_count": len(feature_cols),
                "transform_sample_count": len(features),
                "transform_total_values": len(features) * len(feature_cols)
            })

            # Check for any scaling issues
            scaled_means = np.mean(scaled_data, axis=0)
            scaled_stds = np.std(scaled_data, axis=0)

            self.run.log({
                "post_scaling_mean_of_means": float(np.mean(scaled_means)),
                "post_scaling_mean_of_stds": float(np.mean(scaled_stds)),
                "post_scaling_inf_count": int(np.isinf(scaled_data).sum()),
                "post_scaling_nan_count": int(np.isnan(scaled_data).sum())
            })

        except Exception as e:
            self.run.log({
                "scaling_transform_error": str(e),
                "scaling_fallback": "identity_transform"
            })
            # Return original features if scaling fails
            return features

        return scaled_features

    def apply_scaler(self, features: pd.DataFrame) -> pd.DataFrame:
        """Apply the fitted scaler to the given features DataFrame."""
        if self.scaler is None:
            raise ValueError("Scaler must be fitted before applying.")

        # Use dynamic feature column identification
        feature_cols = get_feature_columns(features, exclude_metadata=True)

        # Transform only the feature columns and keep structure intact
        scaled_array = self.scaler.transform(features[feature_cols])
        scaled_df = features.copy()
        scaled_df[feature_cols] = scaled_array
        return scaled_df


    def fit_transform(self, baseline_features: pd.DataFrame,
                     method: str = 'robust') -> Tuple[pd.DataFrame, RobustScaler]:
        """Fit scaler and transform baseline features in one step."""
        self.fit_scaler(baseline_features, method)
        scaled_features = self.transform_features(baseline_features)
        return scaled_features, self.scaler

    def save_scaler(self, filepath: str = "models/feature_scaler.joblib"):
        """Save fitted scaler to disk."""
        if self.scaler is None:
            raise ValueError("No scaler to save - fit a scaler first")

        joblib.dump(self.scaler, filepath)

        # Log scaler artifact
        artifact = wandb.Artifact("feature-scaler", type="model")
        artifact.add_file(filepath)
        artifact.metadata = self.scaling_stats
        self.run.log_artifact(artifact)

        self.run.log({"scaler_saved": filepath})

    def load_scaler(self, filepath: str = "models/feature_scaler.joblib"):
        """Load scaler from disk."""
        self.scaler = joblib.load(filepath)
        self.run.log({"scaler_loaded": filepath})

    def create_scaling_visualization(self, original_features: pd.DataFrame,
                                  scaled_features: pd.DataFrame):
        """Create visualization comparing original and scaled features."""
        # Use dynamic feature column identification
        feature_cols = get_feature_columns(original_features, exclude_metadata=True)

        if len(feature_cols) == 0:
            return

        # Select subset of features for visualization
        viz_features = feature_cols[:12]  # First 12 features

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Original feature distributions
        original_data = original_features[viz_features]
        axes[0, 0].boxplot([original_data[col].dropna() for col in viz_features])
        axes[0, 0].set_title('Original Feature Distributions')
        axes[0, 0].set_xticklabels(viz_features, rotation=45)
        axes[0, 0].set_ylabel('Values')

        # Scaled feature distributions
        scaled_data = scaled_features[viz_features]
        axes[0, 1].boxplot([scaled_data[col].dropna() for col in viz_features])
        axes[0, 1].set_title('Scaled Feature Distributions')
        axes[0, 1].set_xticklabels(viz_features, rotation=45)
        axes[0, 1].set_ylabel('Scaled Values')

        # Feature means comparison
        original_means = original_data.mean()
        scaled_means = scaled_data.mean()
        x_pos = np.arange(len(viz_features))
        width = 0.35

        axes[1, 0].bar(x_pos - width/2, original_means, width, label='Original', alpha=0.7)
        axes[1, 0].bar(x_pos + width/2, scaled_means, width, label='Scaled', alpha=0.7)
        axes[1, 0].set_title('Feature Means Comparison')
        axes[1, 0].set_xticks(x_pos)
        axes[1, 0].set_xticklabels(viz_features, rotation=45)
        axes[1, 0].legend()

        # Feature standard deviations comparison
        original_stds = original_data.std()
        scaled_stds = scaled_data.std()

        axes[1, 1].bar(x_pos - width/2, original_stds, width, label='Original', alpha=0.7)
        axes[1, 1].bar(x_pos + width/2, scaled_stds, width, label='Scaled', alpha=0.7)
        axes[1, 1].set_title('Feature Standard Deviations Comparison')
        axes[1, 1].set_xticks(x_pos)
        axes[1, 1].set_xticklabels(viz_features, rotation=45)
        axes[1, 1].legend()

        plt.tight_layout()
        self.run.log({"scaling_visualization": wandb.Image(fig)})
        plt.close()

    def get_feature_importance_by_scale(self, features: pd.DataFrame) -> pd.DataFrame:
        """Analyze which features are most affected by scaling."""
        if self.scaler is None:
            raise ValueError("Scaler must be fitted first")

        # Use dynamic feature column identification
        feature_cols = get_feature_columns(features, exclude_metadata=True)

        original_data = features[feature_cols]
        scaled_data = self.scaler.transform(original_data)

        # Calculate scaling impact metrics
        results = []
        for i, col in enumerate(feature_cols):
            orig_std = original_data[col].std()
            scaled_std = scaled_data[:, i].std()

            orig_range = original_data[col].max() - original_data[col].min()
            scaled_range = scaled_data[:, i].max() - scaled_data[:, i].min()

            results.append({
                'feature': col,
                'original_std': orig_std,
                'scaled_std': scaled_std,
                'std_change_ratio': scaled_std / orig_std if orig_std != 0 else 0,
                'original_range': orig_range,
                'scaled_range': scaled_range,
                'range_change_ratio': scaled_range / orig_range if orig_range != 0 else 0
            })

        importance_df = pd.DataFrame(results)

        # Log as table
        table = wandb.Table(dataframe=importance_df.head(20))
        self.run.log({"feature_scaling_impact": table})

        return importance_df

    def validate_scaling(self, baseline_scaled: pd.DataFrame, features_scaled: pd.DataFrame):
        """
        Validate that scaling applied to baseline and new features is consistent.
        Compares means and standard deviations for the same features.
        Logs results to W&B and warns on large discrepancies.
        """
        # Identify feature columns
        feature_cols = get_feature_columns(baseline_scaled, exclude_metadata=True)
        if len(feature_cols) == 0:
            raise ValueError("No feature columns found for scaling validation.")

        # Calculate means and stds
        baseline_means = baseline_scaled[feature_cols].mean()
        features_means = features_scaled[feature_cols].mean()
        baseline_stds = baseline_scaled[feature_cols].std()
        features_stds = features_scaled[feature_cols].std()

        # Per-feature absolute differences
        mean_diffs = (baseline_means - features_means).abs()
        std_diffs = (baseline_stds - features_stds).abs()

        # Aggregate metrics
        mean_diff_avg = float(mean_diffs.mean())
        std_diff_avg = float(std_diffs.mean())

        # Log summary metrics
        self.run.log({
            "scaling_validation_mean_diff_avg": mean_diff_avg,
            "scaling_validation_std_diff_avg": std_diff_avg
        })

        # Log detailed table to W&B (only top deviations)
        diff_df = pd.DataFrame({
            "feature": feature_cols,
            "mean_diff": mean_diffs.values,
            "std_diff": std_diffs.values
        }).sort_values("mean_diff", ascending=False)

        self.run.log({"scaling_validation_details": wandb.Table(dataframe=diff_df.head(20))})

        # Warn if too large deviation
        if mean_diff_avg > 0.5 or std_diff_avg > 0.5:
            print(f"⚠️ Warning: Large scaling mismatch detected "
                  f"(mean_diff_avg={mean_diff_avg:.4f}, std_diff_avg={std_diff_avg:.4f})")



def main():
    import argparse

    parser = argparse.ArgumentParser(description="Scale features for machine learning")
    parser.add_argument("--input-file", default="data/features.parquet", help="Input features file")
    parser.add_argument("--output-file", default="data/scaled_features.parquet", help="Output scaled features file")
    parser.add_argument("--method", default="robust", choices=["robust", "standard", "minmax"], help="Scaling method")
    parser.add_argument("--save-scaler", default="models/feature_scaler.joblib", help="Path to save scaler")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="feature-scaling")

    # Load features
    print(f"Loading features from {args.input_file}...")
    features = pd.read_parquet(args.input_file)
    print(f"Loaded features shape: {features.shape}")

    # Log feature information using dynamic identification
    from src.utils.sensor_utils import get_feature_columns, log_sensor_column_info
    feature_cols = get_feature_columns(features, exclude_metadata=True)
    feature_info = log_sensor_column_info(features, feature_cols, run)

    print(f"Identified {len(feature_cols)} feature columns for scaling")
    print(f"Feature columns: {feature_cols[:10]}{'...' if len(feature_cols) > 10 else ''}")

    # Initialize scaler
    scaler = FeatureScaler(run)

    # Fit and transform features
    print(f"Scaling features using {args.method} method...")
    scaled_features, fitted_scaler = scaler.fit_transform(features, method=args.method)

    print(f"Scaled features shape: {scaled_features.shape}")

    # Create visualization
    print("Creating scaling visualization...")
    scaler.create_scaling_visualization(features, scaled_features)

    # Analyze scaling impact
    print("Analyzing scaling impact...")
    scaling_impact = scaler.get_feature_importance_by_scale(features)
    print(f"Features most affected by scaling (top 5):")
    top_affected = scaling_impact.nlargest(5, 'std_change_ratio')[['feature', 'std_change_ratio']]
    for _, row in top_affected.iterrows():
        print(f"  {row['feature']}: {row['std_change_ratio']:.3f}x std change")

    # Save scaled features
    print(f"Saving scaled features to {args.output_file}...")
    scaled_features.to_parquet(args.output_file)

    # Save scaler
    print(f"Saving scaler to {args.save_scaler}...")
    scaler.save_scaler(args.save_scaler)

    # Log final artifact
    artifact = wandb.Artifact("scaled-features", type="features")
    artifact.add_file(args.output_file)
    artifact.metadata = {
        "scaling_method": args.method,
        "original_feature_count": len(feature_cols),
        "total_samples": len(features),
        "feature_identification_method": "dynamic_feature_columns",
        "scaler_stats": scaler.scaling_stats
    }
    run.log_artifact(artifact)

    run.finish()
    print("Feature scaling completed!")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/features/select_reduce.py
import pandas as pd
import numpy as np
import wandb
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection
import joblib
import matplotlib.pyplot as plt
from typing import Tuple, List
import os

# Import the new sensor column utility
from src.utils.sensor_utils import get_feature_columns, log_sensor_column_info


class FeatureSelector:
    """
    Feature selection and dimensionality reduction with W&B tracking.
    Now uses DYNAMIC feature column identification instead of hardcoded patterns.
    """

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.selection_stats = {}
        self.models = {}

    def variance_filter(self, features: pd.DataFrame, threshold: float = 0.01) -> Tuple[pd.DataFrame, List[str]]:
        """Filter features based on variance threshold using dynamic column identification."""
        # Use dynamic feature column identification
        feature_cols = get_feature_columns(features, exclude_metadata=True)

        # Log feature identification info
        log_sensor_column_info(features, feature_cols, self.run)

        self.run.log({
            "variance_filter_threshold": threshold,
            "variance_filter_input_features": len(feature_cols),
            "feature_identification_method": "dynamic_feature_columns"
        })

        # Apply variance threshold
        selector = VarianceThreshold(threshold=threshold)
        selected_data = selector.fit_transform(features[feature_cols])

        # Get selected feature names
        selected_features = [feature_cols[i] for i in range(len(feature_cols)) if selector.variances_[i] > threshold]
        removed_features = [feature_cols[i] for i in range(len(feature_cols)) if selector.variances_[i] <= threshold]

        # Create filtered dataframe
        filtered_df = pd.DataFrame(selected_data, columns=selected_features, index=features.index)

        # Add back non-feature columns (metadata)
        non_feature_cols = [col for col in features.columns if col not in feature_cols]
        for col in non_feature_cols:
            filtered_df[col] = features[col]

        # Log statistics
        self.run.log({
            "variance_filter_selected_features": len(selected_features),
            "variance_filter_removed_features": len(removed_features),
            "variance_filter_retention_rate": len(selected_features) / len(feature_cols) if feature_cols else 0,
            "variance_filter_mean_variance": float(np.mean(selector.variances_)),
            "variance_filter_min_variance": float(np.min(selector.variances_)),
            "variance_filter_max_variance": float(np.max(selector.variances_))
        })

        # Store selection statistics
        self.selection_stats['variance_filter'] = {
            'threshold': threshold,
            'input_features': len(feature_cols),
            'selected_features': len(selected_features),
            'removed_features': len(removed_features),
            'retention_rate': len(selected_features) / len(feature_cols) if feature_cols else 0
        }

        # Store the selector model for later use
        self.models['variance_selector'] = selector

        return filtered_df, removed_features

    def correlation_filter(self, features: pd.DataFrame, threshold: float = 0.95) -> Tuple[pd.DataFrame, List[str]]:
        """Remove highly correlated features using dynamic column identification."""
        # Use dynamic feature column identification
        feature_cols = get_feature_columns(features, exclude_metadata=True)

        if len(feature_cols) < 2:
            return features, []

        # Calculate correlation matrix
        feature_data = features[feature_cols]
        corr_matrix = feature_data.corr().abs()

        # Find highly correlated pairs
        upper_triangle = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        )

        # Identify features to remove
        to_remove = [
            column for column in upper_triangle.columns
            if any(upper_triangle[column] > threshold)
        ]

        # Remove features
        filtered_features = features.drop(columns=to_remove)

        # Log statistics
        self.run.log({
            "correlation_filter_threshold": threshold,
            "correlation_filter_input_features": len(feature_cols),
            "correlation_filter_removed_features": len(to_remove),
            "correlation_filter_retained_features": len(feature_cols) - len(to_remove),
            "correlation_filter_retention_rate": (len(feature_cols) - len(to_remove)) / len(feature_cols)
        })

        # Store selection statistics
        self.selection_stats['correlation_filter'] = {
            'threshold': threshold,
            'input_features': len(feature_cols),
            'removed_features': len(to_remove),
            'retention_rate': (len(feature_cols) - len(to_remove)) / len(feature_cols)
        }

        return filtered_features, to_remove

    def univariate_selection(self, features: pd.DataFrame, target: pd.Series,
                           k: int = 100, score_func=mutual_info_regression) -> Tuple[pd.DataFrame, List[str]]:
        """Select k best features using univariate statistical tests."""
        # Use dynamic feature column identification
        feature_cols = get_feature_columns(features, exclude_metadata=True)

        if len(feature_cols) == 0:
            return features, []

        # Ensure k doesn't exceed available features
        k = min(k, len(feature_cols))

        # Apply univariate selection
        selector = SelectKBest(score_func=score_func, k=k)

        # Handle potential issues with mutual information
        try:
            feature_data = features[feature_cols].fillna(0)  # Fill NaN values
            selected_data = selector.fit_transform(feature_data, target)

            # Get selected feature names
            selected_mask = selector.get_support()
            selected_features = [feature_cols[i] for i in range(len(feature_cols)) if selected_mask[i]]
            removed_features = [feature_cols[i] for i in range(len(feature_cols)) if not selected_mask[i]]

            # Create filtered dataframe
            filtered_df = pd.DataFrame(selected_data, columns=selected_features, index=features.index)

            # Add back non-feature columns
            non_feature_cols = [col for col in features.columns if col not in feature_cols]
            for col in non_feature_cols:
                filtered_df[col] = features[col]

            # Log statistics
            scores = selector.scores_
            self.run.log({
                "univariate_selection_k": k,
                "univariate_selection_input_features": len(feature_cols),
                "univariate_selection_selected_features": len(selected_features),
                "univariate_selection_mean_score": float(np.mean(scores)),
                "univariate_selection_max_score": float(np.max(scores)),
                "univariate_selection_min_score": float(np.min(scores))
            })

            # Store selection statistics
            self.selection_stats['univariate_selection'] = {
                'k': k,
                'input_features': len(feature_cols),
                'selected_features': len(selected_features),
                'mean_score': float(np.mean(scores)),
                'score_function': str(score_func.__name__)
            }

            # Store selector for future use
            self.models['univariate_selector'] = selector

        except Exception as e:
            self.run.log({"univariate_selection_error": str(e)})
            return features, []

        return filtered_df, removed_features

    def pca_reduction(self, features: pd.DataFrame, n_components: float = 0.95) -> Tuple[pd.DataFrame, int]:
        """Apply PCA dimensionality reduction."""
        # Use dynamic feature column identification
        feature_cols = get_feature_columns(features, exclude_metadata=True)

        if len(feature_cols) == 0:
            return features, 0

        # Apply PCA
        if isinstance(n_components, float):
            # Variance retention mode
            pca = PCA(n_components=n_components, random_state=42)
        else:
            # Fixed number of components
            n_components = min(n_components, len(feature_cols))
            pca = PCA(n_components=n_components, random_state=42)

        try:
            feature_data = features[feature_cols].fillna(0)
            pca_data = pca.fit_transform(feature_data)

            # Create PCA feature names
            pca_columns = [f'PC_{i+1}' for i in range(pca_data.shape[1])]

            # Create PCA dataframe
            pca_df = pd.DataFrame(pca_data, columns=pca_columns, index=features.index)

            # Add back non-feature columns
            non_feature_cols = [col for col in features.columns if col not in feature_cols]
            for col in non_feature_cols:
                pca_df[col] = features[col]

            # Log statistics
            explained_variance_ratio = pca.explained_variance_ratio_
            cumulative_variance = np.cumsum(explained_variance_ratio)

            self.run.log({
                "pca_input_features": len(feature_cols),
                "pca_output_components": pca_data.shape[1],
                "pca_explained_variance_total": float(cumulative_variance[-1]),
                "pca_explained_variance_first_component": float(explained_variance_ratio[0]),
                "pca_reduction_ratio": pca_data.shape[1] / len(feature_cols)
            })

            # Store PCA model and statistics
            self.models['pca'] = pca
            self.selection_stats['pca'] = {
                'input_features': len(feature_cols),
                'output_components': pca_data.shape[1],
                'explained_variance_total': float(cumulative_variance[-1]),
                'reduction_ratio': pca_data.shape[1] / len(feature_cols)
            }

            # Log explained variance plot
            self.create_pca_visualization(explained_variance_ratio, cumulative_variance)

        except Exception as e:
            self.run.log({"pca_error": str(e)})
            return features, 0

        return pca_df, len(feature_cols) - pca_data.shape[1]

    def random_projection(self, features: pd.DataFrame, n_components: int = 100) -> Tuple[pd.DataFrame, int]:
        """Apply random projection for dimensionality reduction."""
        # Use dynamic feature column identification
        feature_cols = get_feature_columns(features, exclude_metadata=True)

        if len(feature_cols) == 0:
            return features, 0

        # Ensure n_components doesn't exceed available features
        n_components = min(n_components, len(feature_cols))

        try:
            # Apply random projection
            rp = GaussianRandomProjection(n_components=n_components, random_state=42)
            feature_data = features[feature_cols].fillna(0)
            rp_data = rp.fit_transform(feature_data)

            # Create random projection feature names
            rp_columns = [f'RP_{i+1}' for i in range(rp_data.shape[1])]

            # Create random projection dataframe
            rp_df = pd.DataFrame(rp_data, columns=rp_columns, index=features.index)

            # Add back non-feature columns
            non_feature_cols = [col for col in features.columns if col not in feature_cols]
            for col in non_feature_cols:
                rp_df[col] = features[col]

            # Log statistics
            self.run.log({
                "random_projection_input_features": len(feature_cols),
                "random_projection_output_components": rp_data.shape[1],
                "random_projection_reduction_ratio": rp_data.shape[1] / len(feature_cols)
            })

            # Store model and statistics
            self.models['random_projection'] = rp
            self.selection_stats['random_projection'] = {
                'input_features': len(feature_cols),
                'output_components': rp_data.shape[1],
                'reduction_ratio': rp_data.shape[1] / len(feature_cols)
            }

        except Exception as e:
            self.run.log({"random_projection_error": str(e)})
            return features, 0

        return rp_df, len(feature_cols) - rp_data.shape[1]

    def create_pca_visualization(self, explained_variance_ratio: np.ndarray,
                               cumulative_variance: np.ndarray):
        """Create PCA visualization."""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Individual explained variance
        components = range(1, len(explained_variance_ratio) + 1)
        ax1.bar(components[:20], explained_variance_ratio[:20])  # Show first 20 components
        ax1.set_xlabel('Principal Component')
        ax1.set_ylabel('Explained Variance Ratio')
        ax1.set_title('Explained Variance by Component')

        # Cumulative explained variance
        ax2.plot(components, cumulative_variance, 'bo-')
        ax2.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
        ax2.axhline(y=0.90, color='orange', linestyle='--', label='90% Variance')
        ax2.set_xlabel('Number of Components')
        ax2.set_ylabel('Cumulative Explained Variance')
        ax2.set_title('Cumulative Explained Variance')
        ax2.legend()
        ax2.grid(True)

        plt.tight_layout()
        self.run.log({"pca_visualization": wandb.Image(fig)})
        plt.close()

    def save_models(self, filepath_prefix: str = "models/feature_selection"):
        """Save all fitted models."""
        # Create models directory if it doesn't exist
        os.makedirs(os.path.dirname(filepath_prefix) if os.path.dirname(filepath_prefix) else "models", exist_ok=True)

        for model_name, model in self.models.items():
            # Special handling for different model types
            if model_name == 'variance_selector':
                filepath = "models/variance_selector.pkl"
            elif model_name == 'pca':
                filepath = "models/pca.pkl"
            else:
                filepath = f"{filepath_prefix}_{model_name}.joblib"

            joblib.dump(model, filepath)

            # Log model artifact
            artifact = wandb.Artifact(f"feature-selection-{model_name}", type="model")
            artifact.add_file(filepath)
            artifact.metadata = self.selection_stats.get(model_name, {})
            self.run.log_artifact(artifact)

        self.run.log({"models_saved": len(self.models)})

    def combine_selection_methods(
        self,
        features,
        variance_threshold=0.0,
        mi_top_k=None,
        pca_variance=None
    ):
        """
        Combines multiple feature selection methods:
        - Variance Threshold
        - Mutual Information
        - PCA dimensionality reduction

        FIXED VERSION: Correctly calls existing methods and handles return values
        """
        selected_features = features

        # Variance threshold
        if variance_threshold > 0.0:
            self.run.log({"feature_selection_step": "variance_threshold"})
            selected_features, removed = self.variance_filter(
                selected_features,
                threshold=variance_threshold
            )
            self.run.log({
                "variance_removed_features": len(removed),
                "variance_remaining_features": len(selected_features.columns)
            })

        # Mutual Information
        if mi_top_k is not None and mi_top_k > 0:
            self.run.log({"feature_selection_step": "mutual_information"})

            # Generate target for mutual information
            # TODO: Replace with actual anomaly scores or degradation indicators
            # Options:
            # 1. Use RUL (Remaining Useful Life) if available
            # 2. Use health index computed from domain knowledge
            # 3. Use anomaly scores from unsupervised methods
            # 4. Use time-based degradation assumption (current placeholder)

            # For now, using time-based progression as proxy for degradation
            # This assumes degradation increases with time (reasonable for many systems)
            target = np.arange(len(selected_features))

            # Normalize target to [0, 1] range for better MI computation
            target = (target - target.min()) / (target.max() - target.min() + 1e-10)

            self.run.log({
                "mi_target_type": "time_progression_placeholder",
                "mi_target_warning": "Using synthetic target - replace with actual anomaly/degradation scores"
            })

            selected_features, removed = self.univariate_selection(
                selected_features,
                target=target,
                k=mi_top_k
            )
            self.run.log({
                "mi_removed_features": len(removed),
                "mi_selected_features": mi_top_k,
                "mi_remaining_features": len(selected_features.columns)
            })

        # PCA
        if pca_variance is not None and 0 < pca_variance <= 1.0:
            self.run.log({"feature_selection_step": "pca_reduction"})
            selected_features, n_reduced = self.pca_reduction(
                selected_features,
                n_components=pca_variance
            )
            self.run.log({
                "pca_components_reduced": n_reduced,
                "pca_variance_retained": pca_variance,
                "pca_final_components": len([col for col in selected_features.columns if col.startswith('PC_')])
            })

        # Log final statistics
        feature_cols = get_feature_columns(selected_features, exclude_metadata=True)
        self.run.log({
            "final_feature_count": len(feature_cols),
            "total_features": len(selected_features.columns),
            "metadata_columns": len(selected_features.columns) - len(feature_cols)
        })

        # Final combined set
        combined_features = selected_features
        return combined_features


def main():
    import argparse

    parser = argparse.ArgumentParser(description="Feature selection and dimensionality reduction")
    parser.add_argument("--input-file", default="data/scaled_features.parquet", help="Input features file")
    parser.add_argument("--output-file", default="data/selected_features.parquet", help="Output selected features file")
    parser.add_argument("--method", default="variance",
                       choices=["variance", "correlation", "univariate", "pca", "random_projection", "all"],
                       help="Feature selection method")
    parser.add_argument("--variance-threshold", type=float, default=0.01, help="Variance threshold")
    parser.add_argument("--correlation-threshold", type=float, default=0.95, help="Correlation threshold")
    parser.add_argument("--k-best", type=int, default=100, help="Number of best features for univariate selection")
    parser.add_argument("--pca-components", type=float, default=0.95, help="PCA components (float for variance ratio)")
    parser.add_argument("--rp-components", type=int, default=100, help="Random projection components")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="feature-selection")

    # Load features
    print(f"Loading features from {args.input_file}...")
    features = pd.read_parquet(args.input_file)
    print(f"Loaded features shape: {features.shape}")

    # Log feature information using dynamic identification
    from src.utils.sensor_utils import get_feature_columns, log_sensor_column_info
    feature_cols = get_feature_columns(features, exclude_metadata=True)
    feature_info = log_sensor_column_info(features, feature_cols, run)

    print(f"Identified {len(feature_cols)} feature columns for selection")
    print(f"Feature columns: {feature_cols[:10]}{'...' if len(feature_cols) > 10 else ''}")

    # Initialize selector
    selector = FeatureSelector(run)

    # Apply feature selection based on method
    selected_features = features.copy()
    total_removed = 0

    if args.method in ["variance", "all"]:
        print(f"Applying variance filter (threshold={args.variance_threshold})...")
        selected_features, removed = selector.variance_filter(selected_features, args.variance_threshold)
        total_removed += len(removed)
        print(f"Removed {len(removed)} low-variance features")

    if args.method in ["correlation", "all"]:
        print(f"Applying correlation filter (threshold={args.correlation_threshold})...")
        selected_features, removed = selector.correlation_filter(selected_features, args.correlation_threshold)
        total_removed += len(removed)
        print(f"Removed {len(removed)} highly correlated features")

    if args.method in ["univariate", "all"]:
        print(f"Applying univariate selection (k={args.k_best})...")
        # For demonstration, create a dummy target (in practice, this should be provided)
        dummy_target = np.random.rand(len(features))
        selected_features, removed = selector.univariate_selection(selected_features, dummy_target, args.k_best)
        total_removed += len(removed)
        print(f"Selected {args.k_best} best features using univariate selection")

    if args.method in ["pca", "all"]:
        print(f"Applying PCA (components={args.pca_components})...")
        selected_features, reduced = selector.pca_reduction(selected_features, args.pca_components)
        print(f"Reduced {reduced} features using PCA")

    if args.method == "random_projection":
        print(f"Applying random projection (components={args.rp_components})...")
        selected_features, reduced = selector.random_projection(selected_features, args.rp_components)
        print(f"Reduced {reduced} features using random projection")

    print(f"Final feature shape: {selected_features.shape}")
    print(f"Total features removed/reduced: {total_removed}")

    # Save selected features
    print(f"Saving selected features to {args.output_file}...")
    selected_features.to_parquet(args.output_file)

    # Save models
    print("Saving feature selection models...")
    selector.save_models()

    # Log final artifact
    artifact = wandb.Artifact("selected-features", type="features")
    artifact.add_file(args.output_file)
    artifact.metadata = {
        "selection_method": args.method,
        "original_feature_count": len(feature_cols),
        "final_feature_count": len(get_feature_columns(selected_features, exclude_metadata=True)),
        "total_removed": total_removed,
        "feature_identification_method": "dynamic_feature_columns",
        "selection_stats": selector.selection_stats
    }
    run.log_artifact(artifact)

    run.finish()
    print("Feature selection completed!")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/model_input/assemble.py
import pandas as pd
import numpy as np
import wandb
from typing import Tuple, List, Dict, Optional
import h5py
import os
import sys # Import sys
from src.utils.sensor_utils import get_feature_columns, log_sensor_column_info

class SequenceAssembler:
    """Assemble sequences for model input with W&B tracking and pipeline mode support."""

    def __init__(self, wandb_run: wandb.run = None, pipeline_mode: str = "train"):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.pipeline_mode = pipeline_mode
        self.assembly_stats = {}

    def create_sequences_lstm(self, features: pd.DataFrame,
                            sequence_length: int = 10,
                            overlap: float = 0.5) -> Tuple[np.ndarray, np.ndarray]:
        """Create sequences for LSTM models."""
        # Sort by window_id to ensure temporal order
        features = features.sort_values('window_id')

        # Get feature columns
        feature_cols = get_feature_columns(df, exclude_metadata=True)

        data = features[feature_cols].values
        n_features = len(feature_cols)
        n_samples = len(data)

        # Calculate stride based on overlap
        stride = max(1, int(sequence_length * (1 - overlap)))

        # Calculate number of sequences
        num_sequences = (n_samples - sequence_length) // stride + 1

        # Create sequences
        X_lstm = np.zeros((num_sequences, sequence_length, n_features))
        timestamps = np.zeros(num_sequences)

        for i in range(num_sequences):
            start_idx = i * stride
            end_idx = start_idx + sequence_length
            X_lstm[i] = data[start_idx:end_idx]
            timestamps[i] = features.iloc[start_idx]['window_id']

        # Log statistics
        self.assembly_stats['lstm'] = {
            'sequence_length': sequence_length,
            'overlap': overlap,
            'stride': stride,
            'num_sequences': num_sequences,
            'shape': X_lstm.shape
        }

        self.run.log({
            "lstm_num_sequences": num_sequences,
            "lstm_sequence_shape": str(X_lstm.shape),
            "lstm_memory_mb": X_lstm.nbytes / (1024 * 1024)
        })

        return X_lstm, timestamps

    def create_tabular_format(self, features: pd.DataFrame) -> np.ndarray:
        """Create tabular format for traditional ML models."""
        feature_cols = [col for col in features.columns
                       if col not in ['window_id', 'window_start']]

        X_tabular = features[feature_cols].values

        # Log statistics
        self.assembly_stats['tabular'] = {
            'num_samples': len(X_tabular),
            'num_features': X_tabular.shape[1],
            'shape': X_tabular.shape
        }

        self.run.log({
            "tabular_num_samples": len(X_tabular),
            "tabular_num_features": X_tabular.shape[1]
        })

        return X_tabular

    def create_sequences_for_mode(self, features: pd.DataFrame,
                                sequence_length: int = 10,
                                overlap: float = 0.5,
                                train_ratio: float = 0.8) -> Dict[str, np.ndarray]:
        """Create sequences based on pipeline mode."""
        X_lstm, timestamps = self.create_sequences_lstm(features, sequence_length, overlap)

        if self.pipeline_mode == "train":
            # For training mode: create only train/val split (no test)
            n_samples = len(X_lstm)
            train_end = int(n_samples * train_ratio)

            splits = {
                'X_train': X_lstm[:train_end],
                'X_val': X_lstm[train_end:],
                'timestamps_train': timestamps[:train_end],
                'timestamps_val': timestamps[train_end:]
            }

            # Log split statistics
            self.run.log({
                "X_train_samples": len(splits['X_train']),
                "X_train_shape": str(splits['X_train'].shape),
                "X_val_samples": len(splits['X_val']),
                "X_val_shape": str(splits['X_val'].shape)
            })

        elif self.pipeline_mode == "test":
            # For test mode: create full dataset for anomaly detection
            splits = {
                'X_test': X_lstm,
                'timestamps_test': timestamps
            }

            self.run.log({
                "X_test_samples": len(splits['X_test']),
                "X_test_shape": str(splits['X_test'].shape)
            })

        else:
            raise ValueError(f"Invalid pipeline_mode: {self.pipeline_mode}. Must be 'train' or 'test'")

        return splits

    def create_train_val_test_split(self, X: np.ndarray, timestamps: np.ndarray = None,
                                  train_ratio: float = 0.7,
                                  val_ratio: float = 0.15) -> Dict[str, np.ndarray]:
        """Create train/validation/test splits (legacy method for compatibility)."""
        n_samples = len(X)

        # Calculate split indices
        train_end = int(n_samples * train_ratio)
        val_end = int(n_samples * (train_ratio + val_ratio))

        # Split data
        splits = {
            'X_train': X[:train_end],
            'X_val': X[train_end:val_end],
            'X_test': X[val_end:]
        }

        if timestamps is not None:
            splits['timestamps_train'] = timestamps[:train_end]
            splits['timestamps_val'] = timestamps[train_end:val_end]
            splits['timestamps_test'] = timestamps[val_end:]

        # Log split statistics
        for split_name, split_data in splits.items():
            if split_name.startswith('X_'):
                self.run.log({
                    f"{split_name}_samples": len(split_data),
                    f"{split_name}_shape": str(split_data.shape)
                })

        return splits

    def save_sequences(self, sequences: Dict[str, np.ndarray],
                      output_dir: str = "data"):
        """Save sequences in multiple formats with W&B artifacts."""
        os.makedirs(output_dir, exist_ok=True)

        # Save as numpy arrays
        for name, array in sequences.items():
            if isinstance(array, np.ndarray):
                np_path = f"{output_dir}/{name}.npy"
                np.save(np_path, array)

                # Log basic arrays as artifacts
                if name.startswith('X_'):
                    artifact = wandb.Artifact(
                        f"model-input-{name}-{self.pipeline_mode}",
                        type="dataset"
                    )
                    artifact.add_file(np_path)
                    artifact.metadata = {
                        "shape": array.shape,
                        "dtype": str(array.dtype),
                        "size_mb": array.nbytes / (1024 * 1024),
                        "pipeline_mode": self.pipeline_mode
                    }
                    self.run.log_artifact(artifact)

        # Save as HDF5 for efficient loading
        h5_path = f"{output_dir}/model_inputs.h5"
        with h5py.File(h5_path, 'w') as h5f:
            for name, array in sequences.items():
                if isinstance(array, np.ndarray):
                    h5f.create_dataset(name, data=array, compression='gzip')

            # Store pipeline mode as attribute
            h5f.attrs['pipeline_mode'] = self.pipeline_mode

        # Log HDF5 as artifact
        artifact = wandb.Artifact(f"model-inputs-h5-{self.pipeline_mode}", type="dataset")
        artifact.add_file(h5_path)
        artifact.metadata = {"pipeline_mode": self.pipeline_mode}
        self.run.log_artifact(artifact)

        # Create and log metadata
        self.create_assembly_report(sequences, output_dir)

    def create_assembly_report(self, sequences: Dict[str, np.ndarray],
                             output_dir: str):
        """Create comprehensive assembly report."""
        import matplotlib.pyplot as plt

        # Summary statistics
        summary = []
        for name, array in sequences.items():
            if isinstance(array, np.ndarray) and name.startswith('X_'):
                summary.append({
                    'dataset': name,
                    'shape': str(array.shape),
                    'size_mb': f"{array.nbytes / (1024 * 1024):.2f}",
                    'dtype': str(array.dtype),
                    'min': f"{np.min(array):.4f}",
                    'max': f"{np.max(array):.4f}",
                    'mean': f"{np.mean(array):.4f}",
                    'std': f"{np.std(array):.4f}",
                    'pipeline_mode': self.pipeline_mode
                })

        if summary:
            summary_df = pd.DataFrame(summary)
            table = wandb.Table(dataframe=summary_df)
            self.run.log({f"assembly_summary_{self.pipeline_mode}": table})

        # Create visualization
        if summary:
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            fig.suptitle(f'Sequence Assembly Report - {self.pipeline_mode.upper()} Mode', fontsize=16)

            # Dataset sizes
            ax1 = axes[0, 0]
            datasets = [s['dataset'] for s in summary]
            sizes = [float(s['size_mb']) for s in summary]
            ax1.bar(datasets, sizes, color='skyblue')
            ax1.set_xlabel('Dataset')
            ax1.set_ylabel('Size (MB)')
            ax1.set_title('Dataset Sizes')
            ax1.tick_params(axis='x', rotation=45)

            # Sample distribution (first dataset)
            ax2 = axes[0, 1]
            first_dataset_key = next((k for k in sequences.keys() if k.startswith('X_')), None)
            if first_dataset_key and first_dataset_key in sequences:
                sample_data = sequences[first_dataset_key].flatten()[:10000]  # Sample first 10k points
                ax2.hist(sample_data, bins=50, alpha=0.7, color='green')
                ax2.set_xlabel('Value')
                ax2.set_ylabel('Frequency')
                ax2.set_title(f'Sample Value Distribution ({first_dataset_key})')
                ax2.grid(True, alpha=0.3)

            # Mode-specific information
            ax3 = axes[1, 0]
            mode_info = [
                f"Pipeline Mode: {self.pipeline_mode.upper()}",
                f"Total Datasets: {len([k for k in sequences.keys() if k.startswith('X_')])}",
                f"Assembly Stats: {len(self.assembly_stats)} formats"
            ]

            for i, info in enumerate(mode_info):
                ax3.text(0.1, 0.8 - i*0.2, info, fontsize=12, va='center', transform=ax3.transAxes)

            ax3.set_xlim(0, 1)
            ax3.set_ylim(0, 1)
            ax3.set_xticks([])
            ax3.set_yticks([])
            ax3.set_title('Pipeline Information')
            for spine in ax3.spines.values():
                spine.set_visible(False)

            # Memory usage
            ax4 = axes[1, 1]
            if len(summary) > 0:
                memory_usage = {s['dataset']: float(s['size_mb']) for s in summary}
                colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow'][:len(memory_usage)]
                wedges, texts, autotexts = ax4.pie(
                    memory_usage.values(),
                    labels=memory_usage.keys(),
                    autopct='%1.1f%%',
                    startangle=90,
                    colors=colors
                )
                ax4.set_title('Memory Usage Distribution')

            plt.tight_layout()
            self.run.log({f"assembly_visualization_{self.pipeline_mode}": wandb.Image(fig)})
            plt.close()

        # Save assembly statistics
        import json
        stats_path = f"{output_dir}/assembly_stats_{self.pipeline_mode}.json"
        with open(stats_path, 'w') as f:
            # Convert numpy types to native Python types for JSON serialization
            stats_serializable = {
                "pipeline_mode": self.pipeline_mode,
                "assembly_stats": {}
            }
            for key, value in self.assembly_stats.items():
                if isinstance(value, dict):
                    stats_serializable["assembly_stats"][key] = {
                        k: v.tolist() if isinstance(v, np.ndarray) else v
                        for k, v in value.items()
                    }
                else:
                    stats_serializable["assembly_stats"][key] = value
            json.dump(stats_serializable, f, indent=2)

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Assemble sequences for model input")
    parser.add_argument("--input-file", default="features/reduced_features.csv", help="Input file")
    parser.add_argument("--output-dir", default="data", help="Output directory")
    parser.add_argument("--sequence-length", type=int, default=10, help="Sequence length for LSTM")
    parser.add_argument("--overlap", type=float, default=0.5, help="Overlap for sequences")
    parser.add_argument("--train-ratio", type=float, default=0.8, help="Training set ratio (for train mode)")
    parser.add_argument("--pipeline-mode", choices=["train", "test"], default="train",
                       help="Pipeline mode: 'train' or 'test'")

    # Parse known arguments, ignoring the rest
    args, unknown = parser.parse_known_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="sequence-assembly")

    # Load features
    features = pd.read_csv(args.input_file)

    # Create sequences with mode-specific behavior
    assembler = SequenceAssembler(run, pipeline_mode=args.pipeline_mode)

    # Create sequences based on mode
    sequences = assembler.create_sequences_for_mode(
        features,
        sequence_length=args.sequence_length,
        overlap=args.overlap,
        train_ratio=args.train_ratio
    )

    # Also create tabular format for compatibility
    X_tabular = assembler.create_tabular_format(features)
    if args.pipeline_mode == "train":
        # For train mode, split tabular data too
        n_samples = len(X_tabular)
        train_end = int(n_samples * args.train_ratio)
        sequences['X_tabular_train'] = X_tabular[:train_end]
        sequences['X_tabular_val'] = X_tabular[train_end:]
    else:
        # For test mode, keep all tabular data
        sequences['X_tabular_test'] = X_tabular

    # Save sequences
    assembler.save_sequences(sequences, args.output_dir)

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/monitoring/drift.py
import pandas as pd
import numpy as np
import wandb
from scipy.stats import wasserstein_distance, ks_2samp
from typing import Dict, Tuple, List
import json
import yaml

class DriftMonitor:
    """Monitor data drift and trigger retraining with W&B integration."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.drift_metrics = {}

    def compute_psi(self, baseline: np.ndarray, current: np.ndarray,
                   bins: int = 10) -> float:
        """Compute Population Stability Index (PSI)."""
        # Create bins based on baseline
        min_val = min(baseline.min(), current.min())
        max_val = max(baseline.max(), current.max())
        bins_edges = np.linspace(min_val, max_val, bins + 1)

        # Calculate frequencies
        baseline_freq, _ = np.histogram(baseline, bins=bins_edges)
        current_freq, _ = np.histogram(current, bins=bins_edges)

        # Normalize to probabilities
        baseline_prob = (baseline_freq + 1) / (len(baseline) + bins)  # Add 1 to avoid log(0)
        current_prob = (current_freq + 1) / (len(current) + bins)

        # Calculate PSI
        psi = np.sum((current_prob - baseline_prob) * np.log(current_prob / baseline_prob))

        return float(psi)

    def compute_kl_divergence(self, baseline: np.ndarray, current: np.ndarray,
                            bins: int = 50) -> float:
        """Compute Kullback-Leibler divergence."""
        # Create common bins
        min_val = min(baseline.min(), current.min())
        max_val = max(baseline.max(), current.max())
        bins_edges = np.linspace(min_val, max_val, bins + 1)

        # Calculate probabilities
        p, _ = np.histogram(baseline, bins=bins_edges, density=True)
        q, _ = np.histogram(current, bins=bins_edges, density=True)

        # Normalize
        p = p + 1e-10
        q = q + 1e-10
        p = p / p.sum()
        q = q / q.sum()

        # KL divergence
        kl = np.sum(p * np.log(p / q))

        return float(kl)

    def compute_wasserstein(self, baseline: np.ndarray, current: np.ndarray) -> float:
        """Compute Wasserstein distance."""
        return float(wasserstein_distance(baseline, current))

    def compute_ks_statistic(self, baseline: np.ndarray, current: np.ndarray) -> Tuple[float, float]:
        """Compute Kolmogorov-Smirnov test statistic."""
        ks_stat, p_value = ks_2samp(baseline, current)
        return float(ks_stat), float(p_value)

    def monitor_feature_drift(self, baseline_features: pd.DataFrame,
                            current_features: pd.DataFrame) -> Dict[str, Dict[str, float]]:
        """Monitor drift for all features."""
        # Get common feature columns
        feature_cols = [col for col in baseline_features.columns
                       if col in current_features.columns and col not in ['window_id', 'window_start']]

        drift_results = {}

        for col in feature_cols:
            baseline_vals = baseline_features[col].dropna().values
            current_vals = current_features[col].dropna().values

            if len(baseline_vals) < 10 or len(current_vals) < 10:
                continue

            # Compute drift metrics
            psi = self.compute_psi(baseline_vals, current_vals)
            kl = self.compute_kl_divergence(baseline_vals, current_vals)
            wasserstein = self.compute_wasserstein(baseline_vals, current_vals)
            ks_stat, ks_pvalue = self.compute_ks_statistic(baseline_vals, current_vals)

            drift_results[col] = {
                'psi': psi,
                'kl_divergence': kl,
                'wasserstein': wasserstein,
                'ks_statistic': ks_stat,
                'ks_pvalue': ks_pvalue
            }

            # Log individual metrics
            self.run.log({
                f"drift_psi_{col}": psi,
                f"drift_kl_{col}": kl,
                f"drift_wasserstein_{col}": wasserstein,
                f"drift_ks_{col}": ks_stat
            })

        self.drift_metrics = drift_results
        return drift_results

    def evaluate_drift_severity(self, drift_results: Dict[str, Dict[str, float]],
                              thresholds: Dict[str, float] = None) -> Dict[str, str]:
        """Evaluate drift severity based on thresholds."""
        if thresholds is None:
            thresholds = {
                'psi': {'low': 0.1, 'medium': 0.2, 'high': 0.3},
                'kl_divergence': {'low': 0.5, 'medium': 1.0, 'high': 2.0},
                'wasserstein': {'low': 0.1, 'medium': 0.3, 'high': 0.5},
                'ks_statistic': {'low': 0.1, 'medium': 0.2, 'high': 0.3}
            }

        severity_results = {}

        for feature, metrics in drift_results.items():
            severities = []

            for metric_name, metric_value in metrics.items():
                if metric_name == 'ks_pvalue':
                    continue

                if metric_name in thresholds:
                    thresh = thresholds[metric_name]
                    if metric_value < thresh['low']:
                        severity = 'none'
                    elif metric_value < thresh['medium']:
                        severity = 'low'
                    elif metric_value < thresh['high']:
                        severity = 'medium'
                    else:
                        severity = 'high'

                    severities.append(severity)

            # Overall severity is the maximum
            severity_order = ['none', 'low', 'medium', 'high']
            max_severity = max(severities, key=lambda x: severity_order.index(x))
            severity_results[feature] = max_severity

        return severity_results

    def create_drift_dashboard(self, drift_results: Dict[str, Dict[str, float]],
                             severity_results: Dict[str, str]):
        """Create comprehensive drift visualization dashboard."""
        import matplotlib.pyplot as plt
        import seaborn as sns

        # Prepare data for visualization
        features = list(drift_results.keys())[:20]  # Top 20 features
        psi_values = [drift_results[f]['psi'] for f in features]
        kl_values = [drift_results[f]['kl_divergence'] for f in features]

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # PSI values
        ax1 = axes[0, 0]
        colors = ['green' if v < 0.1 else 'yellow' if v < 0.2 else 'red' for v in psi_values]
        bars = ax1.barh(range(len(features)), psi_values, color=colors)
        ax1.set_yticks(range(len(features)))
        ax1.set_yticklabels(features, fontsize=8)
        ax1.set_xlabel('PSI Value')
        ax1.set_title('Population Stability Index by Feature')
        ax1.axvline(x=0.1, color='green', linestyle='--', alpha=0.5, label='Low drift')
        ax1.axvline(x=0.2, color='orange', linestyle='--', alpha=0.5, label='Medium drift')
        ax1.axvline(x=0.3, color='red', linestyle='--', alpha=0.5, label='High drift')
        ax1.legend()

        # Severity distribution
        ax2 = axes[0, 1]
        severity_counts = pd.Series(severity_results.values()).value_counts()
        colors_severity = {'none': 'green', 'low': 'yellow', 'medium': 'orange', 'high': 'red'}
        ax2.pie(severity_counts.values, labels=severity_counts.index, autopct='%1.1f%%',
               colors=[colors_severity[s] for s in severity_counts.index],
               startangle=90)
        ax2.set_title('Overall Drift Severity Distribution')

        # Heatmap of drift metrics
        ax3 = axes[1, 0]
        metrics_df = pd.DataFrame(drift_results).T.head(15)
        metrics_df = metrics_df[['psi', 'kl_divergence', 'wasserstein', 'ks_statistic']]

        # Normalize for better visualization
        metrics_normalized = (metrics_df - metrics_df.min()) / (metrics_df.max() - metrics_df.min())

        sns.heatmap(metrics_normalized, annot=True, fmt='.2f', cmap='YlOrRd',
                   ax=ax3, cbar_kws={'label': 'Normalized Drift Score'})
        ax3.set_title('Drift Metrics Heatmap (Top 15 Features)')
        ax3.set_xlabel('Drift Metric')
        ax3.set_ylabel('Feature')

        # Time series of average drift (if historical data available)
        ax4 = axes[1, 1]
        avg_psi = np.mean(psi_values)
        avg_kl = np.mean(kl_values)

        # Simulate historical data for demo
        time_points = pd.date_range(end=pd.Timestamp.now(), periods=10, freq='D')
        historical_psi = np.random.normal(avg_psi, 0.05, 10)
        historical_psi[-1] = avg_psi

        ax4.plot(time_points, historical_psi, 'o-', label='PSI', linewidth=2)
        ax4.axhline(y=0.1, color='green', linestyle='--', alpha=0.5)
        ax4.axhline(y=0.2, color='orange', linestyle='--', alpha=0.5)
        ax4.axhline(y=0.3, color='red', linestyle='--', alpha=0.5)
        ax4.set_xlabel('Date')
        ax4.set_ylabel('Average PSI')
        ax4.set_title('Drift Trend Over Time')
        ax4.legend()
        ax4.grid(True, alpha=0.3)

        plt.tight_layout()
        self.run.log({"drift_dashboard": wandb.Image(fig)})
        plt.close()

    def generate_retraining_sweep(self, drift_severity: Dict[str, str],
                                output_path: str = "configs/sweep.yaml"):
        """Generate W&B sweep configuration for retraining."""
        # Count high severity drifts
        high_drift_count = sum(1 for s in drift_severity.values() if s == 'high')
        medium_drift_count = sum(1 for s in drift_severity.values() if s == 'medium')

        # Determine if retraining is needed
        retrain_needed = high_drift_count > 5 or medium_drift_count > 10

        if retrain_needed:
            # Create sweep configuration
            sweep_config = {
                'method': 'bayes',
                'name': f'drift-triggered-retraining-{pd.Timestamp.now().strftime("%Y%m%d")}',
                'metric': {
                    'name': 'validation_loss',
                    'goal': 'minimize'
                },
                'parameters': {
                    'learning_rate': {
                        'min': 0.0001,
                        'max': 0.01,
                        'distribution': 'log_uniform'
                    },
                    'batch_size': {
                        'values': [32, 64, 128, 256]
                    },
                    'hidden_size': {
                        'values': [64, 128, 256]
                    },
                    'dropout': {
                        'min': 0.1,
                        'max': 0.5,
                        'distribution': 'uniform'
                    },
                    'retrain_preprocessing': {
                        'value': True
                    }
                },
                'early_terminate': {
                    'type': 'hyperband',
                    'min_iter': 5,
                    'eta': 3
                }
            }

            # Save sweep configuration
            with open(output_path, 'w') as f:
                yaml.dump(sweep_config, f)

            # Log sweep config as artifact
            artifact = wandb.Artifact("retraining-sweep-config", type="config")
            artifact.add_file(output_path)
            artifact.metadata = {
                "triggered_by": "drift_detection",
                "high_drift_features": high_drift_count,
                "medium_drift_features": medium_drift_count
            }
            self.run.log_artifact(artifact)

            # Log alert
            wandb.alert(
                title="High Data Drift Detected",
                text=f"Detected {high_drift_count} features with high drift. Retraining recommended.",
                level=wandb.AlertLevel.WARN
            )

            self.run.log({
                "retraining_triggered": True,
                "high_drift_features": high_drift_count,
                "medium_drift_features": medium_drift_count
            })

        return retrain_needed

    def save_drift_report(self, drift_results: Dict[str, Dict[str, float]],
                         severity_results: Dict[str, str],
                         output_dir: str = "monitoring"):
        """Save comprehensive drift report."""
        import os
        os.makedirs(output_dir, exist_ok=True)

        # Create report dataframe
        report_data = []
        for feature, metrics in drift_results.items():
            row = {'feature': feature}
            row.update(metrics)
            row['severity'] = severity_results.get(feature, 'unknown')
            report_data.append(row)

        report_df = pd.DataFrame(report_data)
        report_df = report_df.sort_values('psi', ascending=False)

        # Save report
        report_path = f"{output_dir}/drift_report.csv"
        report_df.to_csv(report_path, index=False)

        # Log as table
        table = wandb.Table(dataframe=report_df.head(50))
        self.run.log({"drift_report": table})

        # Save summary statistics
        summary = {
            "timestamp": pd.Timestamp.now().isoformat(),
            "total_features": len(drift_results),
            "avg_psi": float(report_df['psi'].mean()),
            "max_psi": float(report_df['psi'].max()),
            "features_with_high_drift": int((report_df['severity'] == 'high').sum()),
            "features_with_medium_drift": int((report_df['severity'] == 'medium').sum()),
            "features_with_low_drift": int((report_df['severity'] == 'low').sum())
        }

        summary_path = f"{output_dir}/drift_summary.json"
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2)

def main():
    import argparse

    parser = argparse.ArgumentParser(description="Monitor data drift")
    parser.add_argument("--baseline-file", default="features/reduced_features.csv",
                       help="Baseline features file")
    parser.add_argument("--current-file", required=True, help="Current features file")
    parser.add_argument("--psi-threshold", type=float, default=0.2, help="PSI threshold for alerts")
    parser.add_argument("--output-dir", default="monitoring", help="Output directory")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="drift-monitoring")

    # Load data
    baseline_features = pd.read_csv(args.baseline_file)
    current_features = pd.read_csv(args.current_file)

    # Monitor drift
    monitor = DriftMonitor(run)
    drift_results = monitor.monitor_feature_drift(baseline_features, current_features)

    # Evaluate severity
    severity_results = monitor.evaluate_drift_severity(drift_results)

    # Create visualizations
    monitor.create_drift_dashboard(drift_results, severity_results)

    # Generate retraining sweep if needed
    retrain_needed = monitor.generate_retraining_sweep(severity_results)

    # Save report
    monitor.save_drift_report(drift_results, severity_results, args.output_dir)

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/model/lstm_anomaly_detector.py
import pandas as pd
import numpy as np
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from typing import Dict, Tuple, Optional, Union
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import os
from sklearn.metrics import precision_recall_curve, roc_auc_score

class LSTMAutoencoder(nn.Module):
    """LSTM Autoencoder for anomaly detection."""

    def __init__(self, input_dim: int, hidden_dim: int = 64,
                 num_layers: int = 2, dropout: float = 0.2):
        super(LSTMAutoencoder, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Encoder
        self.encoder = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True, dropout=dropout
        )

        # Decoder
        self.decoder = nn.LSTM(
            hidden_dim, hidden_dim, num_layers,
            batch_first=True, dropout=dropout
        )

        # Output layer
        self.output_layer = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        # Encoder
        encoded, (h_n, c_n) = self.encoder(x)

        # Use the last hidden state for decoding
        batch_size, seq_len, _ = x.shape
        decoder_input = encoded[:, -1, :].unsqueeze(1).repeat(1, seq_len, 1)

        # Decoder
        decoded, _ = self.decoder(decoder_input, (h_n, c_n))

        # Output
        output = self.output_layer(decoded)

        return output

class AnomalyDetector:
    """LSTM-based anomaly detector with pipeline mode support."""

    def __init__(self, wandb_run: wandb.run = None, pipeline_mode: str = "train"):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.pipeline_mode = pipeline_mode
        self.model = None
        self.threshold = None
        self.training_history = {}
        self.anomaly_stats = {}
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Log pipeline mode
        self.run.log({"pipeline_mode": pipeline_mode})

    def load_sequences(self, data_path: str) -> Dict[str, np.ndarray]:
        """Load sequences from HDF5 file."""
        sequences = {}

        with h5py.File(data_path, 'r') as h5f:
            for key in h5f.keys():
                sequences[key] = h5f[key][:]

            # Check if pipeline mode is stored in file
            if 'pipeline_mode' in h5f.attrs:
                file_mode = h5f.attrs['pipeline_mode']
                if isinstance(file_mode, bytes):
                    file_mode = file_mode.decode('utf-8')

                self.run.log({"data_file_pipeline_mode": file_mode})

        return sequences

    def create_model(self, input_dim: int, config: Dict):
        """Create LSTM autoencoder model."""
        self.model = LSTMAutoencoder(
            input_dim=input_dim,
            hidden_dim=config.get('hidden_dim', 64),
            num_layers=config.get('num_layers', 2),
            dropout=config.get('dropout', 0.2)
        ).to(self.device)

        # Log model configuration
        self.run.log({
            "model_input_dim": input_dim,
            "model_hidden_dim": config.get('hidden_dim', 64),
            "model_num_layers": config.get('num_layers', 2),
            "model_dropout": config.get('dropout', 0.2),
            "model_total_params": sum(p.numel() for p in self.model.parameters())
        })

    def train_model(self, X_train: np.ndarray, X_val: np.ndarray,
                   epochs: int = 100, batch_size: int = 32):
        """Train the LSTM autoencoder (only in train mode)."""
        if self.pipeline_mode != "train":
            raise ValueError("Model training is only available in 'train' pipeline mode")

        if self.model is None:
            raise ValueError("Model not created. Call create_model() first.")

        # Convert to tensors
        X_train_tensor = torch.FloatTensor(X_train).to(self.device)
        X_val_tensor = torch.FloatTensor(X_val).to(self.device)

        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, X_val_tensor)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Optimizer and loss
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        # Training history
        train_losses = []
        val_losses = []

        self.model.train()
        for epoch in range(epochs):
            # Training
            train_loss = 0.0
            for batch_x, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = self.model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            # Validation
            val_loss = 0.0
            self.model.eval()
            with torch.no_grad():
                for batch_x, batch_y in val_loader:
                    outputs = self.model(batch_x)
                    loss = criterion(outputs, batch_y)
                    val_loss += loss.item()
            self.model.train()

            avg_train_loss = train_loss / len(train_loader)
            avg_val_loss = val_loss / len(val_loader)

            train_losses.append(avg_train_loss)
            val_losses.append(avg_val_loss)

            # Log every 10 epochs
            if (epoch + 1) % 10 == 0:
                self.run.log({
                    "epoch": epoch + 1,
                    "train_loss": avg_train_loss,
                    "val_loss": avg_val_loss
                })

            if (epoch + 1) % 50 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

        # Store training history
        self.training_history = {
            'train_losses': train_losses,
            'val_losses': val_losses,
            'epochs': epochs
        }

        # Log final metrics
        self.run.log({
            "final_train_loss": train_losses[-1],
            "final_val_loss": val_losses[-1],
            "training_epochs": epochs
        })

    def load_pretrained_model(self, model_path: str):
        """Load a pre-trained model (for test mode)."""
        if self.pipeline_mode != "test":
            raise ValueError("Loading pre-trained model is only available in 'test' pipeline mode")

        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found: {model_path}")

        checkpoint = torch.load(model_path, map_location=self.device)

        # Create model with saved configuration
        config = checkpoint.get('config', {})
        input_dim = checkpoint['input_dim']

        self.create_model(input_dim, config)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.threshold = checkpoint.get('threshold', None)
        self.training_history = checkpoint.get('training_history', {})

        self.model.eval()

        self.run.log({
            "loaded_model_path": model_path,
            "loaded_threshold": self.threshold,
            "model_loaded_successfully": True
        })

    def set_threshold(self, X_val: np.ndarray, percentile: float = 95):
        """Set anomaly detection threshold based on validation data."""
        if self.model is None:
            raise ValueError("Model not available. Train or load a model first.")

        self.model.eval()
        X_val_tensor = torch.FloatTensor(X_val).to(self.device)

        with torch.no_grad():
            reconstructions = self.model(X_val_tensor)
            reconstruction_errors = torch.mean((X_val_tensor - reconstructions) ** 2, dim=(1, 2))
            reconstruction_errors = reconstruction_errors.cpu().numpy()

        self.threshold = np.percentile(reconstruction_errors, percentile)

        self.run.log({
            "threshold_percentile": percentile,
            "threshold_value": self.threshold,
            "val_reconstruction_error_mean": np.mean(reconstruction_errors),
            "val_reconstruction_error_std": np.std(reconstruction_errors)
        })

    def detect_anomalies(self, X_test: np.ndarray,
                        timestamps: Optional[np.ndarray] = None) -> Dict:
        """Detect anomalies in test data."""
        if self.model is None:
            raise ValueError("Model not available. Train or load a model first.")
        if self.threshold is None:
            raise ValueError("Threshold not set. Call set_threshold() first.")

        self.model.eval()
        X_test_tensor = torch.FloatTensor(X_test).to(self.device)

        with torch.no_grad():
            reconstructions = self.model(X_test_tensor)
            reconstruction_errors = torch.mean((X_test_tensor - reconstructions) ** 2, dim=(1, 2))
            reconstruction_errors = reconstruction_errors.cpu().numpy()

        # Detect anomalies
        anomalies = reconstruction_errors > self.threshold

        # Statistics
        num_anomalies = np.sum(anomalies)
        anomaly_rate = num_anomalies / len(X_test)

        self.anomaly_stats = {
            'total_samples': len(X_test),
            'anomalies_detected': int(num_anomalies),
            'anomaly_rate': float(anomaly_rate),
            'mean_reconstruction_error': float(np.mean(reconstruction_errors)),
            'std_reconstruction_error': float(np.std(reconstruction_errors)),
            'max_reconstruction_error': float(np.max(reconstruction_errors)),
            'min_reconstruction_error': float(np.min(reconstruction_errors))
        }

        # Log results
        self.run.log(self.anomaly_stats)

        return {
            'anomalies': anomalies,
            'reconstruction_errors': reconstruction_errors,
            'timestamps': timestamps,
            'statistics': self.anomaly_stats,
            'threshold': self.threshold
        }

    def create_training_visualizations(self):
        """Create training visualizations (only for train mode)."""
        if self.pipeline_mode != "train" or not self.training_history:
            return

        fig, axes = plt.subplots(1, 2, figsize=(15, 5))

        # Training history
        epochs = range(1, len(self.training_history['train_losses']) + 1)
        axes[0].plot(epochs, self.training_history['train_losses'], 'b-', label='Training Loss')
        axes[0].plot(epochs, self.training_history['val_losses'], 'r-', label='Validation Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss (MSE)')
        axes[0].set_title('Training History')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        # Loss distribution
        train_losses = self.training_history['train_losses']
        val_losses = self.training_history['val_losses']

        axes[1].hist(train_losses, bins=50, alpha=0.7, label='Training Loss', color='blue')
        axes[1].hist(val_losses, bins=50, alpha=0.7, label='Validation Loss', color='red')
        axes[1].set_xlabel('Loss Value')
        axes[1].set_ylabel('Frequency')
        axes[1].set_title('Loss Distribution')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()
        self.run.log({"training_visualization": wandb.Image(fig)})
        plt.close()

    def create_anomaly_visualizations(self, results: Dict, X_test: np.ndarray):
        """Create anomaly detection visualizations."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # Reconstruction errors over time
        ax1 = axes[0, 0]
        errors = results['reconstruction_errors']
        ax1.plot(errors, 'b-', alpha=0.7, linewidth=1)
        ax1.axhline(y=self.threshold, color='r', linestyle='--', label=f'Threshold: {self.threshold:.4f}')
        ax1.fill_between(range(len(errors)), errors, self.threshold,
                        where=(errors > self.threshold), alpha=0.3, color='red', label='Anomalies')
        ax1.set_xlabel('Sample Index')
        ax1.set_ylabel('Reconstruction Error')
        ax1.set_title('Reconstruction Errors Over Time')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Reconstruction error distribution
        ax2 = axes[0, 1]
        ax2.hist(errors, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        ax2.axvline(x=self.threshold, color='r', linestyle='--', label=f'Threshold: {self.threshold:.4f}')
        ax2.set_xlabel('Reconstruction Error')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Reconstruction Error Distribution')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # Anomaly detection summary
        ax3 = axes[1, 0]
        anomalies = results['anomalies']
        normal_count = np.sum(~anomalies)
        anomaly_count = np.sum(anomalies)

        labels = ['Normal', 'Anomaly']
        counts = [normal_count, anomaly_count]
        colors = ['lightgreen', 'lightcoral']

        wedges, texts, autotexts = ax3.pie(counts, labels=labels, colors=colors, autopct='%1.1f%%',
                                          startangle=90)
        ax3.set_title('Anomaly Detection Summary')

        # Sample reconstruction comparison (first few samples)
        ax4 = axes[1, 1]
        n_samples_to_show = min(5, len(X_test))

        if hasattr(self, 'model') and self.model is not None:
            self.model.eval()
            with torch.no_grad():
                X_sample = torch.FloatTensor(X_test[:n_samples_to_show]).to(self.device)
                reconstructions = self.model(X_sample).cpu().numpy()

            # Plot first feature of first few samples
            original_values = X_test[:n_samples_to_show, :, 0].flatten()
            reconstructed_values = reconstructions[:n_samples_to_show, :, 0].flatten()

            indices = range(len(original_values))
            ax4.plot(indices, original_values, 'b-', label='Original', alpha=0.7)
            ax4.plot(indices, reconstructed_values, 'r--', label='Reconstructed', alpha=0.7)
            ax4.set_xlabel('Time Step')
            ax4.set_ylabel('Feature Value')
            ax4.set_title('Sample Reconstruction (First Feature)')
            ax4.legend()
            ax4.grid(True, alpha=0.3)

        plt.tight_layout()
        self.run.log({f"anomaly_detection_visualization_{self.pipeline_mode}": wandb.Image(fig)})
        plt.close()

    def create_predictions_table(self, results: Dict, num_samples: int = 100):
        """Create a table of predictions for W&B."""
        anomalies = results['anomalies']
        errors = results['reconstruction_errors']
        timestamps = results.get('timestamps', np.arange(len(anomalies)))

        # Sample data for the table
        n_samples = min(num_samples, len(anomalies))
        indices = np.linspace(0, len(anomalies)-1, n_samples, dtype=int)

        table_data = []
        for i in indices:
            table_data.append({
                'sample_id': int(i),
                'timestamp': float(timestamps[i]) if timestamps is not None else i,
                'reconstruction_error': float(errors[i]),
                'is_anomaly': bool(anomalies[i]),
                'anomaly_score': float(errors[i] / self.threshold)
            })

        table = wandb.Table(dataframe=pd.DataFrame(table_data))
        self.run.log({f"predictions_table_{self.pipeline_mode}": table})

    def create_model_performance_summary(self, X_train: np.ndarray,
                                       X_val: np.ndarray, X_test: np.ndarray):
        """Create a comprehensive model performance summary."""
        summary_data = {
            'pipeline_mode': self.pipeline_mode,
            'model_architecture': {
                'input_dim': X_train.shape[2] if self.pipeline_mode == "train" else X_test.shape[2],
                'hidden_dim': self.model.hidden_dim if self.model else 'N/A',
                'num_layers': self.model.num_layers if self.model else 'N/A'
            },
            'data_shapes': {
                'train': str(X_train.shape) if self.pipeline_mode == "train" else 'N/A',
                'val': str(X_val.shape) if self.pipeline_mode == "train" else 'N/A',
                'test': str(X_test.shape)
            },
            'training_metrics': self.training_history if self.pipeline_mode == "train" else 'N/A',
            'anomaly_detection': self.anomaly_stats
        }

        # Convert to table
        flattened_data = []
        for category, data in summary_data.items():
            if isinstance(data, dict):
                for key, value in data.items():
                    flattened_data.append({
                        'category': category,
                        'metric': key,
                        'value': str(value)
                    })
            else:
                flattened_data.append({
                    'category': category,
                    'metric': 'value',
                    'value': str(data)
                })

        table = wandb.Table(dataframe=pd.DataFrame(flattened_data))
        self.run.log({f"model_performance_summary_{self.pipeline_mode}": table})

    def save_model_and_results(self, results: Dict, output_dir: str = "models"):
        """Save model and results."""
        os.makedirs(output_dir, exist_ok=True)

        if self.pipeline_mode == "train" and self.model is not None:
            # Save trained model
            model_path = f"{output_dir}/lstm_autoencoder_{self.pipeline_mode}.pth"
            torch.save({
                'model_state_dict': self.model.state_dict(),
                'threshold': self.threshold,
                'training_history': self.training_history,
                'config': {
                    'hidden_dim': self.model.hidden_dim,
                    'num_layers': self.model.num_layers,
                    'input_dim': self.model.input_dim
                },
                'input_dim': self.model.input_dim,
                'pipeline_mode': self.pipeline_mode
            }, model_path)

            # Create model artifact
            model_artifact = wandb.Artifact(f"lstm-anomaly-detector-{self.pipeline_mode}", type="model")
            model_artifact.add_file(model_path)
            model_artifact.metadata = {
                'threshold': float(self.threshold) if self.threshold else None,
                'pipeline_mode': self.pipeline_mode,
                'training_epochs': self.training_history.get('epochs', 0)
            }
            self.run.log_artifact(model_artifact)

        # Save results
        results_path = f"{output_dir}/anomaly_results_{self.pipeline_mode}.npz"
        np.savez(results_path,
                anomalies=results['anomalies'],
                reconstruction_errors=results['reconstruction_errors'],
                timestamps=results['timestamps'] if results['timestamps'] is not None else np.array([]),
                threshold=self.threshold,
                pipeline_mode=self.pipeline_mode)

        # Create results artifact
        results_artifact = wandb.Artifact(f"anomaly-detection-results-{self.pipeline_mode}", type="dataset")
        results_artifact.add_file(results_path)
        results_artifact.metadata = {
            'pipeline_mode': self.pipeline_mode,
            'anomalies_detected': int(results['statistics']['anomalies_detected']),
            'anomaly_rate': float(results['statistics']['anomaly_rate'])
        }
        self.run.log_artifact(results_artifact)

def main():
    import argparse

    parser = argparse.ArgumentParser(description="LSTM Anomaly Detection")
    parser.add_argument("--data-path", default="data/model_inputs.h5", help="Path to sequence data")
    parser.add_argument("--pipeline-mode", choices=["train", "test"], default="train",
                       help="Pipeline mode: 'train' or 'test'")
    parser.add_argument("--model-path", default="models/lstm_autoencoder_train.pth",
                       help="Path to pre-trained model (for test mode)")
    parser.add_argument("--epochs", type=int, default=100, help="Training epochs (train mode)")
    parser.add_argument("--batch-size", type=int, default=32, help="Batch size")
    parser.add_argument("--hidden-dim", type=int, default=64, help="LSTM hidden dimension")
    parser.add_argument("--threshold-percentile", type=float, default=95, help="Anomaly threshold percentile")

    args = parser.parse_args()

    # Initialize W&B
    run = wandb.init(project="predictive-maintenance", job_type="anomaly-detection")

    # Create detector with pipeline mode
    detector = AnomalyDetector(run, pipeline_mode=args.pipeline_mode)

    # Load data
    sequences = detector.load_sequences(args.data_path)

    if args.pipeline_mode == "train":
        # Training mode
        if 'X_train' not in sequences or 'X_val' not in sequences:
            raise ValueError("Training data (X_train, X_val) not found in data file")

        X_train = sequences['X_train']
        X_val = sequences['X_val']

        # Create and train model
        input_dim = X_train.shape[2]
        config = {
            'hidden_dim': args.hidden_dim,
            'num_layers': 2,
            'dropout': 0.2
        }

        detector.create_model(input_dim, config)
        detector.train_model(X_train, X_val, epochs=args.epochs, batch_size=args.batch_size)

        # Set threshold
        detector.set_threshold(X_val, percentile=args.threshold_percentile)

        # Create training visualizations
        detector.create_training_visualizations()

        # Save model
        detector.save_model_and_results({
            'anomalies': np.array([]),
            'reconstruction_errors': np.array([]),
            'timestamps': None,
            'statistics': {'anomalies_detected': 0, 'anomaly_rate': 0.0}
        })

    elif args.pipeline_mode == "test":
        # Testing mode
        if 'X_test' not in sequences:
            raise ValueError("Test data (X_test) not found in data file")

        X_test = sequences['X_test']
        timestamps_test = sequences.get('timestamps_test', None)

        # Load pre-trained model
        detector.load_pretrained_model(args.model_path)

        # Detect anomalies
        results = detector.detect_anomalies(X_test, timestamps_test)

        # Create visualizations
        detector.create_anomaly_visualizations(results, X_test)
        detector.create_predictions_table(results)
        detector.create_model_performance_summary(np.array([]), np.array([]), X_test)

        # Save results
        detector.save_model_and_results(results)

        # Print summary
        print(f"\n🎯 ANOMALY DETECTION COMPLETE ({args.pipeline_mode.upper()} MODE)")
        print(f"   Total Anomalies Detected: {results['statistics']['anomalies_detected']}")
        print(f"   Anomaly Rate: {results['statistics']['anomaly_rate']:.2%}")

    run.finish()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile src/monitoring/post_detection.py
import pandas as pd
import numpy as np
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List
from datetime import datetime

class PostDetectionMonitor:
    """Post-detection monitoring and analysis with W&B tracking."""

    def __init__(self, wandb_run: wandb.run = None):
        self.run = wandb_run or wandb.init(project="predictive-maintenance")
        self.monitoring_stats = {}

    def analyze_reconstruction_errors(self, results: Dict):
        """Analyze reconstruction error distribution and patterns."""
        errors = results['reconstruction_errors']
        anomalies = results['anomalies']

        # Calculate error statistics
        error_stats = {
            'mean_error': float(np.mean(errors)),
            'std_error': float(np.std(errors)),
            'median_error': float(np.median(errors)),
            'max_error': float(np.max(errors)),
            'min_error': float(np.min(errors)),
            'error_range': float(np.max(errors) - np.min(errors)),
            'error_iqr': float(np.percentile(errors, 75) - np.percentile(errors, 25))
        }

        # Separate statistics for normal vs anomaly
        normal_errors = errors[~anomalies]
        anomaly_errors = errors[anomalies]

        if len(normal_errors) > 0:
            error_stats['normal_mean_error'] = float(np.mean(normal_errors))
            error_stats['normal_std_error'] = float(np.std(normal_errors))
            error_stats['normal_max_error'] = float(np.max(normal_errors))

        if len(anomaly_errors) > 0:
            error_stats['anomaly_mean_error'] = float(np.mean(anomaly_errors))
            error_stats['anomaly_std_error'] = float(np.std(anomaly_errors))
            error_stats['anomaly_min_error'] = float(np.min(anomaly_errors))

        # Log statistics
        for key, value in error_stats.items():
            self.run.log({f"post_detection_{key}": value})

        self.monitoring_stats.update(error_stats)

        # Create reconstruction error histogram
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

        # Overall error distribution
        ax1.hist(errors, bins=50, alpha=0.7, color='blue', edgecolor='black')
        ax1.axvline(np.mean(errors), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(errors):.2f}')
        ax1.axvline(np.median(errors), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(errors):.2f}')
        ax1.set_xlabel('Reconstruction Error')
        ax1.set_ylabel('Frequency')
        ax1.set_title('Overall Reconstruction Error Distribution')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Log-scale histogram for better visibility
        ax2.hist(errors, bins=50, alpha=0.7, color='purple', edgecolor='black')
        ax2.set_yscale('log')
        ax2.set_xlabel('Reconstruction Error')
        ax2.set_ylabel('Frequency (log scale)')
        ax2.set_title('Reconstruction Error Distribution (Log Scale)')
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        self.run.log({"reconstruction_error_histograms": wandb.Image(fig)})
        plt.close()

        # Create error percentile plot
        fig, ax = plt.subplots(figsize=(10, 6))
        percentiles = np.arange(0, 101, 1)
        error_percentiles = np.percentile(errors, percentiles)

        ax.plot(percentiles, error_percentiles, 'b-', linewidth=2)
        ax.fill_between(percentiles, 0, error_percentiles, alpha=0.3)
        ax.set_xlabel('Percentile')
        ax.set_ylabel('Reconstruction Error')
        ax.set_title('Reconstruction Error Percentile Distribution')
        ax.grid(True, alpha=0.3)

        # Mark key percentiles
        key_percentiles = [50, 75, 90, 95, 99]
        for p in key_percentiles:
            val = np.percentile(errors, p)
            ax.axhline(val, color='red', linestyle=':', alpha=0.5)
            ax.text(5, val, f'P{p}: {val:.2f}', fontsize=9)

        self.run.log({"error_percentile_distribution": wandb.Image(fig)})
        plt.close()

    def track_anomaly_patterns(self, results: Dict):
        """Track and visualize anomaly patterns over time."""
        anomalies = results['anomalies']
        errors = results['reconstruction_errors']
        timestamps = results.get('timestamps', None)

        # Create anomaly rate tracking
        window_size = 20  # Rolling window for anomaly rate
        anomaly_rate_rolling = pd.Series(anomalies.astype(int)).rolling(window_size).mean()

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

        # Anomaly rate over time
        x_axis = timestamps if timestamps is not None else np.arange(len(anomalies))

        ax1.plot(x_axis, anomaly_rate_rolling, 'r-', linewidth=2, label=f'Rolling Anomaly Rate (window={window_size})')
        ax1.fill_between(x_axis, 0, anomaly_rate_rolling, alpha=0.3, color='red')
        ax1.set_ylabel('Anomaly Rate')
        ax1.set_title('Anomaly Rate Over Time')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Reconstruction error over time with anomaly markers
        ax2.scatter(x_axis[~anomalies], errors[~anomalies], c='blue', alpha=0.5, s=20, label='Normal')
        ax2.scatter(x_axis[anomalies], errors[anomalies], c='red', alpha=0.8, s=40, marker='x', label='Anomaly')
        ax2.set_xlabel('Time' if timestamps is not None else 'Sequence Index')
        ax2.set_ylabel('Reconstruction Error')
        ax2.set_title('Reconstruction Error Timeline with Anomaly Markers')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        self.run.log({"anomaly_patterns_timeline": wandb.Image(fig)})
        plt.close()

        # Calculate anomaly clustering metrics
        if np.any(anomalies):
            anomaly_indices = np.where(anomalies)[0]
            if len(anomaly_indices) > 1:
                # Calculate distances between consecutive anomalies
                anomaly_distances = np.diff(anomaly_indices)

                clustering_metrics = {
                    'avg_anomaly_distance': float(np.mean(anomaly_distances)),
                    'min_anomaly_distance': int(np.min(anomaly_distances)),
                    'max_anomaly_distance': int(np.max(anomaly_distances)),
                    'anomaly_clustering_score': float(np.std(anomaly_distances))  # Lower = more clustered
                }

                self.monitoring_stats.update(clustering_metrics)

                # Log clustering metrics
                for key, value in clustering_metrics.items():
                    self.run.log({f"post_detection_{key}": value})

    def create_temporal_analysis(self, results: Dict):
        """Analyze temporal aspects of anomalies."""
        anomalies = results['anomalies']
        errors = results['reconstruction_errors']

        # Create hourly aggregation if possible
        if 'timestamps' in results and results['timestamps'] is not None:
            timestamps = results['timestamps']

            # Convert timestamps to datetime if they're numeric
            if isinstance(timestamps[0], (int, float)):
                timestamps = pd.to_datetime(timestamps, unit='s')

            # Create dataframe for temporal analysis
            df = pd.DataFrame({
                'timestamp': timestamps,
                'error': errors,
                'is_anomaly': anomalies
            })

            # Extract temporal features
            df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
            df['day_of_week'] = pd.to_datetime(df['timestamp']).dt.dayofweek

            # Aggregate by hour
            hourly_stats = df.groupby('hour').agg({
                'is_anomaly': ['sum', 'mean'],
                'error': ['mean', 'std', 'max']
            })

            # Create temporal visualization
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

            # Anomaly rate by hour
            hours = hourly_stats.index
            anomaly_rates = hourly_stats[('is_anomaly', 'mean')]

            ax1.bar(hours, anomaly_rates, color='coral', alpha=0.7)
            ax1.set_xlabel('Hour of Day')
            ax1.set_ylabel('Anomaly Rate')
            ax1.set_title('Anomaly Rate by Hour of Day')
            ax1.grid(True, alpha=0.3)

            # Mean error by hour
            mean_errors = hourly_stats[('error', 'mean')]
            std_errors = hourly_stats[('error', 'std')]

            ax2.plot(hours, mean_errors, 'b-', linewidth=2, label='Mean Error')
            ax2.fill_between(hours,
                            mean_errors - std_errors,
                            mean_errors + std_errors,
                            alpha=0.3, label='±1 STD')
            ax2.set_xlabel('Hour of Day')
            ax2.set_ylabel('Mean Reconstruction Error')
            ax2.set_title('Reconstruction Error by Hour of Day')
            ax2.legend()
            ax2.grid(True, alpha=0.3)

            plt.tight_layout()
            self.run.log({"temporal_analysis": wandb.Image(fig)})
            plt.close()

    def generate_summary_report(self, results: Dict):
        """Generate comprehensive post-detection summary report."""
        # Create summary statistics table
        summary_data = {
            'Metric': [],
            'Value': []
        }

        # Basic statistics
        summary_data['Metric'].extend([
            'Total Sequences',
            'Anomalies Detected',
            'Anomaly Rate (%)',
            'Mean Reconstruction Error',
            'Max Reconstruction Error',
            'Threshold Used'
        ])

        summary_data['Value'].extend([
            results['statistics']['total_samples'],
            results['statistics']['anomalies_detected'],
            f"{results['statistics']['anomaly_rate'] * 100:.2f}",
            f"{np.mean(results['reconstruction_errors']):.4f}",
            f"{np.max(results['reconstruction_errors']):.4f}",
            f"{results.get('threshold', 'N/A')}"
        ])

        # Add monitoring statistics
        if self.monitoring_stats:
            for key, value in self.monitoring_stats.items():
                if key not in ['mean_error', 'max_error']:  # Avoid duplicates
                    metric_name = key.replace('_', ' ').title()
                    summary_data['Metric'].append(metric_name)
                    summary_data['Value'].append(f"{value:.4f}" if isinstance(value, float) else str(value))

        # Create summary table
        summary_df = pd.DataFrame(summary_data)
        summary_table = wandb.Table(dataframe=summary_df)
        self.run.log({"post_detection_summary": summary_table})

        # Create final summary visualization
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

        # Error distribution comparison
        errors = results['reconstruction_errors']
        anomalies = results['anomalies']

        if np.any(~anomalies):
            ax1.hist(errors[~anomalies], bins=30, alpha=0.7, label='Normal', color='blue', density=True)
        if np.any(anomalies):
            ax1.hist(errors[anomalies], bins=30, alpha=0.7, label='Anomaly', color='red', density=True)
        ax1.set_xlabel('Reconstruction Error')
        ax1.set_ylabel('Density')
        ax1.set_title('Error Distribution: Normal vs Anomaly')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Anomaly detection effectiveness
        if np.any(anomalies) and np.any(~anomalies):
            data = [errors[~anomalies], errors[anomalies]]
            positions = [1, 2]

            bp = ax2.boxplot(data, positions=positions, widths=0.6, patch_artist=True)
            bp['boxes'][0].set_facecolor('blue')
            bp['boxes'][1].set_facecolor('red')
            ax2.set_xticklabels(['Normal', 'Anomaly'])
            ax2.set_ylabel('Reconstruction Error')
            ax2.set_title('Error Distribution Comparison')
            ax2.grid(True, alpha=0.3)

        # Cumulative anomaly detection
        cumulative_anomalies = np.cumsum(anomalies)
        x_axis = np.arange(len(anomalies))

        ax3.plot(x_axis, cumulative_anomalies, 'r-', linewidth=2)
        ax3.fill_between(x_axis, 0, cumulative_anomalies, alpha=0.3, color='red')
        ax3.set_xlabel('Sequence Index')
        ax3.set_ylabel('Cumulative Anomalies')
        ax3.set_title('Cumulative Anomaly Detection')
        ax3.grid(True, alpha=0.3)

        # Detection confidence distribution
        # Use distance from threshold as confidence measure
        threshold = results.get('threshold', np.percentile(errors, 95))
        confidence = np.abs(errors - threshold) / threshold

        ax4.hist(confidence[anomalies] if np.any(anomalies) else [],
                bins=30, alpha=0.7, color='green', edgecolor='black')
        ax4.set_xlabel('Detection Confidence (Relative Distance from Threshold)')
        ax4.set_ylabel('Frequency')
        ax4.set_title('Anomaly Detection Confidence Distribution')
        ax4.grid(True, alpha=0.3)

        plt.tight_layout()
        self.run.log({"post_detection_summary_plots": wandb.Image(fig)})
        plt.close()

        # Log final metrics
        self.run.log({
            "post_detection_completed": True,
            "monitoring_metrics_generated": len(self.monitoring_stats)
        })

In [ ]:
'''%%writefile src/pipeline.py
#!/usr/bin/env python
"""
End-to-end pipeline orchestration with W&B Pipelines
"""

import os
import sys
import wandb
import argparse
import logging
from pathlib import Path
import time
from typing import Dict, Any

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class PredictiveMaintenancePipeline:
    """Orchestrate the complete preprocessing pipeline with W&B."""

    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.artifacts = {}
        self.metrics = {}

    def run_step(self, step_name: str, job_type: str, run_function, **kwargs):
        """Run a single pipeline step with W&B tracking."""
        logger.info(f"Starting step: {step_name}")

        # Initialize W&B run for this step
        run = wandb.init(
            project=self.config['project'],
            job_type=job_type,
            name=f"{step_name}-{int(time.time())}",
            reinit=True
        )

        try:
            # Execute step
            result = run_function(run, **kwargs)

            # Store artifacts and metrics
            if isinstance(result, dict):
                if 'artifact' in result:
                    self.artifacts[step_name] = result['artifact']
                if 'metrics' in result:
                    self.metrics[step_name] = result['metrics']

            logger.info(f"Completed step: {step_name}")

        except Exception as e:
            logger.error(f"Error in step {step_name}: {str(e)}")
            wandb.alert(
                title=f"Pipeline Step Failed: {step_name}",
                text=str(e),
                level=wandb.AlertLevel.ERROR
            )
            raise

        finally:
            run.finish()

    def step_1_ingestion(self, run):
        """Step 1: Data Ingestion & Time Alignment"""
        from src.ingestion.loader import DataLoader

        loader = DataLoader(run)
        df = loader.load_csv_files(self.config['input_files'], 'timestamp')
        df = loader.resample_to_frequency(df, self.config['resample_freq'])
        loader.save_and_log_artifact(df, "data/unified.parquet")

        return {'artifact': 'raw-data:latest'}

    def step_2_quality_checks(self, run):
        """Step 2: Initial Quality Checks"""
        from src.cleaning.quality_checks import QualityChecker

        # Load previous artifact
        artifact = run.use_artifact(self.artifacts.get('ingestion', 'raw-data:latest'))
        artifact_dir = artifact.download()
        df = pd.read_parquet(f"{artifact_dir}/unified.parquet")

        checker = QualityChecker(run)
        limits = checker.load_limits("configs/limits.json")
        df = checker.remove_duplicates(df)
        df = checker.filter_by_limits(df, limits)
        df, report = checker.profile_missingness(df)
        checker.save_quality_artifacts(df, report, "data")
        df.to_parquet("data/cleaned.parquet")

        return {'artifact': 'quality-metrics:latest'}

    def step_3_time_features(self, run):
        """Step 3: Timestamp & Categorical Feature Engineering"""
        from src.cleaning.time_features import TimeFeatureEngineer

        df = pd.read_parquet("data/cleaned.parquet")
        engineer = TimeFeatureEngineer(run)
        df = engineer.add_time_features(df)
        df.to_parquet("data/time_features.parquet")

        artifact = wandb.Artifact("time-features", type="dataset")
        artifact.add_file("data/time_features.parquet")
        run.log_artifact(artifact)

        return {'artifact': 'time-features:latest'}

    def step_4_imputation(self, run):
        """Step 4: Missing-Value Imputation"""
        from src.cleaning.impute import MissingValueImputer

        df = pd.read_parquet("data/time_features.parquet")
        imputer = MissingValueImputer(run)
        df = imputer.impute_missing(df, short_limit=self.config['impute_short_limit'])
        df.to_parquet("data/imputed.parquet")

        artifact = wandb.Artifact("imputed-data", type="dataset")
        artifact.add_file("data/imputed.parquet")
        run.log_artifact(artifact)

        return {'artifact': 'imputed-data:latest'}

    def step_5_outlier_flagging(self, run):
        """Step 5: Minimal 'Safe' Outlier Flagging"""
        from src.cleaning.outlier_flags import OutlierFlagger

        df = pd.read_parquet("data/imputed.parquet")
        flagger = OutlierFlagger(run)
        config = flagger.load_config("configs/outlier_config.json")
        df = flagger.flag_spikes(df, config)
        df.to_parquet("data/flagged.parquet")

        artifact = wandb.Artifact("flagged-data", type="dataset")
        artifact.add_file("data/flagged.parquet")
        run.log_artifact(artifact)

        return {'artifact': 'flagged-data:latest'}

    def step_6_windowing(self, run):
        """Step 6: Local Windowing"""
        from src.windowing.segment import WindowSegmenter

        df = pd.read_parquet("data/flagged.parquet")
        segmenter = WindowSegmenter(run)
        segmenter.log_sample_windows(
            df,
            self.config['window_size'],
            self.config['stride'],
            self.config['num_sample_windows']
        )
        segmenter.save_windows_metadata()

        return {'artifact': 'windows-metadata:latest'}

    def step_7_baseline_cleaning(self, run):
        """Step 7: Window-Level Baseline Cleaning"""
        from src.cleaning.window_clean import WindowCleaner

        df = pd.read_parquet("data/flagged.parquet")
        cleaner = WindowCleaner(run)
        cleaned_windows, stats = cleaner.clean_baseline_windows(
            df,
            self.config['window_size'],
            self.config['stride'],
            cleaning_method=self.config['baseline_cleaning_method']
        )
        cleaner.save_models_and_stats()

        return {'artifact': 'baseline-statistics:latest'}

    def step_8_decomposition(self, run):
        """Step 8: Seasonal & Trend Decomposition"""
        from src.features.decompose import TimeSeriesDecomposer

        df = pd.read_parquet("data/flagged.parquet")
        decomposer = TimeSeriesDecomposer(run)
        decompositions = decomposer.decompose_windows(
            df,
            self.config['decomposition_window_size'],
            self.config['decomposition_stride'],
            self.config['num_decomposition_examples']
        )
        decomposer.save_decomposition_artifacts(decompositions)

        return {'artifact': 'decomposition-stats:latest'}

    def step_9_feature_extraction(self, run):
        """Step 9: Window-Level Feature Extraction"""
        from src.features.extract import FeatureExtractor

        df = pd.read_parquet("data/flagged.parquet")
        extractor = FeatureExtractor(run)
        feature_matrix = extractor.extract_window_features(
            df,
            self.config['window_size'],
            self.config['stride']
        )
        extractor.save_feature_matrix(feature_matrix, "features/windows_features.csv")

        return {'artifact': 'feature-matrix:latest'}

    def step_10_scaling(self, run):
        """Step 10: Scaling & Normalization"""
        from src.scaling.scale import FeatureScaler

        features = pd.read_csv("features/windows_features.csv")
        n_baseline = int(len(features) * self.config['baseline_ratio'])
        baseline_features = features.iloc[:n_baseline]

        scaler = FeatureScaler(run)
        scaler.fit_scaler(baseline_features, method=self.config['scaling_method'])
        scaled_features = scaler.apply_scaler(features)
        scaled_baseline = scaled_features.iloc[:n_baseline]

        scaler.validate_scaling(scaled_baseline, scaled_features)
        scaled_features.to_csv("features/scaled_features.csv", index=False)
        scaler.save_scaler()

        artifact = wandb.Artifact("scaled-features", type="dataset")
        artifact.add_file("features/scaled_features.csv")
        run.log_artifact(artifact)

        return {'artifact': 'scaled-features:latest'}

    def step_11_feature_selection(self, run):
        """Step 11: Dimensionality Reduction & Feature Selection"""
        from src.features.select_reduce import FeatureSelector

        features = pd.read_csv("features/scaled_features.csv")
        selector = FeatureSelector(run)
        reduced_features = selector.combine_selection_methods(
            features,
            variance_threshold=self.config['variance_threshold'],
            mi_top_k=self.config['mi_top_k'],
            pca_variance=self.config['pca_variance']
        )

        reduced_features.to_csv("features/reduced_features.csv", index=False)
        selector.save_models()

        artifact = wandb.Artifact("reduced-features", type="dataset")
        artifact.add_file("features/reduced_features.csv")
        artifact.metadata = selector.selection_stats
        run.log_artifact(artifact)

        return {'artifact': 'reduced-features:latest'}

    def step_12_sequence_assembly(self, run):
        """Step 12: Sequence Assembly & Model Input"""
        from src.model_input.assemble import SequenceAssembler

        features = pd.read_csv("features/reduced_features.csv")
        assembler = SequenceAssembler(run)

        # LSTM format
        X_lstm, timestamps = assembler.create_sequences_lstm(
            features,
            self.config['sequence_length'],
            self.config['sequence_overlap']
        )

        # Tabular format
        X_tabular = assembler.create_tabular_format(features)

        # Create splits
        lstm_splits = assembler.create_train_val_test_split(
            X_lstm, timestamps,
            self.config['train_ratio'],
            self.config['val_ratio']
        )

        tabular_splits = assembler.create_train_val_test_split(
            X_tabular,
            train_ratio=self.config['train_ratio'],
            val_ratio=self.config['val_ratio']
        )

        # Combine and save
        all_sequences = {}
        all_sequences.update(lstm_splits)
        all_sequences.update({f"tabular_{k}": v for k, v in tabular_splits.items()})

        assembler.save_sequences(all_sequences, "data")

        return {'artifact': 'model-inputs-h5:latest'}

    def step_13_drift_monitoring(self, run):
        """Step 14: Drift Monitoring & Retraining Hooks"""
        from src.monitoring.drift import DriftMonitor

        # This step would typically run on new data
        # For pipeline demo, we'll use a portion of existing data
        features = pd.read_csv("features/reduced_features.csv")
        n_baseline = int(len(features) * 0.7)
        baseline_features = features.iloc[:n_baseline]
        current_features = features.iloc[n_baseline:]

        monitor = DriftMonitor(run)
        drift_results = monitor.monitor_feature_drift(baseline_features, current_features)
        severity_results = monitor.evaluate_drift_severity(drift_results)

        monitor.create_drift_dashboard(drift_results, severity_results)
        retrain_needed = monitor.generate_retraining_sweep(severity_results)
        monitor.save_drift_report(drift_results, severity_results, "monitoring")

        return {
            'metrics': {
                'retrain_needed': retrain_needed,
                'high_drift_features': sum(1 for s in severity_results.values() if s == 'high')
            }
        }

    def step_14_lstm_anomaly_detection(self, run):
        """Step 14: LSTM Anomaly Detection"""
        from src.model.lstm_anomaly_detector import AnomalyDetector

        # Load sequences
        detector = AnomalyDetector(run)
        sequences = detector.load_sequences("data/model_inputs.h5")

        # Get train/val/test splits
        X_train = sequences['X_train']
        X_val = sequences['X_val']
        X_test = sequences['X_test']
        timestamps_test = sequences.get('timestamps_test', None)

        # Create and train model
        input_dim = X_train.shape[2]
        config = {
            'hidden_dim': self.config.get('lstm_hidden_dim', 64),
            'num_layers': self.config.get('lstm_num_layers', 2),
            'dropout': self.config.get('lstm_dropout', 0.2),
            'learning_rate': self.config.get('lstm_learning_rate', 0.001)
        }

        detector.create_model(input_dim, config)
        detector.train_model(
            X_train, X_val,
            epochs=self.config.get('lstm_epochs', 50),
            batch_size=self.config.get('lstm_batch_size', 32)
        )

        # Set threshold and detect anomalies
        detector.set_threshold(X_val, percentile=self.config.get('anomaly_threshold_percentile', 95))
        results = detector.detect_anomalies(X_test, timestamps_test)

        # Create all visualizations
        detector.create_training_visualizations()
        detector.create_anomaly_visualizations(results, X_test)
        detector.create_predictions_table(results, num_samples=200)
        detector.create_model_performance_summary(X_train, X_val, X_test)

        # Save model and results
        detector.save_model_and_results(results)

        # Store results
        anomalies_detected = results['statistics']['anomalies_detected']
        anomaly_rate = results['statistics']['anomaly_rate']

        logger.info(f"🎯 ANOMALY DETECTION COMPLETE")
        logger.info(f"   Total Anomalies Detected: {anomalies_detected}")
        logger.info(f"   Anomaly Rate: {anomaly_rate:.2%}")

        return {
            'artifact': 'lstm-anomaly-detector:latest',
            'metrics': {
                'anomalies_detected': anomalies_detected,
                'anomaly_rate': anomaly_rate,
                'total_samples': results['statistics']['total_samples']
            }
        }

    def run_pipeline(self):
        """Execute the complete pipeline with W&B orchestration."""
        logger.info("Starting Predictive Maintenance Pipeline")
        start_time = time.time()

        # Create pipeline run
        pipeline_run = wandb.init(
            project=self.config['project'],
            job_type="pipeline-orchestration",
            name=f"pipeline-{int(time.time())}",
            config=self.config
        )

        try:
            # Execute steps in sequence
            steps = [
                ("ingestion", "data-ingestion", self.step_1_ingestion),
                ("quality_checks", "quality-checks", self.step_2_quality_checks),
                ("time_features", "feature-engineering", self.step_3_time_features),
                ("imputation", "imputation", self.step_4_imputation),
                ("outlier_flagging", "outlier-detection", self.step_5_outlier_flagging),
                ("windowing", "windowing", self.step_6_windowing),
                ("baseline_cleaning", "baseline-cleaning", self.step_7_baseline_cleaning),
                ("decomposition", "decomposition", self.step_8_decomposition),
                ("feature_extraction", "feature-extraction", self.step_9_feature_extraction),
                ("scaling", "feature-scaling", self.step_10_scaling),
                ("feature_selection", "feature-selection", self.step_11_feature_selection),
                ("sequence_assembly", "sequence-assembly", self.step_12_sequence_assembly),
                ("drift_monitoring", "drift-monitoring", self.step_13_drift_monitoring),  # Step 13
                ("lstm_anomaly_detection", "anomaly-detection", self.step_14_lstm_anomaly_detection)  # Step 14
            ]

            for step_name, job_type, step_function in steps:
                self.run_step(step_name, job_type, step_function)

            # Log pipeline summary
            pipeline_time = time.time() - start_time

            # Get anomaly detection results
            anomaly_metrics = self.metrics.get('lstm_anomaly_detection', {})

            pipeline_run.log({
                "pipeline_duration_minutes": pipeline_time / 60,
                "pipeline_status": "completed",
                "total_steps": len(steps),
                "final_anomalies_detected": anomaly_metrics.get('anomalies_detected', 0),
                "final_anomaly_rate": anomaly_metrics.get('anomaly_rate', 0)
            })

            # Create pipeline summary table
            summary_data = []
            for step_name, _, _ in steps:
                summary_data.append({
                    'step': step_name,
                    'artifact': self.artifacts.get(step_name, 'N/A'),
                    'status': 'completed'
                })

            summary_table = wandb.Table(dataframe=pd.DataFrame(summary_data))
            pipeline_run.log({"pipeline_summary": summary_table})

            logger.info(f"Pipeline completed successfully in {pipeline_time:.2f} seconds")
            logger.info(f"🎯 Final Results: {anomaly_metrics.get('anomalies_detected', 0)} anomalies detected ({anomaly_metrics.get('anomaly_rate', 0):.2%} of test data)")

            # Check if retraining is needed
            if self.metrics.get('drift_monitoring', {}).get('retrain_needed', False):
                logger.warning("Drift detected - retraining recommended!")
                wandb.alert(
                    title="Pipeline Complete - Retraining Needed",
                    text="High drift detected. Please review the sweep configuration and initiate retraining.",
                    level=wandb.AlertLevel.WARN
                )

        except Exception as e:
            logger.error(f"Pipeline failed: {str(e)}")
            pipeline_run.log({"pipeline_status": "failed", "error": str(e)})
            raise

        finally:
            pipeline_run.finish()

        return self.artifacts, self.metrics

# Default configuration
DEFAULT_CONFIG = {
    'project': 'predictive-maintenance',
    'input_files': ['data/sample_sensors.csv'],
    'resample_freq': '1T',
    'impute_short_limit': 5,
    'window_size': 60,
    'stride': 30,
    'num_sample_windows': 3,
    'baseline_cleaning_method': 'mad',
    'decomposition_window_size': 720,
    'decomposition_stride': 360,
    'num_decomposition_examples': 3,
    'baseline_ratio': 0.2,
    'scaling_method': 'robust',
    'variance_threshold': 0.01,
    'mi_top_k': 100,
    'pca_variance': 0.95,
    'sequence_length': 10,
    'sequence_overlap': 0.5,
    'train_ratio': 0.7,
    'val_ratio': 0.15,
    # LSTM parameters
    'lstm_hidden_dim': 64,
    'lstm_num_layers': 2,
    'lstm_dropout': 0.2,
    'lstm_learning_rate': 0.001,
    'lstm_epochs': 50,
    'lstm_batch_size': 32,
    'anomaly_threshold_percentile': 95
}

def main():
    parser = argparse.ArgumentParser(description="Run predictive maintenance pipeline")
    parser.add_argument("--config", type=str, help="Path to config file")
    parser.add_argument("--project", type=str, default="predictive-maintenance",
                       help="W&B project name")
    parser.add_argument("--dry-run", action="store_true",
                       help="Run without executing steps")

    args = parser.parse_args()

    # Load configuration
    config = DEFAULT_CONFIG.copy()
    if args.config:
        import yaml
        with open(args.config, 'r') as f:
            config.update(yaml.safe_load(f))

    config['project'] = args.project

    # Run pipeline
    pipeline = PredictiveMaintenancePipeline(config)
    artifacts, metrics = pipeline.run_pipeline()

    logger.info("Pipeline execution complete!")
    logger.info(f"Artifacts: {list(artifacts.keys())}")
    logger.info(f"Metrics: {metrics}")

if __name__ == "__main__":
    import pandas as pd
    main()'''

In [ ]:
%%writefile run_pipeline.py
#!/usr/bin/env python
"""
Complete pipeline execution script for Google Colab
Orchestrates predictive maintenance pipeline with separate train/test processing
"""

import os
import sys
import wandb
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import logging
import time

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Setup paths for Colab
os.chdir('/content/predictive-maintenance')
sys.path.append('/content/predictive-maintenance')

# Import all modules
from src.wandb_init import setup_wandb_colab, init_wandb
from src.ingestion.loader import DataLoader
from src.cleaning.quality_checks import QualityChecker
from src.cleaning.time_features import TimeFeatureEngineer
from src.cleaning.impute import MissingValueImputer
from src.cleaning.outlier_flags import OutlierFlagger
from src.windowing.segment import WindowSegmenter
from src.cleaning.window_clean import WindowCleaner
from src.features.decompose import TimeSeriesDecomposer
from src.features.extract import FeatureExtractor
from src.scaling.scale import FeatureScaler
from src.features.select_reduce import FeatureSelector
from src.model_input.assemble import SequenceAssembler
from src.monitoring.drift import DriftMonitor
from src.model.lstm_anomaly_detector import AnomalyDetector
from src.monitoring.post_detection import PostDetectionMonitor

def run_preprocessing_pipeline(input_file: str, pipeline_mode: str = "train"):
    """
    Run steps 1-12 (or 1-13 for test) of the preprocessing pipeline.

    Args:
        input_file: Path to input CSV file
        pipeline_mode: "train" or "test" to determine pipeline steps

    Returns:
        Path to the final preprocessed data
    """
    logger.info(f"\n{'='*60}")
    logger.info(f"Running preprocessing pipeline in {pipeline_mode.upper()} mode")
    logger.info(f"Input file: {input_file}")
    logger.info(f"{'='*60}\n")

    # Step 1: Data Ingestion
    logger.info("\n📥 Step 1: Data Ingestion & Time Alignment")
    run = init_wandb(run_name=f"step1-data-ingestion-{pipeline_mode}")
    loader = DataLoader(run)
    df = loader.load_csv_files([input_file], 'timestamp')
    df = loader.resample_to_frequency(df, '1T')
    loader.save_and_log_artifact(df, f"data/unified_{pipeline_mode}.parquet")
    run.finish()

    # Step 2: Quality Checks
    logger.info("\n✅ Step 2: Initial Quality Checks")
    run = init_wandb(run_name=f"step2-quality-checks-{pipeline_mode}")
    checker = QualityChecker(run)
    df = pd.read_parquet(f"data/unified_{pipeline_mode}.parquet")
    #limits = checker.load_limits("configs/limits.json")
    df = checker.remove_duplicates(df)
    #df = checker.filter_by_limits(df, limits)
    df, report = checker.profile_missingness(df)
    checker.save_quality_artifacts(df, report, f"data/{pipeline_mode}")
    df.to_parquet(f"data/cleaned_{pipeline_mode}.parquet")
    run.finish()

    # Step 3: Time Features
    logger.info("\n⏰ Step 3: Timestamp & Categorical Feature Engineering")
    run = init_wandb(run_name=f"step3-time-features-{pipeline_mode}")
    engineer = TimeFeatureEngineer(run)
    df = pd.read_parquet(f"data/cleaned_{pipeline_mode}.parquet")
    df = engineer.add_time_features(df)
    df.to_parquet(f"data/time_features_{pipeline_mode}.parquet")
    artifact = wandb.Artifact(f"time-features-{pipeline_mode}", type="dataset")
    artifact.add_file(f"data/time_features_{pipeline_mode}.parquet")
    run.log_artifact(artifact)
    run.finish()

    # Step 4: Imputation
    logger.info("\n🔧 Step 4: Missing-Value Imputation")
    run = init_wandb(run_name=f"step4-imputation-{pipeline_mode}")
    imputer = MissingValueImputer(run)
    df = pd.read_parquet(f"data/time_features_{pipeline_mode}.parquet")
    df = imputer.impute_missing(df, short_limit=5)
    df.to_parquet(f"data/imputed_{pipeline_mode}.parquet")
    artifact = wandb.Artifact(f"imputed-data-{pipeline_mode}", type="dataset")
    artifact.add_file(f"data/imputed_{pipeline_mode}.parquet")
    run.log_artifact(artifact)
    run.finish()

    # Step 5: Outlier Flagging
    logger.info("\n🚨 Step 5: Minimal 'Safe' Outlier Flagging")
    run = init_wandb(run_name=f"step5-outlier-flagging-{pipeline_mode}")
    flagger = OutlierFlagger(run)
    df = pd.read_parquet(f"data/imputed_{pipeline_mode}.parquet")
    config = flagger.load_config("configs/outlier_config.json")
    df = flagger.flag_spikes(df, config)
    df.to_parquet(f"data/flagged_{pipeline_mode}.parquet")
    artifact = wandb.Artifact(f"flagged-data-{pipeline_mode}", type="dataset")
    artifact.add_file(f"data/flagged_{pipeline_mode}.parquet")
    run.log_artifact(artifact)
    run.finish()

    # Step 6: Local Windowing
    logger.info("\n🪟 Step 6: Local Windowing")
    run = init_wandb(run_name=f"step6-windowing-{pipeline_mode}")
    segmenter = WindowSegmenter(run)
    df = pd.read_parquet(f"data/flagged_{pipeline_mode}.parquet")
    segmenter.log_sample_windows(df, window_size=60, stride=30, num_samples=3)
    segmenter.save_windows_metadata(f"data/windows_metadata_{pipeline_mode}.csv")
    run.finish()

    # Step 7: Window-Level Baseline Cleaning
    logger.info("\n🧹 Step 7: Window-Level Baseline Cleaning")
    run = init_wandb(run_name=f"step7-baseline-cleaning-{pipeline_mode}")
    cleaner = WindowCleaner(run)
    df = pd.read_parquet(f"data/flagged_{pipeline_mode}.parquet")
    cleaned_windows, stats = cleaner.clean_baseline_windows(
        df, window_size=60, stride=30, cleaning_method='mad'
    )
    cleaner.save_models_and_stats(f"models/{pipeline_mode}")
    run.finish()

    # Step 8: Seasonal & Trend Decomposition
    logger.info("\n📈 Step 8: Seasonal & Trend Decomposition")
    run = init_wandb(run_name=f"step8-decomposition-{pipeline_mode}")
    decomposer = TimeSeriesDecomposer(run)
    df = pd.read_parquet(f"data/flagged_{pipeline_mode}.parquet")
    decompositions = decomposer.decompose_windows(
        df, window_size=720, stride=360, num_examples=3
    )
    decomposer.save_decomposition_artifacts(decompositions, f"features/{pipeline_mode}")
    run.finish()

    # Step 9: Window-Level Feature Extraction
    logger.info("\n🔍 Step 9: Window-Level Feature Extraction")
    run = init_wandb(run_name=f"step9-feature-extraction-{pipeline_mode}")
    extractor = FeatureExtractor(run)
    df = pd.read_parquet(f"data/flagged_{pipeline_mode}.parquet")
    feature_matrix = extractor.extract_window_features(df, window_size=60, stride=30)
    extractor.save_feature_matrix(feature_matrix, f"features/windows_features_{pipeline_mode}.csv")
    run.finish()

    # Step 10: Scaling & Normalization
    logger.info("\n⚖️ Step 10: Scaling & Normalization")
    run = init_wandb(run_name=f"step10-scaling-{pipeline_mode}")
    features = pd.read_csv(f"features/windows_features_{pipeline_mode}.csv")

    if pipeline_mode == "train":
        # Fit scaler on training data
        n_baseline = int(len(features) * 0.2)
        baseline_features = features.iloc[:n_baseline]
        scaler = FeatureScaler(run)
        scaler.fit_scaler(baseline_features, method='robust')
        scaled_features = scaler.apply_scaler(features)
        scaled_baseline = scaled_features.iloc[:n_baseline]
        scaler.validate_scaling(scaled_baseline, scaled_features)
        scaler.save_scaler("models/robust_scaler.joblib")
    else:
        # Load and apply existing scaler for test data
        import joblib
        scaler_data = joblib.load("models/robust_scaler.joblib")
        scaler = FeatureScaler(run)
        scaler.scaler = scaler_data['scaler']
        scaler.feature_names = scaler_data['feature_names']
        scaler.scaling_stats = scaler_data['scaling_stats']
        scaled_features = scaler.apply_scaler(features)

    scaled_features.to_csv(f"features/scaled_features_{pipeline_mode}.csv", index=False)
    artifact = wandb.Artifact(f"scaled-features-{pipeline_mode}", type="dataset")
    artifact.add_file(f"features/scaled_features_{pipeline_mode}.csv")
    run.log_artifact(artifact)
    run.finish()

    # Step 11: Dimensionality Reduction & Feature Selection
    logger.info("\n📐 Step 11: Dimensionality Reduction & Feature Selection")
    run = init_wandb(run_name=f"step11-feature-selection-{pipeline_mode}")
    features = pd.read_csv(f"features/scaled_features_{pipeline_mode}.csv")
    selector = FeatureSelector(run)

    if pipeline_mode == "train":
        # Fit selection models on training data
        reduced_features = selector.combine_selection_methods(
            features, variance_threshold=0.01, mi_top_k=100, pca_variance=0.95
        )
        selector.save_models("models")
    else:
        # Apply existing selection models for test data
        import joblib
        # Apply variance selector
        if os.path.exists("models/variance_selector.pkl"):
            selector.models['variance_selector'] = joblib.load("models/variance_selector.pkl")
        # Apply PCA
        if os.path.exists("models/pca.pkl"):
            selector.models['pca'] = joblib.load("models/pca.pkl")

        # Apply transformations
        feature_cols = [col for col in features.columns if col not in ['window_id', 'window_start']]
        if 'variance_selector' in selector.models:
            selected_data = selector.models['variance_selector'].transform(features[feature_cols])
            selected_features = [feature_cols[i] for i in range(len(feature_cols))
                               if selector.models['variance_selector'].variances_[i] > 0.01]
            features_var_filtered = pd.DataFrame(selected_data, columns=selected_features, index=features.index)
            for col in features.columns:
                if col not in feature_cols:
                    features_var_filtered[col] = features[col]
        else:
            features_var_filtered = features

        # Apply PCA
        if 'pca' in selector.models:
            from sklearn.impute import SimpleImputer
            imputer = SimpleImputer(strategy='median')
            feature_cols = [col for col in features_var_filtered.columns if col not in ['window_id', 'window_start']]
            features_clean = pd.DataFrame(
                imputer.fit_transform(features_var_filtered[feature_cols]),
                columns=feature_cols,
                index=features_var_filtered.index
            )
            pca_data = selector.models['pca'].transform(features_clean)
            pca_cols = [f'pca_{i}' for i in range(pca_data.shape[1])]
            reduced_features = pd.DataFrame(pca_data, columns=pca_cols, index=features_var_filtered.index)
            reduced_features['window_id'] = features_var_filtered['window_id']
            reduced_features['window_start'] = features_var_filtered['window_start']
        else:
            reduced_features = features_var_filtered

    reduced_features.to_csv(f"features/reduced_features_{pipeline_mode}.csv", index=False)
    artifact = wandb.Artifact(f"reduced-features-{pipeline_mode}", type="dataset")
    artifact.add_file(f"features/reduced_features_{pipeline_mode}.csv")
    run.log_artifact(artifact)
    run.finish()

    # Step 12: Sequence Assembly & Model Input
    logger.info("\n🔢 Step 12: Sequence Assembly & Model Input")
    run = init_wandb(run_name=f"step12-sequence-assembly-{pipeline_mode}")
    features = pd.read_csv(f"features/reduced_features_{pipeline_mode}.csv")
    assembler = SequenceAssembler(run)

    if pipeline_mode == "train":
        # For training, create only train/val split (no test)
        X_lstm, timestamps = assembler.create_sequences_lstm(features, sequence_length=10, overlap=0.5)
        X_tabular = assembler.create_tabular_format(features)

        # Modified split for training - only train and val
        n_samples = len(X_lstm)
        train_ratio = 0.8
        train_end = int(n_samples * train_ratio)

        lstm_splits = {
            'X_train': X_lstm[:train_end],
            'X_val': X_lstm[train_end:],
            'timestamps_train': timestamps[:train_end],
            'timestamps_val': timestamps[train_end:]
        }

        # Log split statistics
        run.log({
            "X_train_samples": len(lstm_splits['X_train']),
            "X_train_shape": str(lstm_splits['X_train'].shape),
            "X_val_samples": len(lstm_splits['X_val']),
            "X_val_shape": str(lstm_splits['X_val'].shape)
        })

    else:
        # For test, create full dataset for anomaly detection
        X_lstm, timestamps = assembler.create_sequences_lstm(features, sequence_length=10, overlap=0.5)
        lstm_splits = {
            'X_test': X_lstm,
            'timestamps_test': timestamps
        }

        run.log({
            "X_test_samples": len(lstm_splits['X_test']),
            "X_test_shape": str(lstm_splits['X_test'].shape)
        })

    assembler.save_sequences(lstm_splits, f"data/{pipeline_mode}")
    run.finish()

    # Step 13: Drift Monitoring (only for test data)
    if pipeline_mode == "test":
        logger.info("\n📊 Step 13: Drift Monitoring")
        run = init_wandb(run_name="step13-drift-monitoring")

        # Load train and test features for comparison
        train_features = pd.read_csv("features/reduced_features_train.csv")
        test_features = pd.read_csv("features/reduced_features_test.csv")

        monitor = DriftMonitor(run)
        drift_results = monitor.monitor_feature_drift(train_features, test_features)
        severity_results = monitor.evaluate_drift_severity(drift_results)
        monitor.create_drift_dashboard(drift_results, severity_results)
        retrain_needed = monitor.generate_retraining_sweep(severity_results)
        monitor.save_drift_report(drift_results, severity_results, "monitoring")
        run.finish()

    return f"data/{pipeline_mode}/model_inputs.h5"

def train_lstm_autoencoder(train_data_path: str):
    """
    Step 14: Train LSTM Autoencoder on stable training data
    """
    logger.info("\n🤖 Step 14: LSTM Autoencoder Training on Stable Data")
    run = init_wandb(run_name="step14-lstm-training")

    # Load sequences
    detector = AnomalyDetector(run)
    sequences = detector.load_sequences(train_data_path)

    # Get train/val splits
    X_train = sequences['X_train']
    X_val = sequences['X_val']

    logger.info(f"Training data shape: {X_train.shape}")
    logger.info(f"Validation data shape: {X_val.shape}")

    # Create and train model
    input_dim = X_train.shape[2]
    config = {
        'hidden_dim': 64,
        'num_layers': 2,
        'dropout': 0.2,
        'learning_rate': 0.001
    }

    detector.create_model(input_dim, config)
    detector.train_model(X_train, X_val, epochs=100, batch_size=32)

    # Set threshold based on validation data (stable baseline)
    detector.set_threshold(X_val, percentile=99)  # Higher percentile for stable data

    # Create training visualizations
    detector.create_training_visualizations()
    detector.create_model_performance_summary(X_train, X_val, X_val)  # Using val as pseudo-test

    # Save model
    import torch
    model_path = "models/lstm_autoencoder.pth"
    torch.save({
        'model_state_dict': detector.model.state_dict(),
        'threshold': detector.threshold,
        'training_history': detector.training_history,
        'config': config,
        'input_dim': input_dim
    }, model_path)

    # Log model as artifact
    model_artifact = wandb.Artifact("lstm-autoencoder-trained", type="model")
    model_artifact.add_file(model_path)
    model_artifact.metadata = {
        'threshold': float(detector.threshold),
        'input_dim': input_dim,
        'trained_on': 'stable_data'
    }
    run.log_artifact(model_artifact)

    logger.info(f"Model trained and saved. Threshold: {detector.threshold:.4f}")

    run.finish()
    return detector.threshold

def detect_anomalies_on_test(test_data_path: str, model_path: str, threshold: float):
    """
    Apply trained LSTM Autoencoder to detect anomalies in test data
    """
    logger.info("\n🔍 Step 14: Anomaly Detection on Test Data")
    run = init_wandb(run_name="step14-anomaly-detection")

    # Load sequences
    detector = AnomalyDetector(run)
    sequences = detector.load_sequences(test_data_path)
    X_test = sequences['X_test']
    timestamps_test = sequences.get('timestamps_test', None)

    logger.info(f"Test data shape: {X_test.shape}")

    # Load trained model
    import torch
    checkpoint = torch.load(model_path)

    # Recreate model with saved configuration
    detector.create_model(checkpoint['input_dim'], checkpoint['config'])
    detector.model.load_state_dict(checkpoint['model_state_dict'])
    detector.threshold = checkpoint['threshold']
    detector.training_history = checkpoint['training_history']

    # Detect anomalies
    results = detector.detect_anomalies(X_test, timestamps_test)

    # Create comprehensive visualizations
    detector.create_anomaly_visualizations(results, X_test)
    detector.create_predictions_table(results, num_samples=200)

    # Save results
    results_path = "models/anomaly_results.npz"
    np.savez(results_path,
            anomalies=results['anomalies'],
            reconstruction_errors=results['reconstruction_errors'],
            timestamps=results['timestamps'] if results['timestamps'] is not None else np.array([]))

    # Create results artifact
    results_artifact = wandb.Artifact("anomaly-detection-results", type="dataset")
    results_artifact.add_file(results_path)
    results_artifact.metadata = {
        'anomalies_detected': int(results['statistics']['anomalies_detected']),
        'anomaly_rate': float(results['statistics']['anomaly_rate']),
        'total_samples': int(results['statistics']['total_samples'])
    }
    run.log_artifact(results_artifact)

    logger.info(f"\n🎯 ANOMALY DETECTION COMPLETE")
    logger.info(f"   Total Anomalies Detected: {results['statistics']['anomalies_detected']}")
    logger.info(f"   Anomaly Rate: {results['statistics']['anomaly_rate']:.2%}")

    run.finish()
    return results

def run_complete_pipeline():
    """Execute the complete preprocessing and anomaly detection pipeline."""

    logger.info("🚀 Starting Predictive Maintenance Pipeline")
    pipeline_start = time.time()

    # Step 0: Setup W&B
    logger.info("\n📊 Step 0: Setting up Weights & Biases...")
    setup_wandb_colab()

    # Check for required input files
    if not os.path.exists('data/train.csv'):
        raise FileNotFoundError("train.csv not found in data/ directory. Please provide stable sensor data for training.")

    if not os.path.exists('data/test.csv'):
        raise FileNotFoundError("test.csv not found in data/ directory. Please provide test sensor data with anomalies.")

    # Phase 1: Process training data (stable baseline)
    logger.info("\n" + "="*80)
    logger.info("PHASE 1: PROCESSING TRAINING DATA (STABLE BASELINE)")
    logger.info("="*80)

    train_data_path = run_preprocessing_pipeline("data/train.csv", pipeline_mode="train")

    # Train LSTM Autoencoder
    threshold = train_lstm_autoencoder(train_data_path)

    # Phase 2: Process test data and detect anomalies
    logger.info("\n" + "="*80)
    logger.info("PHASE 2: PROCESSING TEST DATA AND DETECTING ANOMALIES")
    logger.info("="*80)

    test_data_path = run_preprocessing_pipeline("data/test.csv", pipeline_mode="test")

    # Detect anomalies
    results = detect_anomalies_on_test(test_data_path, "models/lstm_autoencoder.pth", threshold)

    # Step 15: Post-Detection Monitoring
    logger.info("\n📈 Step 15: Post-Detection Monitoring")
    run = init_wandb(run_name="step15-post-detection-monitoring")

    post_monitor = PostDetectionMonitor(run)
    post_monitor.analyze_reconstruction_errors(results)
    post_monitor.track_anomaly_patterns(results)
    post_monitor.create_temporal_analysis(results)
    post_monitor.generate_summary_report(results)

    run.finish()

    # Step 16: Pipeline Summary
    logger.info("\n📋 Step 16: Pipeline Summary & Orchestration")
    pipeline_time = time.time() - pipeline_start
    run = init_wandb(run_name="step16-pipeline-summary")

    # Log pipeline metrics
    run.log({
        "pipeline_duration_minutes": pipeline_time / 60,
        "pipeline_status": "completed",
        "total_steps": 16,
        "final_anomalies_detected": results['statistics']['anomalies_detected'],
        "final_anomaly_rate": results['statistics']['anomaly_rate'],
        "anomaly_threshold": threshold
    })

    # Create summary visualization
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(12, 10))

    steps = [
        "Data Ingestion (Train)", "Quality Checks (Train)", "Time Features (Train)",
        "Imputation (Train)", "Outlier Flagging (Train)", "Windowing (Train)",
        "Baseline Cleaning (Train)", "Decomposition (Train)", "Feature Extraction (Train)",
        "Scaling (Train)", "Feature Selection (Train)", "Sequence Assembly (Train)",
        "LSTM Training", "Data Ingestion (Test)", "Quality Checks (Test)",
        "Time Features (Test)", "Imputation (Test)", "Outlier Flagging (Test)",
        "Windowing (Test)", "Baseline Cleaning (Test)", "Decomposition (Test)",
        "Feature Extraction (Test)", "Scaling (Test)", "Feature Selection (Test)",
        "Sequence Assembly (Test)", "Drift Monitoring", "Anomaly Detection",
        "Post-Detection Monitoring", "Pipeline Summary"
    ]

    y_pos = np.arange(len(steps))
    ax.barh(y_pos, [1]*len(steps), color=['blue']*13 + ['green'] + ['orange']*12 + ['red'] + ['purple'] + ['gray'])
    ax.set_yticks(y_pos)
    ax.set_yticklabels(steps, fontsize=8)
    ax.set_xlabel('Status')
    ax.set_title('Pipeline Execution Status')
    ax.set_xlim(0, 1.2)

    for i, step in enumerate(steps):
        ax.text(1.05, i, '✓', fontsize=16, color='green', va='center')

    plt.tight_layout()
    run.log({"pipeline_status_chart": wandb.Image(fig)})
    plt.close()

    run.finish()

    logger.info(f"\n✨ Pipeline execution completed in {pipeline_time/60:.2f} minutes!")
    logger.info(f"🎯 Final Results: {results['statistics']['anomalies_detected']} anomalies detected")
    logger.info(f"📊 Anomaly Rate: {results['statistics']['anomaly_rate']:.2%}")
    logger.info(f"🎚️ Detection Threshold: {threshold:.4f}")
    logger.info("Check your W&B dashboard for detailed metrics and artifacts.")

    return results

# Execute pipeline
if __name__ == "__main__":
    # Create necessary directories
    os.makedirs('data', exist_ok=True)
    os.makedirs('data/train', exist_ok=True)
    os.makedirs('data/test', exist_ok=True)
    os.makedirs('models', exist_ok=True)
    os.makedirs('models/train', exist_ok=True)
    os.makedirs('models/test', exist_ok=True)
    os.makedirs('features', exist_ok=True)
    os.makedirs('features/train', exist_ok=True)
    os.makedirs('features/test', exist_ok=True)
    os.makedirs('configs', exist_ok=True)
    os.makedirs('monitoring', exist_ok=True)
    os.makedirs('artifacts', exist_ok=True)

    # Create default configs if they don't exist
    '''if not os.path.exists('configs/limits.json'):
        import json
        limits = {
            "sensor_0": {"min": 0, "max": 200},
            "sensor_1": {"min": 10, "max": 210},
            "sensor_2": {"min": 20, "max": 220},
            "sensor_3": {"min": 30, "max": 230},
            "sensor_4": {"min": 40, "max": 240}
        }
        with open('configs/limits.json', 'w') as f:
            json.dump(limits, f, indent=2)'''

    if not os.path.exists('configs/outlier_config.json'):
        import json
        outlier_config = {
            "zscore_threshold": 3.5,
            "iqr_multiplier": 1.5,
            "min_spike_duration": 1,
            "use_rolling_stats": True,
            "rolling_window": 60
        }
        with open('configs/outlier_config.json', 'w') as f:
            json.dump(outlier_config, f, indent=2)

    # Run the pipeline
    results = run_complete_pipeline()

In [ ]:
!chmod +x run_pipeline.py
!python run_pipeline.py

In [ ]:
'''%%writefile test/test_pipeline_steps.py
import pytest
import pandas as pd
import numpy as np
import wandb
from unittest.mock import Mock, patch

# Test data ingestion
def test_data_loader():
    """Test data loading and resampling."""
    from src.ingestion.loader import DataLoader

    # Mock W&B run
    mock_run = Mock()
    loader = DataLoader(mock_run)

    # Create test data
    dates = pd.date_range('2024-01-01', periods=100, freq='1T')
    df = pd.DataFrame({
        'timestamp': dates,
        'sensor_0': np.random.randn(100),
        'sensor_1': np.random.randn(100)
    })

    # Test resampling
    df_resampled = loader.resample_to_frequency(df.set_index('timestamp'), '5T')

    assert len(df_resampled) == 20  # 100 minutes / 5 minutes
    assert mock_run.log.called

# Test quality checks
def test_quality_checker():
    """Test quality checking functionality."""
    from src.cleaning.quality_checks import QualityChecker

    mock_run = Mock()
    checker = QualityChecker(mock_run)

    # Create test data with duplicates
    df = pd.DataFrame({
        'sensor_0': [1, 2, 2, 3, 4],
        'sensor_1': [5, 6, 7, 8, 9]
    }, index=pd.date_range('2024-01-01', periods=5, freq='1T'))

    # Duplicate the last row
    df = pd.concat([df, df.iloc[[-1]]])

    # Test duplicate removal
    df_clean = checker.remove_duplicates(df)
    assert len(df_clean) == 5

# Test feature extraction
def test_feature_extractor():
    """Test feature extraction from windows."""
    from src.features.extract import FeatureExtractor

    mock_run = Mock()
    extractor = FeatureExtractor(mock_run)

    # Create test window
    window = pd.DataFrame({
        'sensor_0': np.sin(np.linspace(0, 2*np.pi, 60)),
        'sensor_1': np.cos(np.linspace(0, 2*np.pi, 60))
    }, index=pd.date_range('2024-01-01', periods=60, freq='1T'))

    # Extract features
    features = extractor.extract_features(window)

    # Check feature extraction
    assert 'sensor_0_mean' in features
    assert 'sensor_0_std' in features
    assert 'sensor_0_dominant_freq' in features
    assert len(features) > 20  # Should have many features

# Test drift monitoring
def test_drift_monitor():
    """Test drift detection functionality."""
    from src.monitoring.drift import DriftMonitor

    mock_run = Mock()
    monitor = DriftMonitor(mock_run)

    # Create baseline and current data
    np.random.seed(42)
    baseline = np.random.normal(0, 1, 1000)
    current = np.random.normal(0.5, 1.2, 1000)  # Shifted distribution

    # Test PSI calculation
    psi = monitor.compute_psi(baseline, current)
    assert psi > 0.1  # Should detect some drift

    # Test KL divergence
    kl = monitor.compute_kl_divergence(baseline, current)
    assert kl > 0  # Should be positive

# Test W&B artifact logging
@patch('wandb.init')
@patch('wandb.Artifact')
def test_wandb_integration(mock_artifact, mock_init):
    """Test W&B artifact creation and logging."""
    from src.ingestion.loader import DataLoader

    mock_run = Mock()
    mock_init.return_value = mock_run

    loader = DataLoader(mock_run)

    # Check artifact creation
    df = pd.DataFrame({'a': [1, 2, 3]})
    loader.save_and_log_artifact(df, "test.parquet")

    assert mock_artifact.called
    assert mock_run.log_artifact.called

# Test pipeline configuration
def test_pipeline_config():
    """Test pipeline configuration loading."""
    from src.pipeline import DEFAULT_CONFIG

    assert 'window_size' in DEFAULT_CONFIG
    assert 'stride' in DEFAULT_CONFIG
    assert DEFAULT_CONFIG['window_size'] > 0
    assert DEFAULT_CONFIG['train_ratio'] + DEFAULT_CONFIG['val_ratio'] < 1.0

if __name__ == "__main__":
    pytest.main([__file__, "-v"])'''

In [ ]:
#!pytest -q tests/